已连接到 base (Python 3.9.7)

In [ ]:
import os
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
from analyzing_tools import Analyzing_Tools
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


def SMA_H(target_assets, paths,window_1=61,window_2=101):
    #信号结果字典
    results = {}
    #全数据字典，包含计算指标用于检查
    full_info={}
    
    #编写策略主体部分
    for code in target_assets:
        # 读取数据
        daily_data = pd.read_csv(os.path.join(paths['daily'], f"{code}.csv"), index_col=[0])
        daily_data.index = pd.to_datetime(daily_data.index)
        df=daily_data.copy()
        hourly_data = pd.read_csv(os.path.join(paths['hourly'], f"{code}.csv"), index_col=[0])
        hourly_data.index = pd.to_datetime(hourly_data.index)

        hourly_data["var_1"] = hourly_data['close'].rolling(window_1).mean()
        hourly_data["var_2"] =hourly_data['close'].rolling(window_2).mean()
        # 添加信号列
        hourly_data.loc[(hourly_data["var_1"].shift(1) <= hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] >= hourly_data["var_2"]) , 'signal'] = 1
        hourly_data.loc[(hourly_data["var_1"].shift(1) > hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] < hourly_data["var_2"]) , 'signal'] = -1
        
        hourly_data['signal'].fillna(method='ffill', inplace=True)
        hourly_exchange = hourly_data.resample('D').last()

        df = pd.merge(df, hourly_exchange[['signal']], left_index=True, right_index=True, how='left')        
        df['signal'].fillna(method='ffill', inplace=True)


        result=df

        # 将信号合并回每日数据
        daily_data = daily_data.join(result[['signal']], how='left')
        daily_data[['signal']].fillna(0, inplace=True)
        daily_data=daily_data.dropna()

        # 存储结果
        results[code] = daily_data
        full_info[code]=result

    return results,full_info

# 自定义数据类，包含 'signal'
class PandasDataPlusSignal(bt.feeds.PandasData):
    lines = ('signal',)
    params = (
        ('signal', -1),  # 默认情况下，'signal' 列在最后一列   
    )

# 策略类，包含调试信息和导出方法
class SMA_H_Strategy(bt.Strategy):
    params = (
        ('size_pct',0.19),  # 每个资产的仓位百分比
    )

    def __init__(self):
        self.orders = {}         # 用于跟踪每个资产的订单状态
        self.trade_counts = {}   # 记录每个资产的交易次数
        self.value = []          # 存储组合总净值
        self.dates = []          # 存储日期序列
        self.debug_info = []     # 存储调试信息

        for data in self.datas:
            name = data._name
            self.trade_counts[name] = 0
            self.orders[name] = None

    def next(self):
        total_value = self.broker.getvalue()
        self.value.append(total_value)
        current_date = self.datas[0].datetime.datetime(0)
        self.dates.append(current_date)

        for data in self.datas:
            name = data._name
            position_size = self.getposition(data).size
            signal = data.signal[0]

            # 根据信号执行交易
            if signal == 1 and position_size == 0:
                size = self.calculate_position_size(data)
                self.orders[name] = self.buy(data=data, size=size)
                self.trade_counts[name] += 1

            elif signal == -1 and position_size > 0:
                self.orders[name] = self.close(data=data)
                self.trade_counts[name] += 1


            # 存储调试信息
            self.debug_info.append({
                'Date': current_date,
                'Asset': name,
                'Position': position_size,
                'Signal': signal,
                'Size': self.calculate_position_size(data),
                'Open':data.open[0],
                'High':data.high[0],
                'Low':data.low[0],
                'Volume':data.volume[0],
                'Close': data.close[0],
                'Cash': self.broker.getcash(),
                'Value': total_value,
                'Trades': self.trade_counts[name],
            })


    def calculate_position_size(self, data):
        """
        计算仓位大小
        """
        available_cash = self.broker.getvalue()
        current_price = data.close[0]
        max_investment = available_cash * self.params.size_pct
        max_shares = int(max_investment / current_price)
        return max_shares

    def notify_order(self, order):
        """
        订单完成后重置状态
        """
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            name = order.data._name
            self.orders[name] = None

    def get_net_value_series(self):
        """
        返回净值序列，用于后续分析
        """
        return pd.DataFrame(self.value, index=self.dates)

    def get_debug_df(self):
        """
        返回包含调试信息的DataFrame
        """
        df = pd.DataFrame(self.debug_info)
        df.set_index('Date', inplace=True)
        return df


def run_backtest(strategy, target_assets, strategy_results, cash=100000.0, commission=0.0005, slippage_perc=0.0005, slippage_fixed=None, **kwargs):
    
    cerebro = bt.Cerebro()  # 初始化Cerebro引擎
    cerebro.addstrategy(strategy, **kwargs)  # 添加策略
    
    for code in target_assets:
        data = PandasDataPlusSignal(dataname=strategy_results[code])
        data._name = code  # 为数据设置名称，便于识别
        cerebro.adddata(data)
    
    # 使用setcommission设置股票模式的佣金
    cerebro.broker.setcommission(
        commission=commission,  # 佣金百分比
        stocklike=True  # 将交易设置为股票模式
    )
    
    cerebro.broker.setcash(cash)  # 设置初始资金

    # 设置滑点
    if slippage_perc is not None:
        cerebro.broker.set_slippage_perc(slippage_perc)  # 设置百分比滑点
    elif slippage_fixed is not None:
        cerebro.broker.set_slippage_fixed(slippage_fixed)  # 设置固定点滑点
    
    strategies = cerebro.run()  # 运行回测
    return strategies[0]



#加载分析工具
AT=Analyzing_Tools()


# 定义数据路径
paths = {
    'daily': r'E:\数据库同花顺ETF跟踪指数量价数据\1d',
    'hourly': r'E:\数据库同花顺ETF跟踪指数量价数据\1h',
    'min15': r'E:\数据库同花顺ETF跟踪指数量价数据\15min',
    'pv_export':r"E:\量化交易构建\私募基金研究\股票策略研究\策略净值序列"
}


# 资产列表
target_assets = [
    "000300.SH",
    "000852.SH",
    "000905.SH",
    "399006.SZ",
    "399303.SZ"
]



# 生成信号
strategy_results,full_info = SMA_H(target_assets, paths)


# 获取策略实例
strat = run_backtest(SMA_H_Strategy,target_assets,strategy_results,10000000,0.0005,0.0005)

pv=strat.get_net_value_series()

strtegy_name='SMA_H'

pv.to_excel(paths["pv_export"]+'\\'+strtegy_name+'.xlsx')

portfolio_value, returns, drawdown_ts, metrics = AT.performance_analysis(pv, freq='D')

index_price_path=paths['daily']

# 获取净值序列
AT.plot_results('000906.SH',index_price_path,portfolio_value, drawdown_ts, returns, metrics)

# 获取调试信息
debug_df = strat.get_debug_df()

#蒙特卡洛测试

AT.monte_carlo_analysis(strat,num_simulations=10000,num_days=252,freq='D')



#定义参数优化函数
def parameter_optimization(parameter_grid, strategy_function, strategy_class, target_assets, paths, cash=100000.0, commission=0.0002, slippage_perc=0.0005, metric='sharpe_ratio'):   
    """
    执行参数优化，支持一个或两个参数。

    参数：
    - parameter_grid: 字典，包含参数名称和要测试的取值列表。例如：{'window_1': [30, 34, 38]}
    - strategy_function: 生成信号的策略函数，例如 UDVD
    - strategy_class: Backtrader 策略类，例如 UDVD_Strategy
    - target_assets: 资产列表
    - paths: 数据路径字典
    - cash: 初始资金
    - commission: 佣金
    - slippage_perc: 滑点百分比
    - metric: 选择用于评估的绩效指标，默认为 'sharpe_ratio'
    """

    #加载数据类

    # 获取参数名称和取值列表
    param_names = list(parameter_grid.keys())
    param_values = [parameter_grid[key] for key in param_names]

    # 生成所有参数组合
    param_combinations = [dict(zip(param_names, values)) for values in product(*param_values)]

    results = []

    for params in param_combinations:
        print(f"正在测试参数组合：{params}")
        # 生成当前参数下的信号
        strategy_results, full_info = strategy_function(target_assets, paths, **params)

        # 运行回测
        strat = run_backtest(strategy_class, target_assets, strategy_results, cash, commission, slippage_perc)

        # 获取净值序列
        pv = strat.get_net_value_series()

        # 计算绩效指标
        portfolio_value, returns, drawdown_ts, metrics =AT.performance_analysis(pv)

        # 收集指标和参数
        result_entry = {k: v for k, v in params.items()}
        result_entry.update(metrics)
        result_entry=pd.DataFrame(result_entry)
        results.append(result_entry)

    # 将结果转换为 DataFrame
    results_df = pd.concat(results,axis=0)
    results_df=results_df.dropna()

    # 可视化结果
    if len(param_names) == 1:
        # 绘制参数与绩效指标的关系曲线
        param = param_names[0]
        plt.figure(figsize=(10, 6))
        plt.plot(results_df[param], results_df[metric], marker='o')
        plt.xlabel(param)
        plt.ylabel(metric)
        plt.title(f'{metric} vs {param}')
        plt.grid(True)
        plt.show()
    elif len(param_names) == 2:
        # 绘制热力图
        param1 = param_names[0]
        param2 = param_names[1]
        pivot_table = results_df.pivot(index=param1, columns=param2, values=metric)

        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
        plt.title(f'{metric} Heatmap')
        plt.ylabel(param1)
        plt.xlabel(param2)
        plt.show()
    else:
        print("无法可视化超过两个参数的结果，请减少参数数量。")

    # 返回结果 DataFrame
    return results_df

# 定义参数网格
parameter_grid = {
    'window_1': range(50, 80,5),
    'window_2': range(80, 110, 5)
}

# 运行参数优化
# results_df = parameter_optimization(
#     parameter_grid=parameter_grid,
#     strategy_function=SMA_H,
#     strategy_class=SMA_H_Strategy,
#     target_assets=target_assets,
#     paths=paths,
#     cash=10000000,
#     commission=0.0002,
#     slippage_perc=0.0005,
#     metric='sharpe_ratio'
# )

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\数据库同花顺ETF跟踪指数量价数据\\1d\\000300.SH.csv'

In [ ]:
import os
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
from analyzing_tools import Analyzing_Tools
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


def SMA_H(target_assets, paths,window_1=61,window_2=101):
    #信号结果字典
    results = {}
    #全数据字典，包含计算指标用于检查
    full_info={}
    
    #编写策略主体部分
    for code in target_assets:
        # 读取数据
        daily_data = pd.read_csv(os.path.join(paths['daily'], f"{code}.csv"), index_col=[0])
        daily_data.index = pd.to_datetime(daily_data.index)
        df=daily_data.copy()
        hourly_data = pd.read_csv(os.path.join(paths['hourly'], f"{code}.csv"), index_col=[0])
        hourly_data.index = pd.to_datetime(hourly_data.index)

        hourly_data["var_1"] = hourly_data['close'].rolling(window_1).mean()
        hourly_data["var_2"] =hourly_data['close'].rolling(window_2).mean()
        # 添加信号列
        hourly_data.loc[(hourly_data["var_1"].shift(1) <= hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] >= hourly_data["var_2"]) , 'signal'] = 1
        hourly_data.loc[(hourly_data["var_1"].shift(1) > hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] < hourly_data["var_2"]) , 'signal'] = -1
        
        hourly_data['signal'].fillna(method='ffill', inplace=True)
        hourly_exchange = hourly_data.resample('D').last()

        df = pd.merge(df, hourly_exchange[['signal']], left_index=True, right_index=True, how='left')        
        df['signal'].fillna(method='ffill', inplace=True)


        result=df

        # 将信号合并回每日数据
        daily_data = daily_data.join(result[['signal']], how='left')
        daily_data[['signal']].fillna(0, inplace=True)
        daily_data=daily_data.dropna()

        # 存储结果
        results[code] = daily_data
        full_info[code]=result

    return results,full_info

# 自定义数据类，包含 'signal'
class PandasDataPlusSignal(bt.feeds.PandasData):
    lines = ('signal',)
    params = (
        ('signal', -1),  # 默认情况下，'signal' 列在最后一列   
    )

# 策略类，包含调试信息和导出方法
class SMA_H_Strategy(bt.Strategy):
    params = (
        ('size_pct',0.19),  # 每个资产的仓位百分比
    )

    def __init__(self):
        self.orders = {}         # 用于跟踪每个资产的订单状态
        self.trade_counts = {}   # 记录每个资产的交易次数
        self.value = []          # 存储组合总净值
        self.dates = []          # 存储日期序列
        self.debug_info = []     # 存储调试信息

        for data in self.datas:
            name = data._name
            self.trade_counts[name] = 0
            self.orders[name] = None

    def next(self):
        total_value = self.broker.getvalue()
        self.value.append(total_value)
        current_date = self.datas[0].datetime.datetime(0)
        self.dates.append(current_date)

        for data in self.datas:
            name = data._name
            position_size = self.getposition(data).size
            signal = data.signal[0]

            # 根据信号执行交易
            if signal == 1 and position_size == 0:
                size = self.calculate_position_size(data)
                self.orders[name] = self.buy(data=data, size=size)
                self.trade_counts[name] += 1

            elif signal == -1 and position_size > 0:
                self.orders[name] = self.close(data=data)
                self.trade_counts[name] += 1


            # 存储调试信息
            self.debug_info.append({
                'Date': current_date,
                'Asset': name,
                'Position': position_size,
                'Signal': signal,
                'Size': self.calculate_position_size(data),
                'Open':data.open[0],
                'High':data.high[0],
                'Low':data.low[0],
                'Volume':data.volume[0],
                'Close': data.close[0],
                'Cash': self.broker.getcash(),
                'Value': total_value,
                'Trades': self.trade_counts[name],
            })


    def calculate_position_size(self, data):
        """
        计算仓位大小
        """
        available_cash = self.broker.getvalue()
        current_price = data.close[0]
        max_investment = available_cash * self.params.size_pct
        max_shares = int(max_investment / current_price)
        return max_shares

    def notify_order(self, order):
        """
        订单完成后重置状态
        """
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            name = order.data._name
            self.orders[name] = None

    def get_net_value_series(self):
        """
        返回净值序列，用于后续分析
        """
        return pd.DataFrame(self.value, index=self.dates)

    def get_debug_df(self):
        """
        返回包含调试信息的DataFrame
        """
        df = pd.DataFrame(self.debug_info)
        df.set_index('Date', inplace=True)
        return df


def run_backtest(strategy, target_assets, strategy_results, cash=100000.0, commission=0.0005, slippage_perc=0.0005, slippage_fixed=None, **kwargs):
    
    cerebro = bt.Cerebro()  # 初始化Cerebro引擎
    cerebro.addstrategy(strategy, **kwargs)  # 添加策略
    
    for code in target_assets:
        data = PandasDataPlusSignal(dataname=strategy_results[code])
        data._name = code  # 为数据设置名称，便于识别
        cerebro.adddata(data)
    
    # 使用setcommission设置股票模式的佣金
    cerebro.broker.setcommission(
        commission=commission,  # 佣金百分比
        stocklike=True  # 将交易设置为股票模式
    )
    
    cerebro.broker.setcash(cash)  # 设置初始资金

    # 设置滑点
    if slippage_perc is not None:
        cerebro.broker.set_slippage_perc(slippage_perc)  # 设置百分比滑点
    elif slippage_fixed is not None:
        cerebro.broker.set_slippage_fixed(slippage_fixed)  # 设置固定点滑点
    
    strategies = cerebro.run()  # 运行回测
    return strategies[0]



#加载分析工具
AT=Analyzing_Tools()


# 定义数据路径
paths = {
    'daily': r'E:\数据库\同花顺ETF跟踪指数量价数据\1d',
    'hourly': r'E:\数据库\同花顺ETF跟踪指数量价数据\1h',
    'min15': r'E:\数据库\同花顺ETF跟踪指数量价数据\15min',
    'pv_export':r"E:\量化交易构建\私募基金研究\股票策略研究\策略净值序列"
}


# 资产列表
target_assets = [
    "000300.SH",
    "000852.SH",
    "000905.SH",
    "399006.SZ",
    "399303.SZ"
]



# 生成信号
strategy_results,full_info = SMA_H(target_assets, paths)


# 获取策略实例
strat = run_backtest(SMA_H_Strategy,target_assets,strategy_results,10000000,0.0005,0.0005)

pv=strat.get_net_value_series()

strtegy_name='SMA_H'

pv.to_excel(paths["pv_export"]+'\\'+strtegy_name+'.xlsx')

portfolio_value, returns, drawdown_ts, metrics = AT.performance_analysis(pv, freq='D')

index_price_path=paths['daily']

# 获取净值序列
AT.plot_results('000906.SH',index_price_path,portfolio_value, drawdown_ts, returns, metrics)

# 获取调试信息
debug_df = strat.get_debug_df()

#蒙特卡洛测试

AT.monte_carlo_analysis(strat,num_simulations=10000,num_days=252,freq='D')



#定义参数优化函数
def parameter_optimization(parameter_grid, strategy_function, strategy_class, target_assets, paths, cash=100000.0, commission=0.0002, slippage_perc=0.0005, metric='sharpe_ratio'):   
    """
    执行参数优化，支持一个或两个参数。

    参数：
    - parameter_grid: 字典，包含参数名称和要测试的取值列表。例如：{'window_1': [30, 34, 38]}
    - strategy_function: 生成信号的策略函数，例如 UDVD
    - strategy_class: Backtrader 策略类，例如 UDVD_Strategy
    - target_assets: 资产列表
    - paths: 数据路径字典
    - cash: 初始资金
    - commission: 佣金
    - slippage_perc: 滑点百分比
    - metric: 选择用于评估的绩效指标，默认为 'sharpe_ratio'
    """

    #加载数据类

    # 获取参数名称和取值列表
    param_names = list(parameter_grid.keys())
    param_values = [parameter_grid[key] for key in param_names]

    # 生成所有参数组合
    param_combinations = [dict(zip(param_names, values)) for values in product(*param_values)]

    results = []

    for params in param_combinations:
        print(f"正在测试参数组合：{params}")
        # 生成当前参数下的信号
        strategy_results, full_info = strategy_function(target_assets, paths, **params)

        # 运行回测
        strat = run_backtest(strategy_class, target_assets, strategy_results, cash, commission, slippage_perc)

        # 获取净值序列
        pv = strat.get_net_value_series()

        # 计算绩效指标
        portfolio_value, returns, drawdown_ts, metrics =AT.performance_analysis(pv)

        # 收集指标和参数
        result_entry = {k: v for k, v in params.items()}
        result_entry.update(metrics)
        result_entry=pd.DataFrame(result_entry)
        results.append(result_entry)

    # 将结果转换为 DataFrame
    results_df = pd.concat(results,axis=0)
    results_df=results_df.dropna()

    # 可视化结果
    if len(param_names) == 1:
        # 绘制参数与绩效指标的关系曲线
        param = param_names[0]
        plt.figure(figsize=(10, 6))
        plt.plot(results_df[param], results_df[metric], marker='o')
        plt.xlabel(param)
        plt.ylabel(metric)
        plt.title(f'{metric} vs {param}')
        plt.grid(True)
        plt.show()
    elif len(param_names) == 2:
        # 绘制热力图
        param1 = param_names[0]
        param2 = param_names[1]
        pivot_table = results_df.pivot(index=param1, columns=param2, values=metric)

        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
        plt.title(f'{metric} Heatmap')
        plt.ylabel(param1)
        plt.xlabel(param2)
        plt.show()
    else:
        print("无法可视化超过两个参数的结果，请减少参数数量。")

    # 返回结果 DataFrame
    return results_df

# 定义参数网格
parameter_grid = {
    'window_1': range(50, 80,5),
    'window_2': range(80, 110, 5)
}

# 运行参数优化
# results_df = parameter_optimization(
#     parameter_grid=parameter_grid,
#     strategy_function=SMA_H,
#     strategy_class=SMA_H_Strategy,
#     target_assets=target_assets,
#     paths=paths,
#     cash=10000000,
#     commission=0.0002,
#     slippage_perc=0.0005,
#     metric='sharpe_ratio'
# )

d:\Anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [ ]:
import os
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
from analyzing_tools import Analyzing_Tools
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


def SMA_H(target_assets, paths,window_1=61,window_2=101):
    #信号结果字典
    results = {}
    #全数据字典，包含计算指标用于检查
    full_info={}
    
    #编写策略主体部分
    for code in target_assets:
        # 读取数据
        daily_data = pd.read_csv(os.path.join(paths['daily'], f"{code}.csv"), index_col=[0])
        daily_data.index = pd.to_datetime(daily_data.index)
        df=daily_data.copy()
        hourly_data = pd.read_csv(os.path.join(paths['hourly'], f"{code}.csv"), index_col=[0])
        hourly_data.index = pd.to_datetime(hourly_data.index)

        hourly_data["var_1"] = hourly_data['close'].rolling(window_1).mean()
        hourly_data["var_2"] =hourly_data['close'].rolling(window_2).mean()
        # 添加信号列
        hourly_data.loc[(hourly_data["var_1"].shift(1) <= hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] >= hourly_data["var_2"]) , 'signal'] = 1
        hourly_data.loc[(hourly_data["var_1"].shift(1) > hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] < hourly_data["var_2"]) , 'signal'] = -1
        
        hourly_data['signal'].fillna(method='ffill', inplace=True)
        hourly_exchange = hourly_data.resample('D').last()

        df = pd.merge(df, hourly_exchange[['signal']], left_index=True, right_index=True, how='left')        
        df['signal'].fillna(method='ffill', inplace=True)


        result=df

        # 将信号合并回每日数据
        daily_data = daily_data.join(result[['signal']], how='left')
        daily_data[['signal']].fillna(0, inplace=True)
        daily_data=daily_data.dropna()

        # 存储结果
        results[code] = daily_data
        full_info[code]=result

    return results,full_info

# 自定义数据类，包含 'signal'
class PandasDataPlusSignal(bt.feeds.PandasData):
    lines = ('signal',)
    params = (
        ('signal', -1),  # 默认情况下，'signal' 列在最后一列   
    )

# 策略类，包含调试信息和导出方法
class SMA_H_Strategy(bt.Strategy):
    params = (
        ('size_pct',0.19),  # 每个资产的仓位百分比
    )

    def __init__(self):
        self.orders = {}         # 用于跟踪每个资产的订单状态
        self.trade_counts = {}   # 记录每个资产的交易次数
        self.value = []          # 存储组合总净值
        self.dates = []          # 存储日期序列
        self.debug_info = []     # 存储调试信息

        for data in self.datas:
            name = data._name
            self.trade_counts[name] = 0
            self.orders[name] = None

    def next(self):
        total_value = self.broker.getvalue()
        self.value.append(total_value)
        current_date = self.datas[0].datetime.datetime(0)
        self.dates.append(current_date)

        for data in self.datas:
            name = data._name
            position_size = self.getposition(data).size
            signal = data.signal[0]

            # 根据信号执行交易
            if signal == 1 and position_size == 0:
                size = self.calculate_position_size(data)
                self.orders[name] = self.buy(data=data, size=size)
                self.trade_counts[name] += 1

            elif signal == -1 and position_size > 0:
                self.orders[name] = self.close(data=data)
                self.trade_counts[name] += 1


            # 存储调试信息
            self.debug_info.append({
                'Date': current_date,
                'Asset': name,
                'Position': position_size,
                'Signal': signal,
                'Size': self.calculate_position_size(data),
                'Open':data.open[0],
                'High':data.high[0],
                'Low':data.low[0],
                'Volume':data.volume[0],
                'Close': data.close[0],
                'Cash': self.broker.getcash(),
                'Value': total_value,
                'Trades': self.trade_counts[name],
            })


    def calculate_position_size(self, data):
        """
        计算仓位大小
        """
        available_cash = self.broker.getvalue()
        current_price = data.close[0]
        max_investment = available_cash * self.params.size_pct
        max_shares = int(max_investment / current_price)
        return max_shares

    def notify_order(self, order):
        """
        订单完成后重置状态
        """
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            name = order.data._name
            self.orders[name] = None

    def get_net_value_series(self):
        """
        返回净值序列，用于后续分析
        """
        return pd.DataFrame(self.value, index=self.dates)

    def get_debug_df(self):
        """
        返回包含调试信息的DataFrame
        """
        df = pd.DataFrame(self.debug_info)
        df.set_index('Date', inplace=True)
        return df


def run_backtest(strategy, target_assets, strategy_results, cash=100000.0, commission=0.0005, slippage_perc=0.0005, slippage_fixed=None, **kwargs):
    
    cerebro = bt.Cerebro()  # 初始化Cerebro引擎
    cerebro.addstrategy(strategy, **kwargs)  # 添加策略
    
    for code in target_assets:
        data = PandasDataPlusSignal(dataname=strategy_results[code])
        data._name = code  # 为数据设置名称，便于识别
        cerebro.adddata(data)
    
    # 使用setcommission设置股票模式的佣金
    cerebro.broker.setcommission(
        commission=commission,  # 佣金百分比
        stocklike=True  # 将交易设置为股票模式
    )
    
    cerebro.broker.setcash(cash)  # 设置初始资金

    # 设置滑点
    if slippage_perc is not None:
        cerebro.broker.set_slippage_perc(slippage_perc)  # 设置百分比滑点
    elif slippage_fixed is not None:
        cerebro.broker.set_slippage_fixed(slippage_fixed)  # 设置固定点滑点
    
    strategies = cerebro.run()  # 运行回测
    return strategies[0]



#加载分析工具
AT=Analyzing_Tools()


# 定义数据路径
paths = {
    'daily': r'E:\数据库\同花顺ETF跟踪指数量价数据\1d',
    'hourly': r'E:\数据库\同花顺ETF跟踪指数量价数据\1h',
    'min15': r'E:\数据库\同花顺ETF跟踪指数量价数据\15min',
    'pv_export':r"E:\量化交易构建\私募基金研究\股票策略研究\策略净值序列"
}


# 资产列表
target_assets = [
    "000300.SH",
    "000852.SH",
    "000905.SH",
    "399006.SZ",
    "399303.SZ"
]



# 生成信号
strategy_results,full_info = SMA_H(target_assets, paths)


# 获取策略实例
strat = run_backtest(SMA_H_Strategy,target_assets,strategy_results,10000000,0.0005,0.0005)

pv=strat.get_net_value_series()

strtegy_name='SMA_H'

pv.to_excel(paths["pv_export"]+'\\'+strtegy_name+'.xlsx')

portfolio_value, returns, drawdown_ts, metrics = AT.performance_analysis(pv, freq='D')

index_price_path=paths['daily']

# 获取净值序列
AT.plot_results('000906.SH',index_price_path,portfolio_value, drawdown_ts, returns, metrics)

# 获取调试信息
debug_df = strat.get_debug_df()

#蒙特卡洛测试

AT.monte_carlo_analysis(strat,num_simulations=10000,num_days=252,freq='D')



#定义参数优化函数
def parameter_optimization(parameter_grid, strategy_function, strategy_class, target_assets, paths, cash=100000.0, commission=0.0002, slippage_perc=0.0005, metric='sharpe_ratio'):   
    """
    执行参数优化，支持一个或两个参数。

    参数：
    - parameter_grid: 字典，包含参数名称和要测试的取值列表。例如：{'window_1': [30, 34, 38]}
    - strategy_function: 生成信号的策略函数，例如 UDVD
    - strategy_class: Backtrader 策略类，例如 UDVD_Strategy
    - target_assets: 资产列表
    - paths: 数据路径字典
    - cash: 初始资金
    - commission: 佣金
    - slippage_perc: 滑点百分比
    - metric: 选择用于评估的绩效指标，默认为 'sharpe_ratio'
    """

    #加载数据类

    # 获取参数名称和取值列表
    param_names = list(parameter_grid.keys())
    param_values = [parameter_grid[key] for key in param_names]

    # 生成所有参数组合
    param_combinations = [dict(zip(param_names, values)) for values in product(*param_values)]

    results = []

    for params in param_combinations:
        print(f"正在测试参数组合：{params}")
        # 生成当前参数下的信号
        strategy_results, full_info = strategy_function(target_assets, paths, **params)

        # 运行回测
        strat = run_backtest(strategy_class, target_assets, strategy_results, cash, commission, slippage_perc)

        # 获取净值序列
        pv = strat.get_net_value_series()

        # 计算绩效指标
        portfolio_value, returns, drawdown_ts, metrics =AT.performance_analysis(pv)

        # 收集指标和参数
        result_entry = {k: v for k, v in params.items()}
        result_entry.update(metrics)
        result_entry=pd.DataFrame(result_entry)
        results.append(result_entry)

    # 将结果转换为 DataFrame
    results_df = pd.concat(results,axis=0)
    results_df=results_df.dropna()

    # 可视化结果
    if len(param_names) == 1:
        # 绘制参数与绩效指标的关系曲线
        param = param_names[0]
        plt.figure(figsize=(10, 6))
        plt.plot(results_df[param], results_df[metric], marker='o')
        plt.xlabel(param)
        plt.ylabel(metric)
        plt.title(f'{metric} vs {param}')
        plt.grid(True)
        plt.show()
    elif len(param_names) == 2:
        # 绘制热力图
        param1 = param_names[0]
        param2 = param_names[1]
        pivot_table = results_df.pivot(index=param1, columns=param2, values=metric)

        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
        plt.title(f'{metric} Heatmap')
        plt.ylabel(param1)
        plt.xlabel(param2)
        plt.show()
    else:
        print("无法可视化超过两个参数的结果，请减少参数数量。")

    # 返回结果 DataFrame
    return results_df

# 定义参数网格
parameter_grid = {
    'window_1': range(50, 80,5),
    'window_2': range(80, 110, 5)
}

# 运行参数优化
# results_df = parameter_optimization(
#     parameter_grid=parameter_grid,
#     strategy_function=SMA_H,
#     strategy_class=SMA_H_Strategy,
#     target_assets=target_assets,
#     paths=paths,
#     cash=10000000,
#     commission=0.0002,
#     slippage_perc=0.0005,
#     metric='sharpe_ratio'
# )

d:\Anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [ ]:
import os
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
from analyzing_tools import Analyzing_Tools
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


def SMA_H(target_assets, paths,window_1=61,window_2=101):
    #信号结果字典
    results = {}
    #全数据字典，包含计算指标用于检查
    full_info={}
    
    #编写策略主体部分
    for code in target_assets:
        # 读取数据
        daily_data = pd.read_csv(os.path.join(paths['daily'], f"{code}.csv"), index_col=[0])
        daily_data.index = pd.to_datetime(daily_data.index)
        df=daily_data.copy()
        hourly_data = pd.read_csv(os.path.join(paths['hourly'], f"{code}.csv"), index_col=[0])
        hourly_data.index = pd.to_datetime(hourly_data.index)

        hourly_data["var_1"] = hourly_data['close'].rolling(window_1).mean()
        hourly_data["var_2"] =hourly_data['close'].rolling(window_2).mean()
        # 添加信号列
        hourly_data.loc[(hourly_data["var_1"].shift(1) <= hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] >= hourly_data["var_2"]) , 'signal'] = 1
        hourly_data.loc[(hourly_data["var_1"].shift(1) > hourly_data["var_2"].shift(1)) & (hourly_data["var_1"] < hourly_data["var_2"]) , 'signal'] = -1
        
        hourly_data['signal'].fillna(method='ffill', inplace=True)
        hourly_exchange = hourly_data.resample('D').last()

        df = pd.merge(df, hourly_exchange[['signal']], left_index=True, right_index=True, how='left')        
        df['signal'].fillna(method='ffill', inplace=True)


        result=df

        # 将信号合并回每日数据
        daily_data = daily_data.join(result[['signal']], how='left')
        daily_data[['signal']].fillna(0, inplace=True)
        daily_data=daily_data.dropna()

        # 存储结果
        results[code] = daily_data
        full_info[code]=result

    return results,full_info

# 自定义数据类，包含 'signal'
class PandasDataPlusSignal(bt.feeds.PandasData):
    lines = ('signal',)
    params = (
        ('signal', -1),  # 默认情况下，'signal' 列在最后一列   
    )

# 策略类，包含调试信息和导出方法
class SMA_H_Strategy(bt.Strategy):
    params = (
        ('size_pct',0.19),  # 每个资产的仓位百分比
    )

    def __init__(self):
        self.orders = {}         # 用于跟踪每个资产的订单状态
        self.trade_counts = {}   # 记录每个资产的交易次数
        self.value = []          # 存储组合总净值
        self.dates = []          # 存储日期序列
        self.debug_info = []     # 存储调试信息

        for data in self.datas:
            name = data._name
            self.trade_counts[name] = 0
            self.orders[name] = None

    def next(self):
        total_value = self.broker.getvalue()
        self.value.append(total_value)
        current_date = self.datas[0].datetime.datetime(0)
        self.dates.append(current_date)

        for data in self.datas:
            name = data._name
            position_size = self.getposition(data).size
            signal = data.signal[0]

            # 根据信号执行交易
            if signal == 1 and position_size == 0:
                size = self.calculate_position_size(data)
                self.orders[name] = self.buy(data=data, size=size)
                self.trade_counts[name] += 1

            elif signal == -1 and position_size > 0:
                self.orders[name] = self.close(data=data)
                self.trade_counts[name] += 1


            # 存储调试信息
            self.debug_info.append({
                'Date': current_date,
                'Asset': name,
                'Position': position_size,
                'Signal': signal,
                'Size': self.calculate_position_size(data),
                'Open':data.open[0],
                'High':data.high[0],
                'Low':data.low[0],
                'Volume':data.volume[0],
                'Close': data.close[0],
                'Cash': self.broker.getcash(),
                'Value': total_value,
                'Trades': self.trade_counts[name],
            })


    def calculate_position_size(self, data):
        """
        计算仓位大小
        """
        available_cash = self.broker.getvalue()
        current_price = data.close[0]
        max_investment = available_cash * self.params.size_pct
        max_shares = int(max_investment / current_price)
        return max_shares

    def notify_order(self, order):
        """
        订单完成后重置状态
        """
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            name = order.data._name
            self.orders[name] = None

    def get_net_value_series(self):
        """
        返回净值序列，用于后续分析
        """
        return pd.DataFrame(self.value, index=self.dates)

    def get_debug_df(self):
        """
        返回包含调试信息的DataFrame
        """
        df = pd.DataFrame(self.debug_info)
        df.set_index('Date', inplace=True)
        return df


def run_backtest(strategy, target_assets, strategy_results, cash=100000.0, commission=0.0005, slippage_perc=0.0005, slippage_fixed=None, **kwargs):
    
    cerebro = bt.Cerebro()  # 初始化Cerebro引擎
    cerebro.addstrategy(strategy, **kwargs)  # 添加策略
    
    for code in target_assets:
        data = PandasDataPlusSignal(dataname=strategy_results[code])
        data._name = code  # 为数据设置名称，便于识别
        cerebro.adddata(data)
    
    # 使用setcommission设置股票模式的佣金
    cerebro.broker.setcommission(
        commission=commission,  # 佣金百分比
        stocklike=True  # 将交易设置为股票模式
    )
    
    cerebro.broker.setcash(cash)  # 设置初始资金

    # 设置滑点
    if slippage_perc is not None:
        cerebro.broker.set_slippage_perc(slippage_perc)  # 设置百分比滑点
    elif slippage_fixed is not None:
        cerebro.broker.set_slippage_fixed(slippage_fixed)  # 设置固定点滑点
    
    strategies = cerebro.run()  # 运行回测
    return strategies[0]



#加载分析工具
AT=Analyzing_Tools()


# 定义数据路径
paths = {
    'daily': r'E:\数据库\同花顺ETF跟踪指数量价数据\1d',
    'hourly': r'E:\数据库\同花顺ETF跟踪指数量价数据\1h',
    'min15': r'E:\数据库\同花顺ETF跟踪指数量价数据\15min',
    'pv_export':r"E:\量化交易构建\私募基金研究\股票策略研究\策略净值序列"
}


# 资产列表
target_assets = [
    "000300.SH",
    "000852.SH",
    "000905.SH",
    "399006.SZ",
    "399303.SZ"
]



# 生成信号
strategy_results,full_info = SMA_H(target_assets, paths)


# 获取策略实例
strat = run_backtest(SMA_H_Strategy,target_assets,strategy_results,10000000,0.0005,0.0005)

pv=strat.get_net_value_series()

strtegy_name='SMA_H'

pv.to_excel(paths["pv_export"]+'\\'+strtegy_name+'.xlsx')

portfolio_value, returns, drawdown_ts, metrics = AT.performance_analysis(pv, freq='D')

index_price_path=paths['daily']

# 获取净值序列
AT.plot_results('000906.SH',index_price_path,portfolio_value, drawdown_ts, returns, metrics)

# 获取调试信息
debug_df = strat.get_debug_df()

#蒙特卡洛测试

AT.monte_carlo_analysis(strat,num_simulations=10000,num_days=252,freq='D')



#定义参数优化函数
def parameter_optimization(parameter_grid, strategy_function, strategy_class, target_assets, paths, cash=100000.0, commission=0.0002, slippage_perc=0.0005, metric='sharpe_ratio'):   
    """
    执行参数优化，支持一个或两个参数。

    参数：
    - parameter_grid: 字典，包含参数名称和要测试的取值列表。例如：{'window_1': [30, 34, 38]}
    - strategy_function: 生成信号的策略函数，例如 UDVD
    - strategy_class: Backtrader 策略类，例如 UDVD_Strategy
    - target_assets: 资产列表
    - paths: 数据路径字典
    - cash: 初始资金
    - commission: 佣金
    - slippage_perc: 滑点百分比
    - metric: 选择用于评估的绩效指标，默认为 'sharpe_ratio'
    """

    #加载数据类

    # 获取参数名称和取值列表
    param_names = list(parameter_grid.keys())
    param_values = [parameter_grid[key] for key in param_names]

    # 生成所有参数组合
    param_combinations = [dict(zip(param_names, values)) for values in product(*param_values)]

    results = []

    for params in param_combinations:
        print(f"正在测试参数组合：{params}")
        # 生成当前参数下的信号
        strategy_results, full_info = strategy_function(target_assets, paths, **params)

        # 运行回测
        strat = run_backtest(strategy_class, target_assets, strategy_results, cash, commission, slippage_perc)

        # 获取净值序列
        pv = strat.get_net_value_series()

        # 计算绩效指标
        portfolio_value, returns, drawdown_ts, metrics =AT.performance_analysis(pv)

        # 收集指标和参数
        result_entry = {k: v for k, v in params.items()}
        result_entry.update(metrics)
        result_entry=pd.DataFrame(result_entry)
        results.append(result_entry)

    # 将结果转换为 DataFrame
    results_df = pd.concat(results,axis=0)
    results_df=results_df.dropna()

    # 可视化结果
    if len(param_names) == 1:
        # 绘制参数与绩效指标的关系曲线
        param = param_names[0]
        plt.figure(figsize=(10, 6))
        plt.plot(results_df[param], results_df[metric], marker='o')
        plt.xlabel(param)
        plt.ylabel(metric)
        plt.title(f'{metric} vs {param}')
        plt.grid(True)
        plt.show()
    elif len(param_names) == 2:
        # 绘制热力图
        param1 = param_names[0]
        param2 = param_names[1]
        pivot_table = results_df.pivot(index=param1, columns=param2, values=metric)

        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
        plt.title(f'{metric} Heatmap')
        plt.ylabel(param1)
        plt.xlabel(param2)
        plt.show()
    else:
        print("无法可视化超过两个参数的结果，请减少参数数量。")

    # 返回结果 DataFrame
    return results_df

# 定义参数网格
parameter_grid = {
    'window_1': range(50, 80,5),
    'window_2': range(80, 110, 5)
}

# 运行参数优化
# results_df = parameter_optimization(
#     parameter_grid=parameter_grid,
#     strategy_function=SMA_H,
#     strategy_class=SMA_H_Strategy,
#     target_assets=target_assets,
#     paths=paths,
#     cash=10000000,
#     commission=0.0002,
#     slippage_perc=0.0005,
#     metric='sharpe_ratio'
# )

d:\Anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


已连接到 base (Python 3.9.7)

In [ ]:
# -*- coding: utf-8 -*-
from iFinDPy import *
from datetime import datetime
import pandas as pd
import time as _time
import json
from threading import Thread,Lock,Semaphore
import requests
import numpy as np
from datetime import datetime
from numpy import nan as NA
import matplotlib.pyplot as plt
import math
from pandas.tseries.offsets import Day,MonthEnd
import os
from tqdm import tqdm  # 用于显示进度条
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("hwqh100","155d50")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')

thslogindemo()


class Downloading_And_Updating_Data():

    def __init__(self):

        self.A_Share_Trading_Hours= ["10:30", "11:30", "14:00", "15:00"]

        self.A_Share_Trading_15mins=["9:45", "10:00", "10:15", "10:30", 
                                    "10:45", "11:00", "11:15", "11:30",
                                     "13:15", "13:30", "13:45","14:00",
                                      "14:15", "14:30", "14:45", "15:00"]
    
    def Downloading_Market_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HQ(Code_List[i],'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_Market_Vol_Price_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_HQ(code,'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_All_Market_Vol_Data(self,End_Date, Data_Path_List):

        def Updating_Market_Vol_Price_Data(End_Date, Data_Path):
            os.chdir(Data_Path)  # 切换到数据路径

            List = os.listdir()  # 获取路径下所有文件
            Doc_Number = len(List)

            print("更新开始")

            def update_file(file_name):
                try:
                    file_path = os.path.join(Data_Path, file_name)
                    df = pd.read_csv(file_path, index_col=[0])
                    df.index = pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                        New_Begin_Date = (df.index[-1] + timedelta(days=1)).strftime("%Y-%m-%d")
                        code = file_name.replace('.csv', "")

                        # 获取新数据
                        new_data = THS_HQ(code, 'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital', '', New_Begin_Date, End_Date).data
                        new_data.index = pd.to_datetime(new_data["time"])  # 将时间列设为索引
                        new_data = new_data.drop(columns=["time", "thscode"])  # 删除无关列

                        Updated_Data = pd.concat([df, new_data], axis=0)  # 合并新旧数据
                        Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[1,0]
                        Updated_Data.to_csv(file_path)  # 保存更新后的数据
                        return f"{file_name} 更新成功"
                    else:
                        return f"{file_name} 数据更新至最新"
                except Exception as e:
                    return f"{file_name} 出现错误: {e}"

            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = list(tqdm(executor.map(update_file, List), total=Doc_Number, desc="更新进度"))

            for result in results:
                print(result)

            print("更新结束")

        for data_path in Data_Path_List:
            try:
                Updating_Market_Vol_Price_Data(End_Date, data_path)
            except Exception as e:
                print(f"{data_path} 出现错误: {e}")

        return

    def Download_Index_Valuation_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_Valuation_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            Data=Data.set_index('time',drop=True)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_Valuation_Data(i,Begin_Date,End_Date)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_Valuation_Data(self,End_Date, Data_Path):
        def process_file(filename):
            try:
                df = pd.read_csv(os.path.join(Data_Path, filename), index_col=[0])
                df.index = pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                    New_Begin_Date = df.index[-1] + Day()
                    New_Begin_Date = New_Begin_Date.strftime("%Y-%m-%d")

                    code = filename.replace('.csv', "")
                    new_data = THS_DS(code, 'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index', '100,100;100,100;', 'block:history', New_Begin_Date, End_Date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    Updated_Data = pd.concat([df, new_data], axis=0)
                    Updated_Data.to_csv(os.path.join(Data_Path, filename))

                    print(f"{filename} processed successfully.")
                else:
                    print(f"{filename} is already up to date.")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

        os.chdir(Data_Path)
        List = os.listdir()
        Doc_Number = len(List)

        print("更新开始")

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(process_file, filename) for filename in List]
            for i, future in enumerate(futures):
                try:
                    future.result()  # Wait for thread to complete and handle exceptions
                except Exception as e:
                    print(f"Thread {i} generated an exception: {e}")

        print("更新结束")

        return


        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Free_Turn_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Free_Trun_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_free_turnover_ratio_index','','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Free_Trun_Data(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Updating_Index_Free_Turn_Data(self,end_date, data_path):
        """
        更新指定路径下的所有CSV文件中的数据，如果数据不是最新的，则从最新日期开始更新至指定的结束日期。
        
        :param end_date: 数据更新的截止日期，格式为'YYYY-MM-DD'
        :param data_path: 存放CSV文件的路径
        """
        def update_file(file_name):
            try:
                # 读取CSV文件
                df = pd.read_csv(os.path.join(data_path, file_name), index_col=[0])
                df.index = pd.to_datetime(df.index)

                # 检查数据是否已经是最新的
                if df.index[-1].strftime('%Y-%m-%d') != end_date:
                    # 计算新的开始日期
                    new_begin_date = (df.index[-1] + Day(1)).strftime("%Y-%m-%d")
                    code = file_name.replace('.csv', "")

                    # 获取新的数据
                    new_data = THS_DS(code, 'ths_free_turnover_ratio_index', '', 'block:history', new_begin_date, end_date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    # 合并旧数据和新数据
                    updated_data = pd.concat([df, new_data], axis=0)

                    # 保存更新后的数据
                    updated_data.to_csv(os.path.join(data_path, file_name))

                    print(f"{file_name} 更新完成")
                else:
                    print(f"{file_name} 数据已是最新")

            except Exception as e:
                print(f"{file_name} 出现错误: {e}")

        # 获取路径下所有文件的列表
        file_list = os.listdir(data_path)

        print("更新开始")

        # 使用线程池来并行处理文件
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(update_file, file_name) for file_name in file_list]

            for future in futures:
                future.result()  # 等待所有线程完成

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code
    
    def Download_Index_Forcast_2yr_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

            def Downloading_Forcast_2yr_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_fore_np_compound_growth_2y_index','','block:history',i,j).data

                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

            os.chdir(Export_Path)

            Error_Code=[]

            print("更新开始")
            
            for i in Code_List:

                try:
                    
                    Data=Downloading_Forcast_2yr_Data(i,Begin_Date,End_Date)

                    Data.index=Data.loc[:,"time"]

                    Data=Data.iloc[:,1:]

                    Data.to_csv(i+".csv")

                    print(i+"输出完成")

                except:

                    print(i+"出现错误")

                    Error_Code.append(i)
            
            return
    
    def Updating_Forcast_2yr_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_fore_np_compound_growth_2y_index','','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_EDB_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        for i in range(0,len(Code_List)):
        
            try:
                Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data.index=pd.to_datetime(Data.index)

                Data=Data.iloc[:,2:5]

                os.chdir(Export_Path)

                Data.to_csv(Code_List[i]+".csv")

                print('完成度: '+str(i/len(Code_List)))

            except:

                print(Code_List[i]+"出现错误")

        return

    def Update_EDB_Data(self,Data_Path,End_Date):

        def Download_EDB_Data(Code_List,Begin_Date,End_Date,Export_Path):

                for i in range(0,len(Code_List)):
                
                    try:
                        Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                        Data.index=Data.loc[:,"time"]

                        Data.index=pd.to_datetime(Data.index)

                        Data=Data.iloc[:,2:5]

                        os.chdir(Export_Path)

                        Data.to_csv(Code_List[i]+".csv")

                        print('完成度: '+str(i/len(Code_List)))

                    except:

                        print(Code_List[i]+"出现错误")

                return

        List=os.listdir(Data_Path)

        Code_List=[]

        for i in List:

            Code=i.replace(".csv","")

            Code_List.append(Code)

        Download_EDB_Data(Code_List,"2003-01-01",End_Date,Data_Path)

        return

    def Downloading_ETF_Option_PCR_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
    
        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_ETF_Option_PCR(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

            def generate_date_intervals(start_date, end_date):
                # 将输入的日期字符串转换为datetime对象
                start_date = datetime.strptime(start_date, "%Y-%m-%d")
                end_date = datetime.strptime(end_date, "%Y-%m-%d")

                # 初始化开始日期和结束日期的列表
                begin_date_list = []
                end_date_list = []

                temp_date = start_date
                while temp_date <= end_date:
                    # 将当前日期添加到开始日期列表
                    begin_date_list.append(temp_date.strftime("%Y-%m-%d"))

                    if temp_date.year == end_date.year:
                        # 如果是结束日期的年份，那么结束日期就是输入的结束日期
                        temp_end_date = end_date
                    else:
                        # 否则，结束日期是当前年份的12月31日
                        temp_end_date = datetime(temp_date.year, 12, 31)

                    # 将结束日期添加到结束日期列表
                    end_date_list.append(temp_end_date.strftime("%Y-%m-%d"))

                    # 将临时日期设置为下一年的1月1日
                    temp_date = datetime(temp_date.year + 1, 1, 1)

                return begin_date_list, end_date_list

            Begin_Date_List,End_Date_List=generate_date_intervals(Begin_Date,End_Date)

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                date=FY1+';'+FY1

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',i,j).data

                Data.append(DF)
            
            result=pd.concat(Data,axis=0)

            return result

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        # 使用tqdm包装Code_List
        for i in Code_List: 

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Update_Index_Forcast_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    Year=New_Begin_Date[:4]

                    FY1=int(Year)+1

                    FY1=str(FY1)

                    date=FY1+';'+FY1

                    new_data=THS_DS(code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_Mutal_Fund_Netvalue(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_accum_unit_nv_fund','','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data[['ths_accum_unit_nv_fund']]

                Data.columns=[Code_List[i]]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Downloading_F_Score_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path = Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0, len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',Begin_Date,End_Date).data   

                Data.index = Data.loc[:, "time"]

                Data = Data.iloc[:, 1:]

                Data.to_csv(Code_List[i] + ".csv")

                print(str(i / len(Code_List)))

            except:

                print(Code_List[i] + "出现错误")

        print("下载结束")

    def Update_F_Score_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',New_Begin_Date,End_Date).data  
                    
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("个股财务数据更新结束")
        
    def Downloading_Cov_Bond_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',Begin_Date,End_Date).data
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Update_Cov_Bond_Vol_Price_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',New_Begin_Date,End_Date).data
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("可转债数据更新结束")

        return

    def Downloading_A50_Futures_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',Begin_Date,End_Date).data

                Data=Data.set_index('time',drop=True)

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_A50_Futures_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=  THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date).data
                        
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_newHL_Data(self,Begin_Date, End_Date, Export_Path, Code='001005010'):
        
        Begin_Date = pd.to_datetime(Begin_Date)
        End_Date = pd.to_datetime(End_Date)
        
        Gap = (End_Date - Begin_Date) / 250
        Period = Gap.days
        
        Timeseries = pd.date_range(Begin_Date, End_Date, periods=Period)
        Timeseries_List = Timeseries.strftime("%Y-%m-%d").tolist()
        
        Begin_Date_List = Timeseries_List[:-1]
        End_Date_List = Timeseries_List[1:]
        
        Data = []
        
        # 使用 tqdm 来显示进度条
        for i, j in tqdm(zip(Begin_Date_List, End_Date_List), total=len(Begin_Date_List), desc="Downloading Data"):
            
            DF = THS_DS(Code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', i, j).data
            
            Data.append(DF)
        
        Data = pd.concat(Data, axis=0)
        Data = Data.set_index('time', drop=True)
        
        os.chdir(Export_Path)
        Data.to_csv(Code+'.csv')
        
        return Data
     
    def Updating_newHL_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0) 

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_High_Freq_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path,Interval=60):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HF(Code_List[i],'open;high;low;close;volume;amount','Fill:Original,Interval:'+str(Interval),Begin_Date,End_Date).data
                
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def get_next_trading_time_1h(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_Hours
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "10:30"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("10:30", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")

    def get_next_trading_time_15min(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_15mins
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "09:45"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("09:45", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")
        
    def Update_High_Freq_Vol_Price_Data(self, End_Date, Export_Path, Interval=60):
        """
        更新高频数据 (小时级别)，确保数据不重复下载，并且增量更新。
        
        :param End_Date: 数据更新的结束时间（字符串格式 'YYYY-MM-DD HH:MM'）
        :param Export_Path: 导出文件夹的路径
        :param Interval: 数据的时间间隔，默认是 60 分钟（1 小时）
        """
        
        os.chdir(Export_Path)  # 切换到数据存放的目录
        
        # 获取所有以 .csv 结尾的文件名，并去掉 .csv 后缀，生成股票代码列表
        Code_List = [f.replace('.csv', '') for f in os.listdir(Export_Path) if f.endswith('.csv')]
        
        print("更新开始")
        
        if Interval == 60:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_1h(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                    else:
                        df_combined = new_data
                    
                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')
                    
                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")
        
        if Interval == 15:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_15min(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                        df_combined = df_combined[~df_combined.index.duplicated(keep='last')]  # 去重并保留最新的数据
                    else:
                        df_combined = new_data

                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')

                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")

    def Downloading_ETF_Option_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
            # 期权code 上证50=510050 沪深300=510300 500ETF=510500 科创50ETF=588000 科创板50=588080

            Export_Path=Export_Path

            print("下载开始")

            for i in range(0,len(Code_List)):

                try:
                    code=Code_List[i]+'O'
                    sdate=Begin_Date.replace('-','')
                    edate=End_Date.replace('-','')
                    Data=THS_DR('p02872','qqbd='+code+';'+'sdate='+sdate+';'+'edate='+edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    
                    Data=Data.set_index('p02872_f001',drop=True)

                    Data=Data.sort_index()

                    Data.to_csv(Export_Path+'\\'+Code_List[i]+".csv")

                    print(str(i/len(Code_List)))
                
                except:

                    print(Code_List[i]+"出现错误")

            print("下载结束")

    def Updating_ETF_Option(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code_original = List[i].replace('.csv', "")
                    code=code_original+'O'
                    code=Code_List[i]+'O'

                    new_sdate=New_Begin_Date.replace('-','')
                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p02872','qqbd='+code+';'+'sdate='+new_sdate+';'+'edate='+new_edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    new_data=new_data.set_index('p02872_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_DDE_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_DDE_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',i,j).data
                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_DDE_Data(i,Begin_Date,End_Date)

                Data=Data.set_index('time',drop=True)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_DDE_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                    THS_DS('801017.SL','ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data
                    new_data=THS_DS(code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)
                    
                    Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[0,0]

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return
    
        
        def Updating_updown_Data(End_Date, Data_Path):

            Data_Path=Data_Path

            os.chdir(Data_Path)

            List=os.listdir()

            Doc_Number=len(List)

            print("更新开始")

            for i in range(0,Doc_Number):

                try:

                    df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                    df.index=pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                        New_Begin_Date=df.index[-1]+Day()

                        New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                        code = List[i].replace('.csv', "")

                        new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                        new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                        new_data=new_data.iloc[:,2:]

                        Updated_Data=pd.concat([df,new_data],axis=0)

                        Updated_Data.to_csv(List[i])

                        print(str(i/Doc_Number))

                    else:

                        print("数据更新至最新")

                except:

                    print(List[i]+"出现错误")

            print("更新结束")

            return

    def Updating_updown_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_Up_Down_Companies(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code= List[i].replace('.csv', "")

                    new_sdate=New_Begin_Date.replace('-','')

                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p00112','sdate='+new_sdate+';'+'edate='+new_edate+';p0='+code,'p00112_f001:Y,p00112_f002:Y,p00112_f003:Y,p00112_f004:Y,p00112_f005:Y,p00112_f006:Y,p00112_f007:Y,p00112_f008:Y,p00112_f009:Y,p00112_f010:Y,p00112_f011:Y,p00112_f012:Y,p00112_f013:Y,p00112_f014:Y,p00112_f021:Y,p00112_f022:Y,p00112_f023:Y,p00112_f024:Y,p00112_f025:Y',
                                    'format:dataframe').data
                    new_data=new_data.set_index('p00112_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return


DUD=Downloading_And_Updating_Data()



def Update_All(End_Date):

    Price_Data_Path_1=r"E:\数据库\同花顺指数量价数据"

    Price_Data_Path_2=r"E:\数据库\同花顺ETF跟踪指数量价数据\1d"

    Price_Data_Path_3=r'E:\数据库\同花顺ETF量价数据'

    Price_Data_Path_4=r'E:\数据库\同花顺商品指数量价数据'

    Path_List=[Price_Data_Path_1,Price_Data_Path_2,Price_Data_Path_3,Price_Data_Path_4]

    DUD.Updating_All_Market_Vol_Data(End_Date,Path_List)

    Forcast_Path=r"E:\数据库\同花顺ETF跟踪指数一致预期数据\预测净利润两年复合增长"

    DUD.Updating_Forcast_2yr_Data(End_Date,Forcast_Path)

    Free_Turn_Path=r"E:\数据库\同花顺指数自由流通换手率"

    DUD.Updating_Index_Free_Turn_Data(End_Date,Free_Turn_Path)

    Valuation_Path=r"E:\数据库\同花顺ETF跟踪指数估值数据"

    DUD.Updating_Index_Valuation_Data(End_Date,Valuation_Path)

    EDB_Data_Path=r"E:\数据库\同花顺EDB数据"

    DUD.Update_EDB_Data(EDB_Data_Path,End_Date)

    ETF_Option_Path=r"E:\数据库\另类数据\ETF期权数据"

    DUD.Updating_ETF_Option(End_Date,ETF_Option_Path)

    Forcast_Data_Path=r'E:\数据库\同花顺ETF跟踪指数一致预期数据\盈利预测综合值'

    DUD.Update_Index_Forcast_Data(End_Date,Forcast_Data_Path)

    Cov_Bond_Path=r'E:\数据库\同花顺可转债数据'

    DUD.Update_Cov_Bond_Vol_Price_Data(End_Date,Cov_Bond_Path)

    #新高新低
    H_L_Path=r'E:\数据库\另类数据\新高新低'

    DUD.Updating_newHL_Data(End_Date,H_L_Path)

    #高频数据更新

    Export_Path_60 = r'E:\数据库\同花顺ETF跟踪指数量价数据\1h' 

    Export_Path_15=r'E:\数据库\同花顺ETF跟踪指数量价数据\15min'

    HF_End_Date=End_Date+' 15:00'

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date, Export_Path_60,60)

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date,Export_Path_15,15)

    #公司上涨数量更新
    
    Up_Down_Company_Path=r'E:\数据库\另类数据\涨跌家数'

    DUD.Updating_Up_Down_Companies(End_Date,Up_Down_Company_Path)

    #涨停跌停数量

    up_path=r'E:\数据库\另类数据\涨停跌停'

    DUD.Updating_updown_Data(End_Date,up_path)


    return print('updating finished')

#海外数据第二天更新
def Update_Oversea_Data(End_Date):

    A50_Path=r'E:\数据库\另类数据\A50期货数据'

    DUD.Updating_A50_Futures_Data(End_Date,A50_Path)

    return print('海外数据更新完毕')

# Update_Oversea_Data('2024-12-24')

Update_All('2024-12-25')

d:\Anaconda3\Lib\site-packages\iFinDPy.pth
0
登录成功
更新开始


更新进度: 100%|██████████| 30/30 [00:03<00:00,  9.02it/s]


000001.SH.csv 更新成功
000016.SH.csv 更新成功
000300.SH.csv 更新成功
000688.SH.csv 更新成功
000832.CSI.csv 更新成功
000852.SH.csv 更新成功
000905.SH.csv 更新成功
000906.SH.csv 更新成功
000985.CSI.csv 更新成功
399006.SZ.csv 更新成功
399303.SZ.csv 更新成功
930665.CSI.csv 更新成功
930666.CSI.csv 更新成功
931153.CSI.csv 更新成功
H11001.CSI.csv 出现错误: NaTType does not support strftime
H11009.CSI.csv 更新成功
HSCE.HK.csv 出现错误: 'time'
HSCSHEI.HK.csv 出现错误: 'time'
IMCI.SHF.csv 更新成功
IXIC.GI.csv 更新成功
NHAI.SL.csv 更新成功
NHAUI.SL.csv 更新成功
NHCI.SL.csv 更新成功
NHCUI.SL.csv 更新成功
NHECI.SL.csv 更新成功
NHFI.SL.csv 更新成功
NHII.SL.csv 更新成功
NHMI.SL.csv 更新成功
NHNFI.SL.csv 更新成功
NHPMI.SL.csv 更新成功
更新结束
更新开始


更新进度: 100%|██████████| 275/275 [00:35<00:00,  7.72it/s]


000001.SH.csv 更新成功
000009.SH.csv 更新成功
000010.SH.csv 更新成功
000015.SH.csv 更新成功
000016.SH.csv 更新成功
000018.SH.csv 更新成功
000021.SH.csv 更新成功
000029.SH.csv 更新成功
000036.SH.csv 更新成功
000037.SH.csv 更新成功
000038.SH.csv 更新成功
000042.SH.csv 更新成功
000043.SH.csv 更新成功
000044.SH.csv 更新成功
000048.SH.csv 更新成功
000056.SH.csv 更新成功
000063.SH.csv 更新成功
000064.SH.csv 更新成功
000065.SH.csv 更新成功
000066.SH.csv 更新成功
000068.SH.csv 更新成功
000069.SH.csv 更新成功
000171.CSI.csv 更新成功
000300.SH.csv 更新成功
000682.SH.csv 更新成功
000685.SH.csv 更新成功
000688.SH.csv 更新成功
000689.SH.csv 更新成功
000806.CSI.csv 更新成功
000807.CSI.csv 更新成功
000811.CSI.csv 更新成功
000812.CSI.csv 更新成功
000813.CSI.csv 更新成功
000814.SH.csv 更新成功
000815.CSI.csv 更新成功
000819.SH.csv 更新成功
000821.CSI.csv 更新成功
000827.SH.csv 更新成功
000852.SH.csv 更新成功
000859.CSI.csv 更新成功
000860.CSI.csv 更新成功
000861.CSI.csv 更新成功
000901.SH.csv 更新成功
000903.SH.csv 更新成功
000905.SH.csv 更新成功
000906.SH.csv 更新成功
000913.SH.csv 更新成功
000914.SH.csv 更新成功
000918.CSI.csv 更新成功
000919.CSI.csv 更新成功
000922.CSI.csv 更新成功
000925.CSI.csv 更新

...

 83%|████████▎ | 5/6 [00:04<00:00,  1.10it/s]

399006.SZ 数据更新成功。


100%|██████████| 6/6 [00:05<00:00,  1.11it/s]

399303.SZ 数据更新成功。
数据更新结束
更新开始


更新结束
更新开始
0.0
更新结束
updating finished


In [ ]:
#海外数据第二天更新
def Update_Oversea_Data(End_Date):

    A50_Path=r'E:\数据库\另类数据\A50期货数据'

    DUD.Updating_A50_Futures_Data(End_Date,A50_Path)

    return print('海外数据更新完毕')

Update_Oversea_Data('2024-12-25')


更新开始
CN0Y.SG.csv出现错误
更新结束
海外数据更新完毕


In [ ]:
# -*- coding: utf-8 -*-
from iFinDPy import *
from datetime import datetime
import pandas as pd
import time as _time
import json
from threading import Thread,Lock,Semaphore
import requests
import numpy as np
from datetime import datetime
from numpy import nan as NA
import matplotlib.pyplot as plt
import math
from pandas.tseries.offsets import Day,MonthEnd
import os
from tqdm import tqdm  # 用于显示进度条
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("hwqh100","155d50")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')

thslogindemo()


class Downloading_And_Updating_Data():

    def __init__(self):

        self.A_Share_Trading_Hours= ["10:30", "11:30", "14:00", "15:00"]

        self.A_Share_Trading_15mins=["9:45", "10:00", "10:15", "10:30", 
                                    "10:45", "11:00", "11:15", "11:30",
                                     "13:15", "13:30", "13:45","14:00",
                                      "14:15", "14:30", "14:45", "15:00"]
    
    def Downloading_Market_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HQ(Code_List[i],'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_Market_Vol_Price_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_HQ(code,'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_All_Market_Vol_Data(self,End_Date, Data_Path_List):

        def Updating_Market_Vol_Price_Data(End_Date, Data_Path):
            os.chdir(Data_Path)  # 切换到数据路径

            List = os.listdir()  # 获取路径下所有文件
            Doc_Number = len(List)

            print("更新开始")

            def update_file(file_name):
                try:
                    file_path = os.path.join(Data_Path, file_name)
                    df = pd.read_csv(file_path, index_col=[0])
                    df.index = pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                        New_Begin_Date = (df.index[-1] + timedelta(days=1)).strftime("%Y-%m-%d")
                        code = file_name.replace('.csv', "")

                        # 获取新数据
                        new_data = THS_HQ(code, 'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital', '', New_Begin_Date, End_Date).data
                        new_data.index = pd.to_datetime(new_data["time"])  # 将时间列设为索引
                        new_data = new_data.drop(columns=["time", "thscode"])  # 删除无关列

                        Updated_Data = pd.concat([df, new_data], axis=0)  # 合并新旧数据
                        Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[1,0]
                        Updated_Data.to_csv(file_path)  # 保存更新后的数据
                        return f"{file_name} 更新成功"
                    else:
                        return f"{file_name} 数据更新至最新"
                except Exception as e:
                    return f"{file_name} 出现错误: {e}"

            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = list(tqdm(executor.map(update_file, List), total=Doc_Number, desc="更新进度"))

            for result in results:
                print(result)

            print("更新结束")

        for data_path in Data_Path_List:
            try:
                Updating_Market_Vol_Price_Data(End_Date, data_path)
            except Exception as e:
                print(f"{data_path} 出现错误: {e}")

        return

    def Download_Index_Valuation_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_Valuation_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            Data=Data.set_index('time',drop=True)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_Valuation_Data(i,Begin_Date,End_Date)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_Valuation_Data(self,End_Date, Data_Path):
        def process_file(filename):
            try:
                df = pd.read_csv(os.path.join(Data_Path, filename), index_col=[0])
                df.index = pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                    New_Begin_Date = df.index[-1] + Day()
                    New_Begin_Date = New_Begin_Date.strftime("%Y-%m-%d")

                    code = filename.replace('.csv', "")
                    new_data = THS_DS(code, 'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index', '100,100;100,100;', 'block:history', New_Begin_Date, End_Date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    Updated_Data = pd.concat([df, new_data], axis=0)
                    Updated_Data.to_csv(os.path.join(Data_Path, filename))

                    print(f"{filename} processed successfully.")
                else:
                    print(f"{filename} is already up to date.")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

        os.chdir(Data_Path)
        List = os.listdir()
        Doc_Number = len(List)

        print("更新开始")

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(process_file, filename) for filename in List]
            for i, future in enumerate(futures):
                try:
                    future.result()  # Wait for thread to complete and handle exceptions
                except Exception as e:
                    print(f"Thread {i} generated an exception: {e}")

        print("更新结束")

        return


        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Free_Turn_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Free_Trun_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_free_turnover_ratio_index','','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Free_Trun_Data(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Updating_Index_Free_Turn_Data(self,end_date, data_path):
        """
        更新指定路径下的所有CSV文件中的数据，如果数据不是最新的，则从最新日期开始更新至指定的结束日期。
        
        :param end_date: 数据更新的截止日期，格式为'YYYY-MM-DD'
        :param data_path: 存放CSV文件的路径
        """
        def update_file(file_name):
            try:
                # 读取CSV文件
                df = pd.read_csv(os.path.join(data_path, file_name), index_col=[0])
                df.index = pd.to_datetime(df.index)

                # 检查数据是否已经是最新的
                if df.index[-1].strftime('%Y-%m-%d') != end_date:
                    # 计算新的开始日期
                    new_begin_date = (df.index[-1] + Day(1)).strftime("%Y-%m-%d")
                    code = file_name.replace('.csv', "")

                    # 获取新的数据
                    new_data = THS_DS(code, 'ths_free_turnover_ratio_index', '', 'block:history', new_begin_date, end_date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    # 合并旧数据和新数据
                    updated_data = pd.concat([df, new_data], axis=0)

                    # 保存更新后的数据
                    updated_data.to_csv(os.path.join(data_path, file_name))

                    print(f"{file_name} 更新完成")
                else:
                    print(f"{file_name} 数据已是最新")

            except Exception as e:
                print(f"{file_name} 出现错误: {e}")

        # 获取路径下所有文件的列表
        file_list = os.listdir(data_path)

        print("更新开始")

        # 使用线程池来并行处理文件
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(update_file, file_name) for file_name in file_list]

            for future in futures:
                future.result()  # 等待所有线程完成

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code
    
    def Download_Index_Forcast_2yr_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

            def Downloading_Forcast_2yr_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_fore_np_compound_growth_2y_index','','block:history',i,j).data

                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

            os.chdir(Export_Path)

            Error_Code=[]

            print("更新开始")
            
            for i in Code_List:

                try:
                    
                    Data=Downloading_Forcast_2yr_Data(i,Begin_Date,End_Date)

                    Data.index=Data.loc[:,"time"]

                    Data=Data.iloc[:,1:]

                    Data.to_csv(i+".csv")

                    print(i+"输出完成")

                except:

                    print(i+"出现错误")

                    Error_Code.append(i)
            
            return
    
    def Updating_Forcast_2yr_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_fore_np_compound_growth_2y_index','','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_EDB_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        for i in range(0,len(Code_List)):
        
            try:
                Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data.index=pd.to_datetime(Data.index)

                Data=Data.iloc[:,2:5]

                os.chdir(Export_Path)

                Data.to_csv(Code_List[i]+".csv")

                print('完成度: '+str(i/len(Code_List)))

            except:

                print(Code_List[i]+"出现错误")

        return

    def Update_EDB_Data(self,Data_Path,End_Date):

        def Download_EDB_Data(Code_List,Begin_Date,End_Date,Export_Path):

                for i in range(0,len(Code_List)):
                
                    try:
                        Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                        Data.index=Data.loc[:,"time"]

                        Data.index=pd.to_datetime(Data.index)

                        Data=Data.iloc[:,2:5]

                        os.chdir(Export_Path)

                        Data.to_csv(Code_List[i]+".csv")

                        print('完成度: '+str(i/len(Code_List)))

                    except:

                        print(Code_List[i]+"出现错误")

                return

        List=os.listdir(Data_Path)

        Code_List=[]

        for i in List:

            Code=i.replace(".csv","")

            Code_List.append(Code)

        Download_EDB_Data(Code_List,"2003-01-01",End_Date,Data_Path)

        return

    def Downloading_ETF_Option_PCR_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
    
        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_ETF_Option_PCR(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

            def generate_date_intervals(start_date, end_date):
                # 将输入的日期字符串转换为datetime对象
                start_date = datetime.strptime(start_date, "%Y-%m-%d")
                end_date = datetime.strptime(end_date, "%Y-%m-%d")

                # 初始化开始日期和结束日期的列表
                begin_date_list = []
                end_date_list = []

                temp_date = start_date
                while temp_date <= end_date:
                    # 将当前日期添加到开始日期列表
                    begin_date_list.append(temp_date.strftime("%Y-%m-%d"))

                    if temp_date.year == end_date.year:
                        # 如果是结束日期的年份，那么结束日期就是输入的结束日期
                        temp_end_date = end_date
                    else:
                        # 否则，结束日期是当前年份的12月31日
                        temp_end_date = datetime(temp_date.year, 12, 31)

                    # 将结束日期添加到结束日期列表
                    end_date_list.append(temp_end_date.strftime("%Y-%m-%d"))

                    # 将临时日期设置为下一年的1月1日
                    temp_date = datetime(temp_date.year + 1, 1, 1)

                return begin_date_list, end_date_list

            Begin_Date_List,End_Date_List=generate_date_intervals(Begin_Date,End_Date)

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                date=FY1+';'+FY1

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',i,j).data

                Data.append(DF)
            
            result=pd.concat(Data,axis=0)

            return result

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        # 使用tqdm包装Code_List
        for i in Code_List: 

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Update_Index_Forcast_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    Year=New_Begin_Date[:4]

                    FY1=int(Year)+1

                    FY1=str(FY1)

                    date=FY1+';'+FY1

                    new_data=THS_DS(code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_Mutal_Fund_Netvalue(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_accum_unit_nv_fund','','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data[['ths_accum_unit_nv_fund']]

                Data.columns=[Code_List[i]]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Downloading_F_Score_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path = Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0, len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',Begin_Date,End_Date).data   

                Data.index = Data.loc[:, "time"]

                Data = Data.iloc[:, 1:]

                Data.to_csv(Code_List[i] + ".csv")

                print(str(i / len(Code_List)))

            except:

                print(Code_List[i] + "出现错误")

        print("下载结束")

    def Update_F_Score_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',New_Begin_Date,End_Date).data  
                    
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("个股财务数据更新结束")
        
    def Downloading_Cov_Bond_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',Begin_Date,End_Date).data
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Update_Cov_Bond_Vol_Price_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',New_Begin_Date,End_Date).data
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("可转债数据更新结束")

        return

    def Downloading_A50_Futures_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',Begin_Date,End_Date).data

                Data=Data.set_index('time',drop=True)

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_A50_Futures_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=  THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date).data
                        
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_newHL_Data(self,Begin_Date, End_Date, Export_Path, Code='001005010'):
        
        Begin_Date = pd.to_datetime(Begin_Date)
        End_Date = pd.to_datetime(End_Date)
        
        Gap = (End_Date - Begin_Date) / 250
        Period = Gap.days
        
        Timeseries = pd.date_range(Begin_Date, End_Date, periods=Period)
        Timeseries_List = Timeseries.strftime("%Y-%m-%d").tolist()
        
        Begin_Date_List = Timeseries_List[:-1]
        End_Date_List = Timeseries_List[1:]
        
        Data = []
        
        # 使用 tqdm 来显示进度条
        for i, j in tqdm(zip(Begin_Date_List, End_Date_List), total=len(Begin_Date_List), desc="Downloading Data"):
            
            DF = THS_DS(Code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', i, j).data
            
            Data.append(DF)
        
        Data = pd.concat(Data, axis=0)
        Data = Data.set_index('time', drop=True)
        
        os.chdir(Export_Path)
        Data.to_csv(Code+'.csv')
        
        return Data
     
    def Updating_newHL_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0) 

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_High_Freq_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path,Interval=60):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HF(Code_List[i],'open;high;low;close;volume;amount','Fill:Original,Interval:'+str(Interval),Begin_Date,End_Date).data
                
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def get_next_trading_time_1h(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_Hours
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "10:30"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("10:30", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")

    def get_next_trading_time_15min(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_15mins
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "09:45"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("09:45", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")
        
    def Update_High_Freq_Vol_Price_Data(self, End_Date, Export_Path, Interval=60):
        """
        更新高频数据 (小时级别)，确保数据不重复下载，并且增量更新。
        
        :param End_Date: 数据更新的结束时间（字符串格式 'YYYY-MM-DD HH:MM'）
        :param Export_Path: 导出文件夹的路径
        :param Interval: 数据的时间间隔，默认是 60 分钟（1 小时）
        """
        
        os.chdir(Export_Path)  # 切换到数据存放的目录
        
        # 获取所有以 .csv 结尾的文件名，并去掉 .csv 后缀，生成股票代码列表
        Code_List = [f.replace('.csv', '') for f in os.listdir(Export_Path) if f.endswith('.csv')]
        
        print("更新开始")
        
        if Interval == 60:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_1h(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                    else:
                        df_combined = new_data
                    
                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')
                    
                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")
        
        if Interval == 15:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_15min(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                        df_combined = df_combined[~df_combined.index.duplicated(keep='last')]  # 去重并保留最新的数据
                    else:
                        df_combined = new_data

                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')

                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")

    def Downloading_ETF_Option_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
            # 期权code 上证50=510050 沪深300=510300 500ETF=510500 科创50ETF=588000 科创板50=588080

            Export_Path=Export_Path

            print("下载开始")

            for i in range(0,len(Code_List)):

                try:
                    code=Code_List[i]+'O'
                    sdate=Begin_Date.replace('-','')
                    edate=End_Date.replace('-','')
                    Data=THS_DR('p02872','qqbd='+code+';'+'sdate='+sdate+';'+'edate='+edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    
                    Data=Data.set_index('p02872_f001',drop=True)

                    Data=Data.sort_index()

                    Data.to_csv(Export_Path+'\\'+Code_List[i]+".csv")

                    print(str(i/len(Code_List)))
                
                except:

                    print(Code_List[i]+"出现错误")

            print("下载结束")

    def Updating_ETF_Option(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code_original = List[i].replace('.csv', "")
                    code=code_original+'O'
                    code=Code_List[i]+'O'

                    new_sdate=New_Begin_Date.replace('-','')
                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p02872','qqbd='+code+';'+'sdate='+new_sdate+';'+'edate='+new_edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    new_data=new_data.set_index('p02872_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_DDE_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_DDE_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',i,j).data
                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_DDE_Data(i,Begin_Date,End_Date)

                Data=Data.set_index('time',drop=True)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_DDE_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                    THS_DS('801017.SL','ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data
                    new_data=THS_DS(code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)
                    
                    Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[0,0]

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return
    
        
        def Updating_updown_Data(End_Date, Data_Path):

            Data_Path=Data_Path

            os.chdir(Data_Path)

            List=os.listdir()

            Doc_Number=len(List)

            print("更新开始")

            for i in range(0,Doc_Number):

                try:

                    df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                    df.index=pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                        New_Begin_Date=df.index[-1]+Day()

                        New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                        code = List[i].replace('.csv', "")

                        new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                        new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                        new_data=new_data.iloc[:,2:]

                        Updated_Data=pd.concat([df,new_data],axis=0)

                        Updated_Data.to_csv(List[i])

                        print(str(i/Doc_Number))

                    else:

                        print("数据更新至最新")

                except:

                    print(List[i]+"出现错误")

            print("更新结束")

            return

    def Updating_updown_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_Up_Down_Companies(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code= List[i].replace('.csv', "")

                    new_sdate=New_Begin_Date.replace('-','')

                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p00112','sdate='+new_sdate+';'+'edate='+new_edate+';p0='+code,'p00112_f001:Y,p00112_f002:Y,p00112_f003:Y,p00112_f004:Y,p00112_f005:Y,p00112_f006:Y,p00112_f007:Y,p00112_f008:Y,p00112_f009:Y,p00112_f010:Y,p00112_f011:Y,p00112_f012:Y,p00112_f013:Y,p00112_f014:Y,p00112_f021:Y,p00112_f022:Y,p00112_f023:Y,p00112_f024:Y,p00112_f025:Y',
                                    'format:dataframe').data
                    new_data=new_data.set_index('p00112_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return


DUD=Downloading_And_Updating_Data()



def Update_All(End_Date):

    Price_Data_Path_1=r"E:\数据库\同花顺指数量价数据"

    Price_Data_Path_2=r"E:\数据库\同花顺ETF跟踪指数量价数据\1d"

    Price_Data_Path_3=r'E:\数据库\同花顺ETF量价数据'

    Price_Data_Path_4=r'E:\数据库\同花顺商品指数量价数据'

    Path_List=[Price_Data_Path_1,Price_Data_Path_2,Price_Data_Path_3,Price_Data_Path_4]

    DUD.Updating_All_Market_Vol_Data(End_Date,Path_List)

    Forcast_Path=r"E:\数据库\同花顺ETF跟踪指数一致预期数据\预测净利润两年复合增长"

    DUD.Updating_Forcast_2yr_Data(End_Date,Forcast_Path)

    Free_Turn_Path=r"E:\数据库\同花顺指数自由流通换手率"

    DUD.Updating_Index_Free_Turn_Data(End_Date,Free_Turn_Path)

    Valuation_Path=r"E:\数据库\同花顺ETF跟踪指数估值数据"

    DUD.Updating_Index_Valuation_Data(End_Date,Valuation_Path)

    EDB_Data_Path=r"E:\数据库\同花顺EDB数据"

    DUD.Update_EDB_Data(EDB_Data_Path,End_Date)

    ETF_Option_Path=r"E:\数据库\另类数据\ETF期权数据"

    DUD.Updating_ETF_Option(End_Date,ETF_Option_Path)

    Forcast_Data_Path=r'E:\数据库\同花顺ETF跟踪指数一致预期数据\盈利预测综合值'

    DUD.Update_Index_Forcast_Data(End_Date,Forcast_Data_Path)

    Cov_Bond_Path=r'E:\数据库\同花顺可转债数据'

    DUD.Update_Cov_Bond_Vol_Price_Data(End_Date,Cov_Bond_Path)

    #新高新低
    H_L_Path=r'E:\数据库\另类数据\新高新低'

    DUD.Updating_newHL_Data(End_Date,H_L_Path)

    #高频数据更新

    Export_Path_60 = r'E:\数据库\同花顺ETF跟踪指数量价数据\1h' 

    Export_Path_15=r'E:\数据库\同花顺ETF跟踪指数量价数据\15min'

    HF_End_Date=End_Date+' 15:00'

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date, Export_Path_60,60)

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date,Export_Path_15,15)

    #公司上涨数量更新
    
    Up_Down_Company_Path=r'E:\数据库\另类数据\涨跌家数'

    DUD.Updating_Up_Down_Companies(End_Date,Up_Down_Company_Path)

    #涨停跌停数量

    up_path=r'E:\数据库\另类数据\涨停跌停'

    DUD.Updating_updown_Data(End_Date,up_path)


    return print('updating finished')

#海外数据第二天更新
def Update_Oversea_Data(End_Date):

    A50_Path=r'E:\数据库\另类数据\A50期货数据'

    DUD.Updating_A50_Futures_Data(End_Date,A50_Path)

    return print('海外数据更新完毕')

Update_Oversea_Data('2024-12-25')

# Update_All('2024-12-25')

0
登录成功
更新开始
CN0Y.SG.csv出现错误
更新结束
海外数据更新完毕


已重启 base (Python 3.9.7)

In [ ]:
# -*- coding: utf-8 -*-
from iFinDPy import *
from datetime import datetime
import pandas as pd
import time as _time
import json
from threading import Thread,Lock,Semaphore
import requests
import numpy as np
from datetime import datetime
from numpy import nan as NA
import matplotlib.pyplot as plt
import math
from pandas.tseries.offsets import Day,MonthEnd
import os
from tqdm import tqdm  # 用于显示进度条
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("hwqh100","155d50")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')

thslogindemo()


class Downloading_And_Updating_Data():

    def __init__(self):

        self.A_Share_Trading_Hours= ["10:30", "11:30", "14:00", "15:00"]

        self.A_Share_Trading_15mins=["9:45", "10:00", "10:15", "10:30", 
                                    "10:45", "11:00", "11:15", "11:30",
                                     "13:15", "13:30", "13:45","14:00",
                                      "14:15", "14:30", "14:45", "15:00"]
    
    def Downloading_Market_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HQ(Code_List[i],'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_Market_Vol_Price_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_HQ(code,'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_All_Market_Vol_Data(self,End_Date, Data_Path_List):

        def Updating_Market_Vol_Price_Data(End_Date, Data_Path):
            os.chdir(Data_Path)  # 切换到数据路径

            List = os.listdir()  # 获取路径下所有文件
            Doc_Number = len(List)

            print("更新开始")

            def update_file(file_name):
                try:
                    file_path = os.path.join(Data_Path, file_name)
                    df = pd.read_csv(file_path, index_col=[0])
                    df.index = pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                        New_Begin_Date = (df.index[-1] + timedelta(days=1)).strftime("%Y-%m-%d")
                        code = file_name.replace('.csv', "")

                        # 获取新数据
                        new_data = THS_HQ(code, 'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital', '', New_Begin_Date, End_Date).data
                        new_data.index = pd.to_datetime(new_data["time"])  # 将时间列设为索引
                        new_data = new_data.drop(columns=["time", "thscode"])  # 删除无关列

                        Updated_Data = pd.concat([df, new_data], axis=0)  # 合并新旧数据
                        Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[1,0]
                        Updated_Data.to_csv(file_path)  # 保存更新后的数据
                        return f"{file_name} 更新成功"
                    else:
                        return f"{file_name} 数据更新至最新"
                except Exception as e:
                    return f"{file_name} 出现错误: {e}"

            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = list(tqdm(executor.map(update_file, List), total=Doc_Number, desc="更新进度"))

            for result in results:
                print(result)

            print("更新结束")

        for data_path in Data_Path_List:
            try:
                Updating_Market_Vol_Price_Data(End_Date, data_path)
            except Exception as e:
                print(f"{data_path} 出现错误: {e}")

        return

    def Download_Index_Valuation_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_Valuation_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            Data=Data.set_index('time',drop=True)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_Valuation_Data(i,Begin_Date,End_Date)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_Valuation_Data(self,End_Date, Data_Path):
        def process_file(filename):
            try:
                df = pd.read_csv(os.path.join(Data_Path, filename), index_col=[0])
                df.index = pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                    New_Begin_Date = df.index[-1] + Day()
                    New_Begin_Date = New_Begin_Date.strftime("%Y-%m-%d")

                    code = filename.replace('.csv', "")
                    new_data = THS_DS(code, 'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index', '100,100;100,100;', 'block:history', New_Begin_Date, End_Date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    Updated_Data = pd.concat([df, new_data], axis=0)
                    Updated_Data.to_csv(os.path.join(Data_Path, filename))

                    print(f"{filename} processed successfully.")
                else:
                    print(f"{filename} is already up to date.")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

        os.chdir(Data_Path)
        List = os.listdir()
        Doc_Number = len(List)

        print("更新开始")

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(process_file, filename) for filename in List]
            for i, future in enumerate(futures):
                try:
                    future.result()  # Wait for thread to complete and handle exceptions
                except Exception as e:
                    print(f"Thread {i} generated an exception: {e}")

        print("更新结束")

        return


        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Free_Turn_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Free_Trun_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_free_turnover_ratio_index','','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Free_Trun_Data(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Updating_Index_Free_Turn_Data(self,end_date, data_path):
        """
        更新指定路径下的所有CSV文件中的数据，如果数据不是最新的，则从最新日期开始更新至指定的结束日期。
        
        :param end_date: 数据更新的截止日期，格式为'YYYY-MM-DD'
        :param data_path: 存放CSV文件的路径
        """
        def update_file(file_name):
            try:
                # 读取CSV文件
                df = pd.read_csv(os.path.join(data_path, file_name), index_col=[0])
                df.index = pd.to_datetime(df.index)

                # 检查数据是否已经是最新的
                if df.index[-1].strftime('%Y-%m-%d') != end_date:
                    # 计算新的开始日期
                    new_begin_date = (df.index[-1] + Day(1)).strftime("%Y-%m-%d")
                    code = file_name.replace('.csv', "")

                    # 获取新的数据
                    new_data = THS_DS(code, 'ths_free_turnover_ratio_index', '', 'block:history', new_begin_date, end_date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    # 合并旧数据和新数据
                    updated_data = pd.concat([df, new_data], axis=0)

                    # 保存更新后的数据
                    updated_data.to_csv(os.path.join(data_path, file_name))

                    print(f"{file_name} 更新完成")
                else:
                    print(f"{file_name} 数据已是最新")

            except Exception as e:
                print(f"{file_name} 出现错误: {e}")

        # 获取路径下所有文件的列表
        file_list = os.listdir(data_path)

        print("更新开始")

        # 使用线程池来并行处理文件
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(update_file, file_name) for file_name in file_list]

            for future in futures:
                future.result()  # 等待所有线程完成

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code
    
    def Download_Index_Forcast_2yr_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

            def Downloading_Forcast_2yr_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_fore_np_compound_growth_2y_index','','block:history',i,j).data

                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

            os.chdir(Export_Path)

            Error_Code=[]

            print("更新开始")
            
            for i in Code_List:

                try:
                    
                    Data=Downloading_Forcast_2yr_Data(i,Begin_Date,End_Date)

                    Data.index=Data.loc[:,"time"]

                    Data=Data.iloc[:,1:]

                    Data.to_csv(i+".csv")

                    print(i+"输出完成")

                except:

                    print(i+"出现错误")

                    Error_Code.append(i)
            
            return
    
    def Updating_Forcast_2yr_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_fore_np_compound_growth_2y_index','','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_EDB_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        for i in range(0,len(Code_List)):
        
            try:
                Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data.index=pd.to_datetime(Data.index)

                Data=Data.iloc[:,2:5]

                os.chdir(Export_Path)

                Data.to_csv(Code_List[i]+".csv")

                print('完成度: '+str(i/len(Code_List)))

            except:

                print(Code_List[i]+"出现错误")

        return

    def Update_EDB_Data(self,Data_Path,End_Date):

        def Download_EDB_Data(Code_List,Begin_Date,End_Date,Export_Path):

                for i in range(0,len(Code_List)):
                
                    try:
                        Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                        Data.index=Data.loc[:,"time"]

                        Data.index=pd.to_datetime(Data.index)

                        Data=Data.iloc[:,2:5]

                        os.chdir(Export_Path)

                        Data.to_csv(Code_List[i]+".csv")

                        print('完成度: '+str(i/len(Code_List)))

                    except:

                        print(Code_List[i]+"出现错误")

                return

        List=os.listdir(Data_Path)

        Code_List=[]

        for i in List:

            Code=i.replace(".csv","")

            Code_List.append(Code)

        Download_EDB_Data(Code_List,"2003-01-01",End_Date,Data_Path)

        return

    def Downloading_ETF_Option_PCR_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
    
        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_ETF_Option_PCR(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

            def generate_date_intervals(start_date, end_date):
                # 将输入的日期字符串转换为datetime对象
                start_date = datetime.strptime(start_date, "%Y-%m-%d")
                end_date = datetime.strptime(end_date, "%Y-%m-%d")

                # 初始化开始日期和结束日期的列表
                begin_date_list = []
                end_date_list = []

                temp_date = start_date
                while temp_date <= end_date:
                    # 将当前日期添加到开始日期列表
                    begin_date_list.append(temp_date.strftime("%Y-%m-%d"))

                    if temp_date.year == end_date.year:
                        # 如果是结束日期的年份，那么结束日期就是输入的结束日期
                        temp_end_date = end_date
                    else:
                        # 否则，结束日期是当前年份的12月31日
                        temp_end_date = datetime(temp_date.year, 12, 31)

                    # 将结束日期添加到结束日期列表
                    end_date_list.append(temp_end_date.strftime("%Y-%m-%d"))

                    # 将临时日期设置为下一年的1月1日
                    temp_date = datetime(temp_date.year + 1, 1, 1)

                return begin_date_list, end_date_list

            Begin_Date_List,End_Date_List=generate_date_intervals(Begin_Date,End_Date)

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                date=FY1+';'+FY1

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',i,j).data

                Data.append(DF)
            
            result=pd.concat(Data,axis=0)

            return result

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        # 使用tqdm包装Code_List
        for i in Code_List: 

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Update_Index_Forcast_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    Year=New_Begin_Date[:4]

                    FY1=int(Year)+1

                    FY1=str(FY1)

                    date=FY1+';'+FY1

                    new_data=THS_DS(code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_Mutal_Fund_Netvalue(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_accum_unit_nv_fund','','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data[['ths_accum_unit_nv_fund']]

                Data.columns=[Code_List[i]]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Downloading_F_Score_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path = Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0, len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',Begin_Date,End_Date).data   

                Data.index = Data.loc[:, "time"]

                Data = Data.iloc[:, 1:]

                Data.to_csv(Code_List[i] + ".csv")

                print(str(i / len(Code_List)))

            except:

                print(Code_List[i] + "出现错误")

        print("下载结束")

    def Update_F_Score_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',New_Begin_Date,End_Date).data  
                    
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("个股财务数据更新结束")
        
    def Downloading_Cov_Bond_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',Begin_Date,End_Date).data
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Update_Cov_Bond_Vol_Price_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',New_Begin_Date,End_Date).data
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("可转债数据更新结束")

        return

    def Downloading_A50_Futures_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',Begin_Date,End_Date).data

                Data=Data.set_index('time',drop=True)

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_A50_Futures_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=  THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date).data
                        
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_newHL_Data(self,Begin_Date, End_Date, Export_Path, Code='001005010'):
        
        Begin_Date = pd.to_datetime(Begin_Date)
        End_Date = pd.to_datetime(End_Date)
        
        Gap = (End_Date - Begin_Date) / 250
        Period = Gap.days
        
        Timeseries = pd.date_range(Begin_Date, End_Date, periods=Period)
        Timeseries_List = Timeseries.strftime("%Y-%m-%d").tolist()
        
        Begin_Date_List = Timeseries_List[:-1]
        End_Date_List = Timeseries_List[1:]
        
        Data = []
        
        # 使用 tqdm 来显示进度条
        for i, j in tqdm(zip(Begin_Date_List, End_Date_List), total=len(Begin_Date_List), desc="Downloading Data"):
            
            DF = THS_DS(Code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', i, j).data
            
            Data.append(DF)
        
        Data = pd.concat(Data, axis=0)
        Data = Data.set_index('time', drop=True)
        
        os.chdir(Export_Path)
        Data.to_csv(Code+'.csv')
        
        return Data
     
    def Updating_newHL_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0) 

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_High_Freq_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path,Interval=60):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HF(Code_List[i],'open;high;low;close;volume;amount','Fill:Original,Interval:'+str(Interval),Begin_Date,End_Date).data
                
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def get_next_trading_time_1h(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_Hours
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "10:30"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("10:30", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")

    def get_next_trading_time_15min(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_15mins
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "09:45"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("09:45", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")
        
    def Update_High_Freq_Vol_Price_Data(self, End_Date, Export_Path, Interval=60):
        """
        更新高频数据 (小时级别)，确保数据不重复下载，并且增量更新。
        
        :param End_Date: 数据更新的结束时间（字符串格式 'YYYY-MM-DD HH:MM'）
        :param Export_Path: 导出文件夹的路径
        :param Interval: 数据的时间间隔，默认是 60 分钟（1 小时）
        """
        
        os.chdir(Export_Path)  # 切换到数据存放的目录
        
        # 获取所有以 .csv 结尾的文件名，并去掉 .csv 后缀，生成股票代码列表
        Code_List = [f.replace('.csv', '') for f in os.listdir(Export_Path) if f.endswith('.csv')]
        
        print("更新开始")
        
        if Interval == 60:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_1h(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                    else:
                        df_combined = new_data
                    
                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')
                    
                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")
        
        if Interval == 15:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_15min(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                        df_combined = df_combined[~df_combined.index.duplicated(keep='last')]  # 去重并保留最新的数据
                    else:
                        df_combined = new_data

                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')

                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")

    def Downloading_ETF_Option_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
            # 期权code 上证50=510050 沪深300=510300 500ETF=510500 科创50ETF=588000 科创板50=588080

            Export_Path=Export_Path

            print("下载开始")

            for i in range(0,len(Code_List)):

                try:
                    code=Code_List[i]+'O'
                    sdate=Begin_Date.replace('-','')
                    edate=End_Date.replace('-','')
                    Data=THS_DR('p02872','qqbd='+code+';'+'sdate='+sdate+';'+'edate='+edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    
                    Data=Data.set_index('p02872_f001',drop=True)

                    Data=Data.sort_index()

                    Data.to_csv(Export_Path+'\\'+Code_List[i]+".csv")

                    print(str(i/len(Code_List)))
                
                except:

                    print(Code_List[i]+"出现错误")

            print("下载结束")

    def Updating_ETF_Option(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code_original = List[i].replace('.csv', "")
                    code=code_original+'O'
                    code=Code_List[i]+'O'

                    new_sdate=New_Begin_Date.replace('-','')
                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p02872','qqbd='+code+';'+'sdate='+new_sdate+';'+'edate='+new_edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    new_data=new_data.set_index('p02872_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_DDE_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_DDE_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',i,j).data
                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_DDE_Data(i,Begin_Date,End_Date)

                Data=Data.set_index('time',drop=True)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_DDE_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                    THS_DS('801017.SL','ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data
                    new_data=THS_DS(code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)
                    
                    Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[0,0]

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return
    
        
        def Updating_updown_Data(End_Date, Data_Path):

            Data_Path=Data_Path

            os.chdir(Data_Path)

            List=os.listdir()

            Doc_Number=len(List)

            print("更新开始")

            for i in range(0,Doc_Number):

                try:

                    df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                    df.index=pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                        New_Begin_Date=df.index[-1]+Day()

                        New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                        code = List[i].replace('.csv', "")

                        new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                        new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                        new_data=new_data.iloc[:,2:]

                        Updated_Data=pd.concat([df,new_data],axis=0)

                        Updated_Data.to_csv(List[i])

                        print(str(i/Doc_Number))

                    else:

                        print("数据更新至最新")

                except:

                    print(List[i]+"出现错误")

            print("更新结束")

            return

    def Updating_updown_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_Up_Down_Companies(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code= List[i].replace('.csv', "")

                    new_sdate=New_Begin_Date.replace('-','')

                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p00112','sdate='+new_sdate+';'+'edate='+new_edate+';p0='+code,'p00112_f001:Y,p00112_f002:Y,p00112_f003:Y,p00112_f004:Y,p00112_f005:Y,p00112_f006:Y,p00112_f007:Y,p00112_f008:Y,p00112_f009:Y,p00112_f010:Y,p00112_f011:Y,p00112_f012:Y,p00112_f013:Y,p00112_f014:Y,p00112_f021:Y,p00112_f022:Y,p00112_f023:Y,p00112_f024:Y,p00112_f025:Y',
                                    'format:dataframe').data
                    new_data=new_data.set_index('p00112_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return


DUD=Downloading_And_Updating_Data()



def Update_All(End_Date):

    Price_Data_Path_1=r"E:\数据库\同花顺指数量价数据"

    Price_Data_Path_2=r"E:\数据库\同花顺ETF跟踪指数量价数据\1d"

    Price_Data_Path_3=r'E:\数据库\同花顺ETF量价数据'

    Price_Data_Path_4=r'E:\数据库\同花顺商品指数量价数据'

    Path_List=[Price_Data_Path_1,Price_Data_Path_2,Price_Data_Path_3,Price_Data_Path_4]

    DUD.Updating_All_Market_Vol_Data(End_Date,Path_List)

    Forcast_Path=r"E:\数据库\同花顺ETF跟踪指数一致预期数据\预测净利润两年复合增长"

    DUD.Updating_Forcast_2yr_Data(End_Date,Forcast_Path)

    Free_Turn_Path=r"E:\数据库\同花顺指数自由流通换手率"

    DUD.Updating_Index_Free_Turn_Data(End_Date,Free_Turn_Path)

    Valuation_Path=r"E:\数据库\同花顺ETF跟踪指数估值数据"

    DUD.Updating_Index_Valuation_Data(End_Date,Valuation_Path)

    EDB_Data_Path=r"E:\数据库\同花顺EDB数据"

    DUD.Update_EDB_Data(EDB_Data_Path,End_Date)

    ETF_Option_Path=r"E:\数据库\另类数据\ETF期权数据"

    DUD.Updating_ETF_Option(End_Date,ETF_Option_Path)

    Forcast_Data_Path=r'E:\数据库\同花顺ETF跟踪指数一致预期数据\盈利预测综合值'

    DUD.Update_Index_Forcast_Data(End_Date,Forcast_Data_Path)

    Cov_Bond_Path=r'E:\数据库\同花顺可转债数据'

    DUD.Update_Cov_Bond_Vol_Price_Data(End_Date,Cov_Bond_Path)

    #新高新低
    H_L_Path=r'E:\数据库\另类数据\新高新低'

    DUD.Updating_newHL_Data(End_Date,H_L_Path)

    #高频数据更新

    Export_Path_60 = r'E:\数据库\同花顺ETF跟踪指数量价数据\1h' 

    Export_Path_15=r'E:\数据库\同花顺ETF跟踪指数量价数据\15min'

    HF_End_Date=End_Date+' 15:00'

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date, Export_Path_60,60)

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date,Export_Path_15,15)

    #公司上涨数量更新
    
    Up_Down_Company_Path=r'E:\数据库\另类数据\涨跌家数'

    DUD.Updating_Up_Down_Companies(End_Date,Up_Down_Company_Path)

    #涨停跌停数量

    up_path=r'E:\数据库\另类数据\涨停跌停'

    DUD.Updating_updown_Data(End_Date,up_path)


    return print('updating finished')

#海外数据第二天更新
def Update_Oversea_Data(End_Date):

    A50_Path=r'E:\数据库\另类数据\A50期货数据'

    DUD.Updating_A50_Futures_Data(End_Date,A50_Path)

    return print('海外数据更新完毕')

Update_Oversea_Data('2024-12-25')

# Update_All('2024-12-25')

d:\Anaconda3\Lib\site-packages\iFinDPy.pth
0
登录成功
更新开始
CN0Y.SG.csv出现错误
更新结束
海外数据更新完毕


In [ ]:
    A50_Path=r'E:\数据库\另类数据\A50期货数据'


In [ ]:
Data_Path=A50_Path

In [ ]:

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)


In [ ]:
List

['CN0Y.SG.csv']

In [ ]:
i=0

In [ ]:
                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])


In [ ]:
df

,thscode,ths_open_price_future,ths_high_price_future,ths_low_future,ths_close_price_future,ths_basis_trading_rate_future
time,,,,,,
2016-10-28,CN0Y.SG,9915.0,9917.5,9885.0,9890.0,-0.195248
2016-10-31,CN0Y.SG,9867.5,9880.0,9850.0,9855.0,-0.031050
2016-11-01,CN0Y.SG,9917.5,9940.0,9882.5,9882.5,0.294561
2016-11-02,CN0Y.SG,9840.0,9852.5,9830.0,9842.5,-0.149454
2016-11-03,CN0Y.SG,9930.0,9950.0,9915.0,9915.0,0.235804
...,...,...,...,...,...,...
2024-12-18,CN0Y.SG,13408.0,13557.0,13390.0,13488.0,-0.124703
2024-12-19,CN0Y.SG,13476.0,13486.0,13325.0,13409.0,0.049966
2024-12-20,CN0Y.SG,13409.0,13426.0,13235.0,13289.0,0.034239


In [ ]:
End_Date='2024-12-25'

In [ ]:
                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=  THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date).data
                        

AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)


In [ ]:
df

,thscode,ths_open_price_future,ths_high_price_future,ths_low_future,ths_close_price_future,ths_basis_trading_rate_future
time,,,,,,
2016-10-28,CN0Y.SG,9915.0,9917.5,9885.0,9890.0,-0.195248
2016-10-31,CN0Y.SG,9867.5,9880.0,9850.0,9855.0,-0.031050
2016-11-01,CN0Y.SG,9917.5,9940.0,9882.5,9882.5,0.294561
2016-11-02,CN0Y.SG,9840.0,9852.5,9830.0,9842.5,-0.149454
2016-11-03,CN0Y.SG,9930.0,9950.0,9915.0,9915.0,0.235804
...,...,...,...,...,...,...
2024-12-18,CN0Y.SG,13408.0,13557.0,13390.0,13488.0,-0.124703
2024-12-19,CN0Y.SG,13476.0,13486.0,13325.0,13409.0,0.049966
2024-12-20,CN0Y.SG,13409.0,13426.0,13235.0,13289.0,0.034239


In [ ]:
                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:


IndentationError: expected an indented block (<ipython-input-13-2a4650f39dd8>, line 1)

In [ ]:
                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strtime("%Y-%m-%d")

AttributeError: 'Timestamp' object has no attribute 'strtime'

In [ ]:
End_Date'2024-12-25

SyntaxError: EOL while scanning string literal (<ipython-input-15-6728e4e49f3b>, line 1)

In [ ]:
End_Date='2024-12-25'

In [ ]:
                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strtime("%Y-%m-%d")

AttributeError: 'Timestamp' object has no attribute 'strtime'

In [ ]:
                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

In [ ]:
New_Begin_Date

'2024-12-25'

In [ ]:

                    code = List[i].replace('.csv', "")

                    new_data=  THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date).data
                        

In [ ]:
new_data

""


In [ ]:
THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date)

errorcode=0
errmsg=
data=Empty DataFrame
Columns: []
Index: []

In [ ]:
THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','','2024-12-23',End_Date)

errorcode=0
errmsg=
data=         time  thscode  ths_open_price_future  ths_high_price_future  \
0  2024-12-23  CN0Y.SG                13282.0                13524.0   
1  2024-12-24  CN0Y.SG                13426.0                13624.0   

   ths_low_future  ths_close_price_future  ths_basis_trading_rate_future  
0         13247.0                 13419.0                      -0.036590  
1         13394.0                 13595.0                       0.005443  

已连接到 base (Python 3.9.7)

In [ ]:
# -*- coding: utf-8 -*-
from iFinDPy import *
from datetime import datetime
import pandas as pd
import time as _time
import json
from threading import Thread,Lock,Semaphore
import requests
import numpy as np
from datetime import datetime
from numpy import nan as NA
import matplotlib.pyplot as plt
import math
from pandas.tseries.offsets import Day,MonthEnd
import os
from tqdm import tqdm  # 用于显示进度条
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("hwqh100","155d50")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')

thslogindemo()


class Downloading_And_Updating_Data():

    def __init__(self):

        self.A_Share_Trading_Hours= ["10:30", "11:30", "14:00", "15:00"]

        self.A_Share_Trading_15mins=["9:45", "10:00", "10:15", "10:30", 
                                    "10:45", "11:00", "11:15", "11:30",
                                     "13:15", "13:30", "13:45","14:00",
                                      "14:15", "14:30", "14:45", "15:00"]
    
    def Downloading_Market_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HQ(Code_List[i],'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_Market_Vol_Price_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_HQ(code,'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_All_Market_Vol_Data(self,End_Date, Data_Path_List):

        def Updating_Market_Vol_Price_Data(End_Date, Data_Path):
            os.chdir(Data_Path)  # 切换到数据路径

            List = os.listdir()  # 获取路径下所有文件
            Doc_Number = len(List)

            print("更新开始")

            def update_file(file_name):
                try:
                    file_path = os.path.join(Data_Path, file_name)
                    df = pd.read_csv(file_path, index_col=[0])
                    df.index = pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                        New_Begin_Date = (df.index[-1] + timedelta(days=1)).strftime("%Y-%m-%d")
                        code = file_name.replace('.csv', "")

                        # 获取新数据
                        new_data = THS_HQ(code, 'open,high,low,close,volume,amount,turnoverRatio,totalCapital,floatCapital', '', New_Begin_Date, End_Date).data
                        new_data.index = pd.to_datetime(new_data["time"])  # 将时间列设为索引
                        new_data = new_data.drop(columns=["time", "thscode"])  # 删除无关列

                        Updated_Data = pd.concat([df, new_data], axis=0)  # 合并新旧数据
                        Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[1,0]
                        Updated_Data.to_csv(file_path)  # 保存更新后的数据
                        return f"{file_name} 更新成功"
                    else:
                        return f"{file_name} 数据更新至最新"
                except Exception as e:
                    return f"{file_name} 出现错误: {e}"

            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = list(tqdm(executor.map(update_file, List), total=Doc_Number, desc="更新进度"))

            for result in results:
                print(result)

            print("更新结束")

        for data_path in Data_Path_List:
            try:
                Updating_Market_Vol_Price_Data(End_Date, data_path)
            except Exception as e:
                print(f"{data_path} 出现错误: {e}")

        return

    def Download_Index_Valuation_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_Valuation_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            Data=Data.set_index('time',drop=True)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_Valuation_Data(i,Begin_Date,End_Date)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_Valuation_Data(self,End_Date, Data_Path):
        def process_file(filename):
            try:
                df = pd.read_csv(os.path.join(Data_Path, filename), index_col=[0])
                df.index = pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d') != End_Date:
                    New_Begin_Date = df.index[-1] + Day()
                    New_Begin_Date = New_Begin_Date.strftime("%Y-%m-%d")

                    code = filename.replace('.csv', "")
                    new_data = THS_DS(code, 'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index', '100,100;100,100;', 'block:history', New_Begin_Date, End_Date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    Updated_Data = pd.concat([df, new_data], axis=0)
                    Updated_Data.to_csv(os.path.join(Data_Path, filename))

                    print(f"{filename} processed successfully.")
                else:
                    print(f"{filename} is already up to date.")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

        os.chdir(Data_Path)
        List = os.listdir()
        Doc_Number = len(List)

        print("更新开始")

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(process_file, filename) for filename in List]
            for i, future in enumerate(futures):
                try:
                    future.result()  # Wait for thread to complete and handle exceptions
                except Exception as e:
                    print(f"Thread {i} generated an exception: {e}")

        print("更新结束")

        return


        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_pe_ttm_index;ths_pb_index;ths_dividend_rate_index','100,100;100,100;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Free_Turn_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Free_Trun_Data(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                DF=THS_DS(Code,'ths_free_turnover_ratio_index','','block:history',i,j).data

                Data.append(DF)
            
            Data=pd.concat(Data,axis=0)

            return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Free_Trun_Data(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Updating_Index_Free_Turn_Data(self,end_date, data_path):
        """
        更新指定路径下的所有CSV文件中的数据，如果数据不是最新的，则从最新日期开始更新至指定的结束日期。
        
        :param end_date: 数据更新的截止日期，格式为'YYYY-MM-DD'
        :param data_path: 存放CSV文件的路径
        """
        def update_file(file_name):
            try:
                # 读取CSV文件
                df = pd.read_csv(os.path.join(data_path, file_name), index_col=[0])
                df.index = pd.to_datetime(df.index)

                # 检查数据是否已经是最新的
                if df.index[-1].strftime('%Y-%m-%d') != end_date:
                    # 计算新的开始日期
                    new_begin_date = (df.index[-1] + Day(1)).strftime("%Y-%m-%d")
                    code = file_name.replace('.csv', "")

                    # 获取新的数据
                    new_data = THS_DS(code, 'ths_free_turnover_ratio_index', '', 'block:history', new_begin_date, end_date).data
                    new_data.index = pd.to_datetime(new_data.loc[:, "time"])
                    new_data = new_data.iloc[:, 2:]

                    # 合并旧数据和新数据
                    updated_data = pd.concat([df, new_data], axis=0)

                    # 保存更新后的数据
                    updated_data.to_csv(os.path.join(data_path, file_name))

                    print(f"{file_name} 更新完成")
                else:
                    print(f"{file_name} 数据已是最新")

            except Exception as e:
                print(f"{file_name} 出现错误: {e}")

        # 获取路径下所有文件的列表
        file_list = os.listdir(data_path)

        print("更新开始")

        # 使用线程池来并行处理文件
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(update_file, file_name) for file_name in file_list]

            for future in futures:
                future.result()  # 等待所有线程完成

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

        #日期循环
            Begin_Date=Begin_Date

            Begin_Date=pd.to_datetime(Begin_Date)

            End_Date=End_Date

            End_Date=pd.to_datetime(End_Date)

            Gap=(End_Date-Begin_Date)/250

            Period=Gap.days

            Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

            Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

            Begin_Date_List=Timeseries_List[:-1]

            End_Date_List=Timeseries_List[1:]

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                FY2=int(Year)+2

                FY2=str(FY2)

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_np_index',Year,'block:history',i,j).data
                
                DF_FY1=THS_DS(Code,'ths_fore_np_index',FY1,'block:history',i,j).data

                DF_FY1=DF_FY1[["ths_fore_np_index"]]

                DF_FY2=THS_DS(Code,'ths_fore_np_index',FY2,'block:history',i,j).data

                DF_FY2=DF_FY2[["ths_fore_np_index"]]
                
                #财报数据下载

                DF_FY0=THS_DS(Code,'ths_np_index','','block:history',i,j).data

                DF_FY0=DF_FY0[["ths_np_index"]]

                Total=pd.concat([DF,DF_FY1,DF_FY2,DF_FY0],axis=1)

                Total.columns=["time","thscode","fore_np_FY0","fore_np_FY1","fore_np_FY2","np_index"]

                Data.append(Total)
            
            Data=pd.concat(Data,axis=0)

            return Data

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code
    
    def Download_Index_Forcast_2yr_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

            def Downloading_Forcast_2yr_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_fore_np_compound_growth_2y_index','','block:history',i,j).data

                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

            os.chdir(Export_Path)

            Error_Code=[]

            print("更新开始")
            
            for i in Code_List:

                try:
                    
                    Data=Downloading_Forcast_2yr_Data(i,Begin_Date,End_Date)

                    Data.index=Data.loc[:,"time"]

                    Data=Data.iloc[:,1:]

                    Data.to_csv(i+".csv")

                    print(i+"输出完成")

                except:

                    print(i+"出现错误")

                    Error_Code.append(i)
            
            return
    
    def Updating_Forcast_2yr_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_fore_np_compound_growth_2y_index','','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_EDB_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        for i in range(0,len(Code_List)):
        
            try:
                Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data.index=pd.to_datetime(Data.index)

                Data=Data.iloc[:,2:5]

                os.chdir(Export_Path)

                Data.to_csv(Code_List[i]+".csv")

                print('完成度: '+str(i/len(Code_List)))

            except:

                print(Code_List[i]+"出现错误")

        return

    def Update_EDB_Data(self,Data_Path,End_Date):

        def Download_EDB_Data(Code_List,Begin_Date,End_Date,Export_Path):

                for i in range(0,len(Code_List)):
                
                    try:
                        Data=THS_EDB(Code_List[i],'',Begin_Date,End_Date).data

                        Data.index=Data.loc[:,"time"]

                        Data.index=pd.to_datetime(Data.index)

                        Data=Data.iloc[:,2:5]

                        os.chdir(Export_Path)

                        Data.to_csv(Code_List[i]+".csv")

                        print('完成度: '+str(i/len(Code_List)))

                    except:

                        print(Code_List[i]+"出现错误")

                return

        List=os.listdir(Data_Path)

        Code_List=[]

        for i in List:

            Code=i.replace(".csv","")

            Code_List.append(Code)

        Download_EDB_Data(Code_List,"2003-01-01",End_Date,Data_Path)

        return

    def Downloading_ETF_Option_PCR_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
    
        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_ETF_Option_PCR(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data= THS_DS(code,'ths_option_total_volume_pcr_option;ths_option_total_oi_pcr_option;ths_option_total_amount_pcr_option',';;','',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_Forcast_Neteprofit(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Forcast_Neteprofit(Code,Begin_Date,End_Date):

            def generate_date_intervals(start_date, end_date):
                # 将输入的日期字符串转换为datetime对象
                start_date = datetime.strptime(start_date, "%Y-%m-%d")
                end_date = datetime.strptime(end_date, "%Y-%m-%d")

                # 初始化开始日期和结束日期的列表
                begin_date_list = []
                end_date_list = []

                temp_date = start_date
                while temp_date <= end_date:
                    # 将当前日期添加到开始日期列表
                    begin_date_list.append(temp_date.strftime("%Y-%m-%d"))

                    if temp_date.year == end_date.year:
                        # 如果是结束日期的年份，那么结束日期就是输入的结束日期
                        temp_end_date = end_date
                    else:
                        # 否则，结束日期是当前年份的12月31日
                        temp_end_date = datetime(temp_date.year, 12, 31)

                    # 将结束日期添加到结束日期列表
                    end_date_list.append(temp_end_date.strftime("%Y-%m-%d"))

                    # 将临时日期设置为下一年的1月1日
                    temp_date = datetime(temp_date.year + 1, 1, 1)

                return begin_date_list, end_date_list

            Begin_Date_List,End_Date_List=generate_date_intervals(Begin_Date,End_Date)

            Data=[]

        #日期循环下载数据

            for i,j in zip(Begin_Date_List,End_Date_List):
                
                Year=i[:4]

                FY1=int(Year)+1

                FY1=str(FY1)

                date=FY1+';'+FY1

                #一致预期数据下载
                DF=THS_DS(Code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',i,j).data

                Data.append(DF)
            
            result=pd.concat(Data,axis=0)

            return result

        #code list循环下载
        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        # 使用tqdm包装Code_List
        for i in Code_List: 

            try:
                
                Data=Downloading_Forcast_Neteprofit(i,Begin_Date,End_Date)

                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(i+".csv")

                print(i+"输出完成")

        #将出现错误的Code编成list
            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return Error_Code

    def Update_Index_Forcast_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    Year=New_Begin_Date[:4]

                    FY1=int(Year)+1

                    FY1=str(FY1)

                    date=FY1+';'+FY1

                    new_data=THS_DS(code,'ths_fore_eps_index;ths_fore_roe_mean_index',date,'block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_Mutal_Fund_Netvalue(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_accum_unit_nv_fund','','',Begin_Date,End_Date).data

                Data.index=Data.loc[:,"time"]

                Data=Data[['ths_accum_unit_nv_fund']]

                Data.columns=[Code_List[i]]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Downloading_F_Score_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path = Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0, len(Code_List)):

            try:
                Data=THS_DS(Code_List[i],'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',Begin_Date,End_Date).data   

                Data.index = Data.loc[:, "time"]

                Data = Data.iloc[:, 1:]

                Data.to_csv(Code_List[i] + ".csv")

                print(str(i / len(Code_List)))

            except:

                print(Code_List[i] + "出现错误")

        print("下载结束")

    def Update_F_Score_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_np_atoopc_pit_stock;ths_total_assets_pit_stock;ths_ncf_from_oa_pit_stock;ths_total_liab_pit_stock;ths_total_current_assets_pit_stock;ths_total_current_liab_pit_stock;ths_total_shares_stock;ths_operating_total_revenue_stock;ths_gross_selling_rate_stock','0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;0@103,1;;100;','',New_Begin_Date,End_Date).data  
                    
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("个股财务数据更新结束")
        
    def Downloading_Cov_Bond_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',Begin_Date,End_Date).data
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Update_Cov_Bond_Vol_Price_Data(self,End_Date,Data_Path):

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                            
                    new_data=THS_DS(code,'ths_open_price_bond;ths_high_price_bond;ths_low_bond;ths_close_daily_settle_bond;ths_vol_yz_bond;ths_trans_amt_bond','103;103;103;103;100;','',New_Begin_Date,End_Date).data
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("可转债数据更新结束")

        return

    def Downloading_A50_Futures_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_DS(Code_List[i],'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',Begin_Date,End_Date).data

                Data=Data.set_index('time',drop=True)

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def Updating_A50_Futures_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=  THS_DS(code,'ths_open_price_future;ths_high_price_future;ths_low_future;ths_close_price_future;ths_basis_trading_rate_future',';;;;','',New_Begin_Date,End_Date).data
                        
                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,1:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_newHL_Data(self,Begin_Date, End_Date, Export_Path, Code='001005010'):
        
        Begin_Date = pd.to_datetime(Begin_Date)
        End_Date = pd.to_datetime(End_Date)
        
        Gap = (End_Date - Begin_Date) / 250
        Period = Gap.days
        
        Timeseries = pd.date_range(Begin_Date, End_Date, periods=Period)
        Timeseries_List = Timeseries.strftime("%Y-%m-%d").tolist()
        
        Begin_Date_List = Timeseries_List[:-1]
        End_Date_List = Timeseries_List[1:]
        
        Data = []
        
        # 使用 tqdm 来显示进度条
        for i, j in tqdm(zip(Begin_Date_List, End_Date_List), total=len(Begin_Date_List), desc="Downloading Data"):
            
            DF = THS_DS(Code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', i, j).data
            
            Data.append(DF)
        
        Data = pd.concat(Data, axis=0)
        Data = Data.set_index('time', drop=True)
        
        os.chdir(Export_Path)
        Data.to_csv(Code+'.csv')
        
        return Data
     
    def Updating_newHL_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code, 'ths_new_high_num_block;ths_new_low_num_block', '1,250,100;1,250,100', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0) 

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Downloading_High_Freq_Vol_Price_Data(self,Code_List,Begin_Date,End_Date,Export_Path,Interval=60):

        Export_Path=Export_Path

        os.chdir(Export_Path)

        print("下载开始")

        for i in range(0,len(Code_List)):

            try:

                Data=THS_HF(Code_List[i],'open;high;low;close;volume;amount','Fill:Original,Interval:'+str(Interval),Begin_Date,End_Date).data
                
                Data.index=Data.loc[:,"time"]

                Data=Data.iloc[:,1:]

                Data.to_csv(Code_List[i]+".csv")

                print(str(i/len(Code_List)))
            
            except:

                print(Code_List[i]+"出现错误")

        print("下载结束")

    def get_next_trading_time_1h(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_Hours
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "10:30"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("10:30", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")

    def get_next_trading_time_15min(self,last_time):
        """
        根据最后一条数据的时间，计算下一个交易时间。
        :param last_time: 最后一条数据的时间 (pd.Timestamp)
        :return: 下一个交易时间 (str)
        """
        last_date = last_time.date()
        current_time_str = last_time.strftime("%H:%M")
        TRADING_HOURS=self.A_Share_Trading_15mins
        # 查找下一个交易时间
        for trading_time in TRADING_HOURS:
            if current_time_str < trading_time:
                next_trading_time = datetime.combine(last_date, datetime.strptime(trading_time, "%H:%M").time())
                return next_trading_time.strftime("%Y-%m-%d %H:%M")
        
        # 如果当天所有交易时间都过了，返回下一个交易日的第一个交易时间 "09:45"
        next_trading_day = last_date + timedelta(days=1)
        return datetime.combine(next_trading_day, datetime.strptime("09:45", "%H:%M").time()).strftime("%Y-%m-%d %H:%M")
        
    def Update_High_Freq_Vol_Price_Data(self, End_Date, Export_Path, Interval=60):
        """
        更新高频数据 (小时级别)，确保数据不重复下载，并且增量更新。
        
        :param End_Date: 数据更新的结束时间（字符串格式 'YYYY-MM-DD HH:MM'）
        :param Export_Path: 导出文件夹的路径
        :param Interval: 数据的时间间隔，默认是 60 分钟（1 小时）
        """
        
        os.chdir(Export_Path)  # 切换到数据存放的目录
        
        # 获取所有以 .csv 结尾的文件名，并去掉 .csv 后缀，生成股票代码列表
        Code_List = [f.replace('.csv', '') for f in os.listdir(Export_Path) if f.endswith('.csv')]
        
        print("更新开始")
        
        if Interval == 60:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_1h(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                    else:
                        df_combined = new_data
                    
                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')
                    
                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")
        
        if Interval == 15:
            # 使用 tqdm 显示进度条
            for code in tqdm(Code_List):
                try:
                    file_path = os.path.join(Export_Path, f"{code}.csv")
                    
                    # 读取现有的 CSV 文件
                    if os.path.exists(file_path):
                        df_existing = pd.read_csv(file_path, parse_dates=['time'])
                        df_existing.set_index('time', inplace=True)  # 确保 'time' 列作为索引
                        
                        # 获取最后一条数据的时间，并确保是 pd.Timestamp 类型
                        last_time = pd.to_datetime(df_existing.index.max())
                        
                        # 获取下一个交易时间
                        start_time = self.get_next_trading_time_15min(last_time)
                        
                        # 如果最后一条时间已经超过了指定的结束时间，则跳过更新
                        if pd.to_datetime(start_time) > pd.to_datetime(End_Date):
                            print(f"{code} 数据已经是最新的，跳过更新。")
                            continue
                    else:
                        # 如果文件不存在，从头开始下载
                        start_time = "2010-01-01 00:00"
                        df_existing = pd.DataFrame()  # 空的 DataFrame 用于拼接

                    # 调用 API 下载新数据，从 start_time 到 End_Date
                    new_data = THS_HF(code, 'open;high;low;close;volume;amount', 
                                    f'Fill:Original,Interval:{Interval}', start_time, End_Date).data

                    # 确保 'time' 列存在，将其设置为索引
                    if 'time' in new_data.columns:
                        new_data.set_index('time', inplace=True)

                    # 如果没有新数据，跳过处理
                    if new_data.empty:
                        print(f"{code} 无新数据需要更新。")
                        continue

                    # 将新数据与旧数据拼接，并去重
                    if not df_existing.empty:
                        df_combined = pd.concat([df_existing, new_data])
                        df_combined = df_combined[~df_combined.index.duplicated(keep='last')]  # 去重并保留最新的数据
                    else:
                        df_combined = new_data

                    # 将索引转换为日期时间格式
                    df_combined.index = pd.to_datetime(df_combined.index, errors='coerce')

                    # 检查是否有 NaT 索引，删除无效的时间索引
                    df_combined = df_combined[df_combined.index.notna()]

                    # 保存更新后的数据，确保 'time' 列作为索引
                    df_combined.to_csv(file_path, index=True)  # index=True 保留时间索引
                    print(f"{code} 数据更新成功。")

                except Exception as e:
                    print(f"{code} 更新时出现错误: {e}")
            
            print("数据更新结束")

    def Downloading_ETF_Option_Data(self,Code_List,Begin_Date,End_Date,Export_Path):
            # 期权code 上证50=510050 沪深300=510300 500ETF=510500 科创50ETF=588000 科创板50=588080

            Export_Path=Export_Path

            print("下载开始")

            for i in range(0,len(Code_List)):

                try:
                    code=Code_List[i]+'O'
                    sdate=Begin_Date.replace('-','')
                    edate=End_Date.replace('-','')
                    Data=THS_DR('p02872','qqbd='+code+';'+'sdate='+sdate+';'+'edate='+edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    
                    Data=Data.set_index('p02872_f001',drop=True)

                    Data=Data.sort_index()

                    Data.to_csv(Export_Path+'\\'+Code_List[i]+".csv")

                    print(str(i/len(Code_List)))
                
                except:

                    print(Code_List[i]+"出现错误")

            print("下载结束")

    def Updating_ETF_Option(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code_original = List[i].replace('.csv', "")
                    code=code_original+'O'
                    code=Code_List[i]+'O'

                    new_sdate=New_Begin_Date.replace('-','')
                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p02872','qqbd='+code+';'+'sdate='+new_sdate+';'+'edate='+new_edate+';jys=212001;bdlx=基金;date=全部',
                        'p02872_f001:Y,p02872_f002:Y,p02872_f003:Y,p02872_f004:Y,p02872_f005:Y,p02872_f006:Y,p02872_f007:Y,p02872_f008:Y,p02872_f009:Y,p02872_f010:Y,p02872_f014:Y,p02872_f011:Y,p02872_f015:Y,p02872_f016:Y,p02872_f017:Y',
                        'format:dataframe').data
                    new_data=new_data.set_index('p02872_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Download_Index_DDE_Data(self,Code_List,Begin_Date,End_Date,Export_Path):

        def Downloading_Index_DDE_Data(Code,Begin_Date,End_Date):

            #日期循环
                Begin_Date=Begin_Date

                Begin_Date=pd.to_datetime(Begin_Date)

                End_Date=End_Date

                End_Date=pd.to_datetime(End_Date)

                Gap=(End_Date-Begin_Date)/250

                Period=Gap.days

                Timeseries=pd.date_range(Begin_Date,End_Date,periods=Period)

                Timeseries_List=Timeseries.strftime("%Y-%m-%d").tolist()

                Begin_Date_List=Timeseries_List[:-1]

                End_Date_List=Timeseries_List[1:]

                Data=[]

            #日期循环下载数据

                for i,j in zip(Begin_Date_List,End_Date_List):
                    
                    DF=THS_DS(Code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',i,j).data
                    Data.append(DF)
                
                Data=pd.concat(Data,axis=0)

                return Data

        os.chdir(Export_Path)

        Error_Code=[]

        print("更新开始")
        
        for i in Code_List:

            try:
                
                Data=Downloading_Index_DDE_Data(i,Begin_Date,End_Date)

                Data=Data.set_index('time',drop=True)

                Data.to_csv(i+".csv")

                print(i+"输出完成")

            except:

                print(i+"出现错误")

                Error_Code.append(i)
        
        return

    def Updating_Index_DDE_Data(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")
                    THS_DS('801017.SL','ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data
                    new_data=THS_DS(code,'ths_dde_5d_hb_index;ths_dde_10d_hb_index;ths_dde_20d_hb_index',';;','block:history',New_Begin_Date,End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)
                    
                    Updated_Data.loc[:,"thscode"]=Updated_Data.iloc[0,0]

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return
    
        
        def Updating_updown_Data(End_Date, Data_Path):

            Data_Path=Data_Path

            os.chdir(Data_Path)

            List=os.listdir()

            Doc_Number=len(List)

            print("更新开始")

            for i in range(0,Doc_Number):

                try:

                    df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                    df.index=pd.to_datetime(df.index)

                    if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                        New_Begin_Date=df.index[-1]+Day()

                        New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                        code = List[i].replace('.csv', "")

                        new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                        new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                        new_data=new_data.iloc[:,2:]

                        Updated_Data=pd.concat([df,new_data],axis=0)

                        Updated_Data.to_csv(List[i])

                        print(str(i/Doc_Number))

                    else:

                        print("数据更新至最新")

                except:

                    print(List[i]+"出现错误")

            print("更新结束")

            return

    def Updating_updown_Data(self,End_Date, Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code = List[i].replace('.csv', "")

                    new_data=THS_DS(code,'ths_limit_up_stock_num_sector;ths_limit_down_stock_num_sector',';', 'block:latest', New_Begin_Date, End_Date).data

                    new_data.index=pd.to_datetime(new_data.loc[:,"time"])

                    new_data=new_data.iloc[:,2:]

                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                    print(str(i/Doc_Number))

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return

    def Updating_Up_Down_Companies(self,End_Date,Data_Path):

        Data_Path=Data_Path

        os.chdir(Data_Path)

        List=os.listdir()

        Doc_Number=len(List)

        print("更新开始")

        for i in range(0,Doc_Number):

            try:

                df=pd.read_csv(Data_Path+"\\"+List[i],index_col=[0])

                df.index=pd.to_datetime(df.index)

                if df.index[-1].strftime('%Y-%m-%d')!=End_Date:

                    New_Begin_Date=df.index[-1]+Day()

                    New_Begin_Date=New_Begin_Date.strftime("%Y-%m-%d")

                    code= List[i].replace('.csv', "")

                    new_sdate=New_Begin_Date.replace('-','')

                    new_edate=End_Date.replace('-','')

                    new_data=THS_DR('p00112','sdate='+new_sdate+';'+'edate='+new_edate+';p0='+code,'p00112_f001:Y,p00112_f002:Y,p00112_f003:Y,p00112_f004:Y,p00112_f005:Y,p00112_f006:Y,p00112_f007:Y,p00112_f008:Y,p00112_f009:Y,p00112_f010:Y,p00112_f011:Y,p00112_f012:Y,p00112_f013:Y,p00112_f014:Y,p00112_f021:Y,p00112_f022:Y,p00112_f023:Y,p00112_f024:Y,p00112_f025:Y',
                                    'format:dataframe').data
                    new_data=new_data.set_index('p00112_f001',drop=True)
                    new_data.index=pd.to_datetime(new_data.index)
                    new_data=new_data.sort_index()
                    Updated_Data=pd.concat([df,new_data],axis=0)

                    Updated_Data.to_csv(List[i])

                else:

                    print("数据更新至最新")

            except:

                print(List[i]+"出现错误")

        print("更新结束")

        return


DUD=Downloading_And_Updating_Data()



def Update_All(End_Date):

    Price_Data_Path_1=r"E:\数据库\同花顺指数量价数据"

    Price_Data_Path_2=r"E:\数据库\同花顺ETF跟踪指数量价数据\1d"

    Price_Data_Path_3=r'E:\数据库\同花顺ETF量价数据'

    Price_Data_Path_4=r'E:\数据库\同花顺商品指数量价数据'

    Path_List=[Price_Data_Path_1,Price_Data_Path_2,Price_Data_Path_3,Price_Data_Path_4]

    DUD.Updating_All_Market_Vol_Data(End_Date,Path_List)

    Forcast_Path=r"E:\数据库\同花顺ETF跟踪指数一致预期数据\预测净利润两年复合增长"

    DUD.Updating_Forcast_2yr_Data(End_Date,Forcast_Path)

    Free_Turn_Path=r"E:\数据库\同花顺指数自由流通换手率"

    DUD.Updating_Index_Free_Turn_Data(End_Date,Free_Turn_Path)

    Valuation_Path=r"E:\数据库\同花顺ETF跟踪指数估值数据"

    DUD.Updating_Index_Valuation_Data(End_Date,Valuation_Path)

    EDB_Data_Path=r"E:\数据库\同花顺EDB数据"

    DUD.Update_EDB_Data(EDB_Data_Path,End_Date)

    ETF_Option_Path=r"E:\数据库\另类数据\ETF期权数据"

    DUD.Updating_ETF_Option(End_Date,ETF_Option_Path)

    Forcast_Data_Path=r'E:\数据库\同花顺ETF跟踪指数一致预期数据\盈利预测综合值'

    DUD.Update_Index_Forcast_Data(End_Date,Forcast_Data_Path)

    Cov_Bond_Path=r'E:\数据库\同花顺可转债数据'

    DUD.Update_Cov_Bond_Vol_Price_Data(End_Date,Cov_Bond_Path)

    #新高新低
    H_L_Path=r'E:\数据库\另类数据\新高新低'

    DUD.Updating_newHL_Data(End_Date,H_L_Path)

    #高频数据更新

    Export_Path_60 = r'E:\数据库\同花顺ETF跟踪指数量价数据\1h' 

    Export_Path_15=r'E:\数据库\同花顺ETF跟踪指数量价数据\15min'

    HF_End_Date=End_Date+' 15:00'

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date, Export_Path_60,60)

    DUD.Update_High_Freq_Vol_Price_Data(HF_End_Date,Export_Path_15,15)

    #公司上涨数量更新
    
    Up_Down_Company_Path=r'E:\数据库\另类数据\涨跌家数'

    DUD.Updating_Up_Down_Companies(End_Date,Up_Down_Company_Path)

    #涨停跌停数量

    up_path=r'E:\数据库\另类数据\涨停跌停'

    DUD.Updating_updown_Data(End_Date,up_path)


    return print('updating finished')

#海外数据第二天更新
def Update_Oversea_Data(End_Date):

    A50_Path=r'E:\数据库\另类数据\A50期货数据'

    DUD.Updating_A50_Futures_Data(End_Date,A50_Path)

    return print('海外数据更新完毕')

Update_Oversea_Data('2024-12-25')

# Update_All('2024-12-25')

d:\Anaconda3\Lib\site-packages\iFinDPy.pth
0
登录成功
更新开始
CN0Y.SG.csv出现错误
更新结束
海外数据更新完毕


In [ ]:
def Update_Oversea_Data(End_Date):

    A50_Path=r'E:\数据库\另类数据\A50期货数据'

    DUD.Updating_A50_Futures_Data(End_Date,A50_Path)

    return print('海外数据更新完毕')

Update_Oversea_Data('2024-12-26')

更新开始
更新结束
海外数据更新完毕


已连接到 base (Python 3.9.7)

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#数据路径设置

result_path=r'C:\Users\Wesle\Desktop\srcb_daily_report\result'

data_path=r'C:\Users\Wesle\Desktop\srcb_daily_report\原始数据'

#部门归属表文件名
department_list='员工部门归属表.xlsx'

#昨日日报表文件名
yesterday_daily_report='网点鑫e贷月度指标完成情况1226.xlsx'

#【浦东分行鑫e贷】客户经理营销数据文件名
client_manager_data='【浦东分行鑫e贷】客户经理营销数据_2024-12-26.xlsx'

retail_performance_data='零售市场部协同外拓及理财转介业绩报送-9.xlsx'

type_B_data='【浦东分行鑫e贷】鑫e贷b款明细_2024-12-26.xlsx'

T0_Date='2024-12-26'


#网点鑫e贷月度指标完成情况

#鑫e贷总授信
def XY_Dai_Zong_Shou_Xin(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,result_path):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_daily_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:45,2:]

    daily_report.index=index_list

    daily_report.columns=new_columns

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷总授信_指标','鑫e贷放款_指标','鑫e贷授信-B款_指标']

    kpi_result=kpi[['鑫e贷总授信_指标']]
    
    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['完成数']]

    yesterday_finished.columns=['鑫e贷总授信_昨日完成数','鑫e贷放款_昨日完成数','鑫e贷授信-B款_昨日完成数']

    yesterday_finished_result=yesterday_finished[['鑫e贷总授信_昨日完成数']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果(报表数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_shouxin=client_manager_df[['benyueshouxinrenshu']]

    manager_total_shouxin = monthly_shouxin.groupby(monthly_shouxin.index).sum()

    department_df=pd.read_excel(data_path+'\\'+department_list)

    # 重命名列，确保列名一致，便于匹配
    department_df.rename(columns={'员工姓名': 'kehujinglixingm', '部门': 'department'}, inplace=True)

    # 合并客户经理业绩数据和部门数据
    merged_df = manager_total_shouxin.reset_index().merge(department_df, on='kehujinglixingm', how='left')
    
    # 按部门分组，计算业绩总和
    department_totals =  merged_df.groupby('department', as_index=False)['benyueshouxinrenshu'].sum()

    department_totals=department_totals.set_index('department',drop=True)
    #匹配到浦东分行的表里

    result.loc[:,"鑫e贷总授信_报表数"]=department_totals


    #第四个结果(协同外拓)

    #提取昨日协同外拓数

    yesterday_retail_performance=daily_report[['协同外拓']]

    yesterday_retail_performance.columns=['鑫e贷总授信_昨日协同外拓','鑫e贷放款_昨日协同外拓','鑫e贷授信-B款_昨日协同外拓']

    yesterday_retail_performance=yesterday_retail_performance[['鑫e贷总授信_昨日协同外拓']]
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','协同外拓网点','其中本人\nA款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷总授信_今日协同外拓"]=today_retail_df.loc[:,"其中本人\nA款授信（户）"]+today_retail_df.loc[:,"其中本人\nB款授信（户）"]*2
    
    today_retail_result=today_retail_df[['协同外拓网点','鑫e贷总授信_今日协同外拓']]

    today_retail_result=today_retail_result.set_index('协同外拓网点',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷总授信_协同外拓"] = (
        newest_result.loc[:, "鑫e贷总授信_昨日协同外拓"].fillna(0) +
        newest_result.loc[:, "鑫e贷总授信_今日协同外拓"].fillna(0)
    )

    result.loc[:,"鑫e贷总授信_协同外拓"]=newest_result.loc[:, "鑫e贷总授信_协同外拓"]

    result.loc[:,"B款月底额外计1户授信"]=0

    #取昨日数据调整数

    yesterday_daily_adjusted=daily_report[['数据调整数']]

    result.loc[:,"数据调整数"]=yesterday_daily_adjusted

    #完成数

    result.loc[:,"鑫e贷总授信_完成数"]=result[['鑫e贷总授信_报表数','鑫e贷总授信_协同外拓',
                                'B款月底额外计1户授信','数据调整数']].sum(axis=1)
    # 计算完成率（完成数 / 指标），保留小数点后两位
    result.loc[:, "鑫e贷总授信_完成率"] = (
        result.loc[:, "鑫e贷总授信_完成数"] / result.loc[:, "鑫e贷总授信_指标"]
    ).fillna(0).round(2)
    
    result.loc[:,"鑫e贷总授信_昨日完成数(轧差)"]=result.loc[:,"鑫e贷总授信_完成数"]-result.loc[:,"鑫e贷总授信_昨日完成数"]

    final_result=result[['鑫e贷总授信_指标','鑫e贷总授信_昨日完成数(轧差)',
                            '鑫e贷总授信_报表数','鑫e贷总授信_协同外拓','B款月底额外计1户授信','数据调整数',
                                '鑫e贷总授信_完成数','鑫e贷总授信_完成率']]

    final_result.to_excel(result_path+'\\'+'鑫e贷总授信完成情况.xlsx')

    print('恭喜米，鑫e贷总授信计算完成')

    return final_result

#鑫e贷放款
def XY_Dai_Fang_Kuang(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):
    
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_daily_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:45,2:]

    daily_report.index=index_list

    daily_report.columns=new_columns

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷总授信_指标','鑫e贷放款_指标','鑫e贷授信-B款_指标']

    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['完成数']]

    yesterday_finished.columns=['鑫e贷总授信_昨日完成数','鑫e贷放款_昨日完成数','鑫e贷授信-B款_昨日完成数']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日完成数']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果（报表数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()

    department_df=pd.read_excel(data_path+'\\'+department_list)

    # 重命名列，确保列名一致，便于匹配
    department_df.rename(columns={'员工姓名': 'kehujinglixingm', '部门': 'department'}, inplace=True)

    # 合并客户经理业绩数据和部门数据
    merged_df = manager_total_fangkuan.reset_index().merge(department_df, on='kehujinglixingm', how='left')
    
    # 按部门分组，计算业绩总和
    department_totals =  merged_df.groupby('department', as_index=False)['benyuefangkuanjine'].sum()

    department_totals=department_totals.set_index('department',drop=True)

    department_totals=department_totals/10000

    department_totals=department_totals.fillna(0).round(0)

    department_totals.columns=['鑫e贷放款_报表数']

    result.loc[:,"鑫e贷放款_报表数"]=department_totals
    
    #第四个结果（自然流量）

    #读取b款数据

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    department_df_1=pd.read_excel(data_path+'\\'+department_list)

    # 重命名列，确保列名一致，便于匹配
    department_df_1.rename(columns={'员工姓名': 'jingdiaokehujingl', '部门': 'department'}, inplace=True)

    # 合并客户经理业绩数据和部门数据
    netural_merged_df = netural_result.reset_index().merge(department_df_1, on='jingdiaokehujingl', how='left')
    
    netural_department_totals =  netural_merged_df.groupby('department', as_index=False)['fangkuanjine'].sum()

    netural_department_totals=netural_department_totals.set_index('department',drop=True)

    netural_department_totals=netural_department_totals/10000

    netural_department_totals=netural_department_totals.fillna(0).round(2)

    netural_department_totals.columns=['鑫e贷放款_自然流量']

    result.loc[:,"鑫e贷放款_自然流量"]=netural_department_totals

    #第四个指标（协同外拓）

    yesterday_retail_performance=daily_report[['协同外拓']]

    yesterday_retail_performance.columns=['鑫e贷总授信_昨日协同外拓','鑫e贷放款_昨日协同外拓','鑫e贷授信-B款_昨日协同外拓']

    result.loc[:,"鑫e贷放款_协同外拓"]=yesterday_retail_performance[['鑫e贷放款_昨日协同外拓']]

    result.loc[:,"鑫e贷放款_完成数"]=result[['鑫e贷放款_报表数','鑫e贷放款_自然流量',
                                '鑫e贷放款_协同外拓']].sum(axis=1)

    result.loc[:, "鑫e贷放款_完成率"] = (result.loc[:, "鑫e贷放款_完成数"] /
                             result.loc[:, "鑫e贷放款_指标"]).fillna(0).round(2)
    
    result.loc[:,"鑫e贷放款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷放款_完成数"]-result.loc[:,"鑫e贷放款_昨日完成数"]

    final_result=result[['鑫e贷放款_指标','鑫e贷放款_昨日完成数(轧差)',
                        '鑫e贷放款_报表数','鑫e贷放款_自然流量','鑫e贷放款_协同外拓','鑫e贷放款_完成数',
                            '鑫e贷放款_完成率']]
                            
    # final_result.sort_values('鑫e贷放款_完成率', ascending=True)[['鑫e贷放款_完成率']].plot(
    #     kind='barh', 
    #     figsize=(10, 15)
    # )
    # plt.title('团队KPI完成率对比', fontsize=14)
    # plt.tight_layout()
    # plt.show()

    final_result.to_excel(result_path+'\\'+'鑫e贷放款完成情况.xlsx')
    
    print('恭喜米，鑫e贷放款计算完成')

    return final_result

#B宽授信
def B_Kuang_Shou_Xin(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_daily_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:45,2:]

    daily_report.index=index_list

    daily_report.columns=new_columns

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷总授信_指标','鑫e贷放款_指标','鑫e贷授信-B款_指标']

    kpi_result=kpi[['鑫e贷授信-B款_指标']]

    result=pd.DataFrame(index=kpi_result.index)

    result.loc[:,"鑫e贷授信-B款_指标"]=kpi_result

    #第二个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    department_df=pd.read_excel(data_path+'\\'+department_list)

    # 重命名列，确保列名一致，便于匹配
    department_df.rename(columns={'员工姓名': 'jingdiaokehujingli', '部门': 'department'}, inplace=True)

    department_df=department_df.set_index('jingdiaokehujingli',drop=True)

    # 合并客户经理业绩数据和部门数据
    merged_df =pd.merge(DD_result,department_df,right_index=True,left_index=True)
    
    merged_df.columns=['count','department']
    # 按部门分组，计算业绩总和
    department_totals =  merged_df.groupby('department', as_index=False)['count'].sum()

    department_totals=department_totals.set_index('department',drop=True)

    department_totals.columns=['鑫e贷授信-B款_报表数']

    result.loc[:,"鑫e贷授信-B款_报表数"]=department_totals

    #第三个结果（协同外拓）

    yesterday_retail_performance=daily_report[['协同外拓']]

    yesterday_retail_performance.columns=['鑫e贷总授信_昨日协同外拓','鑫e贷放款_昨日协同外拓','鑫e贷授信-B款_昨日协同外拓']

    yesterday_retail_performance=yesterday_retail_performance[['鑫e贷授信-B款_昨日协同外拓']]
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]
    
    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df[['客户经理姓名','协同外拓网点','其中本人\nA款授信（户）','其中本人\nB款授信（户）']]
    
    today_retail_df=today_retail_df.fillna(0)
    
    today_retail_df.loc[:,"鑫e贷授信-B款_今日协同外拓"]=today_retail_df.loc[:,"其中本人\nB款授信（户）"].fillna(0)
    
    today_retail_result=today_retail_df[['协同外拓网点','鑫e贷授信-B款_今日协同外拓']]

    today_retail_result=today_retail_result.set_index('协同外拓网点',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款_协同外拓"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日协同外拓"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款_今日协同外拓"].fillna(0)
        )

    result.loc[:,"鑫e贷授信-B款_协同外拓"]=newest_result.loc[:, "鑫e贷授信-B款_协同外拓"]

    #第四个结果（调整数）

    result.loc[:,"鑫e贷授信-B款_调整数"]=0

    #第五个结果（完成数）

    result.loc[:,"鑫e贷授信-B款_完成数"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_协同外拓',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

    result.loc[:, "鑫e贷授信-B款_完成率"] = (result.loc[:, "鑫e贷授信-B款_完成数"] /
                             result.loc[:, "鑫e贷授信-B款_指标"]).fillna(0).round(2)
                                   
    result.to_excel(result_path+'\\'+'鑫e贷授信-B款完成情况.xlsx')

    print('恭喜米，鑫e贷授信-B款计算完成')

    return result



XY_Dai_Zong_Shou_Xin_result=XY_Dai_Zong_Shou_Xin(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,result_path)


XY_Dai_Fang_Kuang_result=XY_Dai_Fang_Kuang(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)

B_Kuang_Shou_Xin_result=B_Kuang_Shou_Xin(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)


total=pd.concat([XY_Dai_Zong_Shou_Xin_result,XY_Dai_Fang_Kuang_result,B_Kuang_Shou_Xin_result],axis=1)

total.to_excel(result_path+'\\'+'日报总表.xlsx')


#零售团队月度日报

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


恭喜米，鑫e贷总授信计算完成
恭喜米，鑫e贷放款计算完成
恭喜米，鑫e贷授信-B款计算完成


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)


NameError: name 'yesterday_team_report' is not defined

In [ ]:
yesterday_team_report='团队非按揭业绩日报20241218.xlsx'


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)


In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,团队名称,姓名,二手受理,NaN,NaN,NaN,NaN,鑫e贷放款,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,财富转介户数,NaN,NaN,月度目标\n达成率,月度目标\n达成率系数
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN


In [ ]:
    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名


In [ ]:

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

In [ ]:
index_list

['许闻多',
 '宋丽凤',
 '孙仰阳',
 '瞿逸程',
 '顾伟丽',
 '合计',
 '马越骋',
 '陈名',
 '万华',
 '茅敏艳',
 '钱潇伟',
 '合计',
 '杨小东',
 '王美燕',
 '徐君',
 '李亚',
 '杜星瑶',
 '曹倩云',
 '黄旭',
 '合计']

In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
daily_report

NameError: name 'daily_report' is not defined

In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_daily_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
yesterday_daily_report_df

,网点鑫e贷月度指标完成情况,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
1,NaN,NaN,指标,昨日完成数,报表数,协同外拓,B款月底额外计1户授信,数据调整数,完成数,完成率,...,协同外拓,完成数,完成率,指标,报表数,协同外拓,调整数,完成数,完成率,NaN
2,中心支行,陆家嘴支行,38,0,6,163,0,0,169,4.45,...,0,155,1.41,3,6,79,0,85,28.33,0.18
3,NaN,营业部,25,0,2,8,0,0,10,0.4,...,0,78,0.39,2,NaN,1,0,1,0.5,0.0645
4,NaN,曹路支行,20,0,14,6,0,0,20,1,...,0,118.44,0.74,2,2,1,0,3,1.5,0.147
5,NaN,川沙支行,22,5,109,0,0,0,109,4.95,...,0,158.8,1.76,2,NaN,0,0,0,0,0.12
6,NaN,合庆支行,22,1,50,14,0,0,64,2.91,...,20,143,1.19,2,15,7,0,22,11,0.1795
7,NaN,高桥支行,29,0,3,5,0,0,8,0.28,...,0,87,0.58,1,NaN,0,0,0,0,0.043
8,NaN,张江支行,38,0,6,15,0,0,21,0.55,...,0,128,0.64,2,1,6,0,7,3.5,0.1195
9,NaN,高行支行,20,1,6,0,0,0,6,0.3,...,0,83,1.38,1,2,0,0,2,2,0.135
10,NaN,金桥支行,27,0,10,0,0,0,10,0.37,...,0,84,0.42,2,NaN,0,0,0,0,0.0395


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:34,2:]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
    daily_report.index=index_list


ValueError: Length mismatch: Expected axis has 21 elements, new values have 20 elements

In [ ]:

    daily_report=yesterday_daily_report_df.iloc[1:35,2:]

    daily_report.index=index_list

ValueError: Length mismatch: Expected axis has 21 elements, new values have 20 elements

In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:35,2:]

    daily_report.index=index_list

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


ValueError: Length mismatch: Expected axis has 21 elements, new values have 20 elements

In [ ]:
index_list

['许闻多',
 '宋丽凤',
 '孙仰阳',
 '瞿逸程',
 '顾伟丽',
 '合计',
 '马越骋',
 '陈名',
 '万华',
 '茅敏艳',
 '钱潇伟',
 '合计',
 '杨小东',
 '王美燕',
 '徐君',
 '李亚',
 '杜星瑶',
 '曹倩云',
 '黄旭',
 '合计']

In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
new_columns

Unnamed: 2          指标
Unnamed: 3          昨日
Unnamed: 4          昨日
Unnamed: 5          本月
Unnamed: 6         完成率
Unnamed: 7          指标
Unnamed: 8          昨日
Unnamed: 9       报表完成数
Unnamed: 10    自然流量完成数
Unnamed: 11       线下贷款
Unnamed: 12        调整数
Unnamed: 13         本月
Unnamed: 14        完成率
Unnamed: 15         指标
Unnamed: 16         昨日
Unnamed: 17      报表完成数
Unnamed: 18      外拓双算数
Unnamed: 19       自然流量
Unnamed: 20        调整数
Unnamed: 21         本月
Unnamed: 22        完成率
Unnamed: 23         指标
Unnamed: 24         昨日
Unnamed: 25        报表数
Unnamed: 26        外拓数
Unnamed: 27        调整数
Unnamed: 28         本月
Unnamed: 29        完成率
Unnamed: 30         指标
Unnamed: 31         本月
Unnamed: 32        完成率
Unnamed: 33        NaN
Unnamed: 34        NaN
Name: 1, dtype: object

In [ ]:
new_columns

Unnamed: 2          指标
Unnamed: 3          昨日
Unnamed: 4          昨日
Unnamed: 5          本月
Unnamed: 6         完成率
Unnamed: 7          指标
Unnamed: 8          昨日
Unnamed: 9       报表完成数
Unnamed: 10    自然流量完成数
Unnamed: 11       线下贷款
Unnamed: 12        调整数
Unnamed: 13         本月
Unnamed: 14        完成率
Unnamed: 15         指标
Unnamed: 16         昨日
Unnamed: 17      报表完成数
Unnamed: 18      外拓双算数
Unnamed: 19       自然流量
Unnamed: 20        调整数
Unnamed: 21         本月
Unnamed: 22        完成率
Unnamed: 23         指标
Unnamed: 24         昨日
Unnamed: 25        报表数
Unnamed: 26        外拓数
Unnamed: 27        调整数
Unnamed: 28         本月
Unnamed: 29        完成率
Unnamed: 30         指标
Unnamed: 31         本月
Unnamed: 32        完成率
Unnamed: 33        NaN
Unnamed: 34        NaN
Name: 1, dtype: object

In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:34,2:]


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:33,2:]


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:32,2:]


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
daily_report=yesterday_daily_report_df.iloc[1:32,2:]

In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:30,2:]


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:32,2:]


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:30,2:]


In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
yesterday_daily_report_df.iloc[1:30,2:]

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,1,NaN,NaN,1,0.333333,2,0,0,0.856729,NaN


In [ ]:
yesterday_daily_report_df

,零贷专营团队月度指标完成日报1216,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,...,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率,NaN,NaN
2,按揭一队,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,...,3,NaN,NaN,3,1,2,2,1,0.676283,0.711917
3,NaN,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,...,0,NaN,NaN,0,0,2,1,0.5,0.86,NaN
4,NaN,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,...,1,NaN,NaN,1,0.2,3,0,0,0.818222,NaN
5,NaN,瞿逸程,2000,0,209,1108,0.554,150,0,0,...,0,NaN,NaN,0,0,2,0,0,0.3324,NaN
6,NaN,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,...,2,NaN,NaN,2,0.666667,2,0,0,0.354568,NaN
7,NaN,合计,10000,0,NaN,9597.5,0.95975,800,52,274,...,6,NaN,NaN,6,0.352941,11,3,0.272727,0.711917,NaN
8,按揭二队,马越骋,2000,0,32,2485,1.2425,150,0,0,...,1,NaN,NaN,1,0.333333,2,0,0,0.753333,0.767303
9,NaN,陈名,2000,0,186,2956,1.478,150,0,34.2425,...,2,NaN,NaN,2,0.666667,2,0,0,0.832323,NaN
10,NaN,万华,2000,0,NaN,1340,0.67,150,0,112.5334,...,3,NaN,NaN,3,1,2,2,1,0.752045,NaN


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[:,1:33]


In [ ]:
daily_report

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
1,NaN,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,自然流量完成数,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
2,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,...,3,0,3,NaN,NaN,3,1,2,2,1
3,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,...,3,0,0,NaN,NaN,0,0,2,1,0.5
4,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,...,5,0,1,NaN,NaN,1,0.2,3,0,0
5,瞿逸程,2000,0,209,1108,0.554,150,0,0,0,...,3,0,0,NaN,NaN,0,0,2,0,0
6,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
7,合计,10000,0,NaN,9597.5,0.95975,800,52,274,20,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
8,马越骋,2000,0,32,2485,1.2425,150,0,0,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
9,陈名,2000,0,186,2956,1.478,150,0,34.2425,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
10,万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,...,3,0,3,NaN,NaN,3,1,2,2,1


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:21, 1:33]


In [ ]:
daily_report

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
2,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,...,3,0,3,NaN,NaN,3,1,2,2,1
3,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,...,3,0,0,NaN,NaN,0,0,2,1,0.5
4,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,...,5,0,1,NaN,NaN,1,0.2,3,0,0
5,瞿逸程,2000,0,209,1108,0.554,150,0,0,0,...,3,0,0,NaN,NaN,0,0,2,0,0
6,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
7,合计,10000,0,NaN,9597.5,0.95975,800,52,274,20,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
8,马越骋,2000,0,32,2485,1.2425,150,0,0,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
9,陈名,2000,0,186,2956,1.478,150,0,34.2425,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
10,万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,...,3,0,3,NaN,NaN,3,1,2,2,1
11,茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,...,3,0,1,NaN,NaN,1,0.333333,2,0,0


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:]  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 1:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


ValueError: Length mismatch: Expected axis has 32 elements, new values have 33 elements

In [ ]:
daily_report

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
许闻多,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,瞿逸程,2000,0,209,1108,0.554,150,0,0,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
合计,合计,10000,0,NaN,9597.5,0.95975,800,52,274,20,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
马越骋,马越骋,2000,0,32,2485,1.2425,150,0,0,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,陈名,2000,0,186,2956,1.478,150,0,34.2425,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,...,3,0,1,NaN,NaN,1,0.333333,2,0,0


In [ ]:
new_columns

Unnamed: 2          指标
Unnamed: 3          昨日
Unnamed: 4          昨日
Unnamed: 5          本月
Unnamed: 6         完成率
Unnamed: 7          指标
Unnamed: 8          昨日
Unnamed: 9       报表完成数
Unnamed: 10    自然流量完成数
Unnamed: 11       线下贷款
Unnamed: 12        调整数
Unnamed: 13         本月
Unnamed: 14        完成率
Unnamed: 15         指标
Unnamed: 16         昨日
Unnamed: 17      报表完成数
Unnamed: 18      外拓双算数
Unnamed: 19       自然流量
Unnamed: 20        调整数
Unnamed: 21         本月
Unnamed: 22        完成率
Unnamed: 23         指标
Unnamed: 24         昨日
Unnamed: 25        报表数
Unnamed: 26        外拓数
Unnamed: 27        调整数
Unnamed: 28         本月
Unnamed: 29        完成率
Unnamed: 30         指标
Unnamed: 31         本月
Unnamed: 32        完成率
Unnamed: 33        NaN
Unnamed: 34        NaN
Name: 1, dtype: object

In [ ]:
new_columns.dropna()

Unnamed: 2          指标
Unnamed: 3          昨日
Unnamed: 4          昨日
Unnamed: 5          本月
Unnamed: 6         完成率
Unnamed: 7          指标
Unnamed: 8          昨日
Unnamed: 9       报表完成数
Unnamed: 10    自然流量完成数
Unnamed: 11       线下贷款
Unnamed: 12        调整数
Unnamed: 13         本月
Unnamed: 14        完成率
Unnamed: 15         指标
Unnamed: 16         昨日
Unnamed: 17      报表完成数
Unnamed: 18      外拓双算数
Unnamed: 19       自然流量
Unnamed: 20        调整数
Unnamed: 21         本月
Unnamed: 22        完成率
Unnamed: 23         指标
Unnamed: 24         昨日
Unnamed: 25        报表数
Unnamed: 26        外拓数
Unnamed: 27        调整数
Unnamed: 28         本月
Unnamed: 29        完成率
Unnamed: 30         指标
Unnamed: 31         本月
Unnamed: 32        完成率
Name: 1, dtype: object

In [ ]:
    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 1:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


ValueError: Length mismatch: Expected axis has 32 elements, new values have 31 elements

In [ ]:
new_columns

Unnamed: 2          指标
Unnamed: 3          昨日
Unnamed: 4          昨日
Unnamed: 5          本月
Unnamed: 6         完成率
Unnamed: 7          指标
Unnamed: 8          昨日
Unnamed: 9       报表完成数
Unnamed: 10    自然流量完成数
Unnamed: 11       线下贷款
Unnamed: 12        调整数
Unnamed: 13         本月
Unnamed: 14        完成率
Unnamed: 15         指标
Unnamed: 16         昨日
Unnamed: 17      报表完成数
Unnamed: 18      外拓双算数
Unnamed: 19       自然流量
Unnamed: 20        调整数
Unnamed: 21         本月
Unnamed: 22        完成率
Unnamed: 23         指标
Unnamed: 24         昨日
Unnamed: 25        报表数
Unnamed: 26        外拓数
Unnamed: 27        调整数
Unnamed: 28         本月
Unnamed: 29        完成率
Unnamed: 30         指标
Unnamed: 31         本月
Unnamed: 32        完成率
Name: 1, dtype: object

In [ ]:
daily_report

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
许闻多,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,瞿逸程,2000,0,209,1108,0.554,150,0,0,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
合计,合计,10000,0,NaN,9597.5,0.95975,800,52,274,20,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
马越骋,马越骋,2000,0,32,2485,1.2425,150,0,0,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,陈名,2000,0,186,2956,1.478,150,0,34.2425,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,...,3,0,1,NaN,NaN,1,0.333333,2,0,0


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:21, 1:33]


In [ ]:
daily_report

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
2,许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,...,3,0,3,NaN,NaN,3,1,2,2,1
3,宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,...,3,0,0,NaN,NaN,0,0,2,1,0.5
4,孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,...,5,0,1,NaN,NaN,1,0.2,3,0,0
5,瞿逸程,2000,0,209,1108,0.554,150,0,0,0,...,3,0,0,NaN,NaN,0,0,2,0,0
6,顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
7,合计,10000,0,NaN,9597.5,0.95975,800,52,274,20,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
8,马越骋,2000,0,32,2485,1.2425,150,0,0,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
9,陈名,2000,0,186,2956,1.478,150,0,34.2425,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
10,万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,...,3,0,3,NaN,NaN,3,1,2,2,1
11,茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,...,3,0,1,NaN,NaN,1,0.333333,2,0,0


In [ ]:
    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]


In [ ]:
daily_report

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
2,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
3,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,3,0,0,NaN,NaN,0,0,2,1,0.5
4,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,5,0,1,NaN,NaN,1,0.2,3,0,0
5,2000,0,209,1108,0.554,150,0,0,0,NaN,...,3,0,0,NaN,NaN,0,0,2,0,0
6,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
7,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
8,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
9,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
10,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
11,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
daily_report

1,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,自然流量完成数,线下贷款,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,2000,0,209,1108,0.554,150,0,0,0,NaN,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
合计,10000,0,NaN,9597.5,0.95975,800,52,274,20,NaN,...,17,0,6,NaN,NaN,6,0.352941,11,3,0.272727
马越骋,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0


In [ ]:
    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷总授信_指标','鑫e贷放款_指标','鑫e贷授信-B款_指标']

ValueError: Length mismatch: Expected axis has 5 elements, new values have 3 elements

In [ ]:
    kpi=daily_report[['指标']]


In [ ]:
kpi


1,指标,指标,指标,指标,指标
许闻多,2000,150,-,3,2
宋丽凤,2000,150,-,3,2
孙仰阳,2000,200,-,5,3
瞿逸程,2000,150,-,3,2
顾伟丽,2000,150,NaN,3,2
合计,10000,800,-,17,11
马越骋,2000,150,-,3,2
陈名,2000,150,-,3,2
万华,2000,150,-,3,2
茅敏艳,2000,150,-,3,2


In [ ]:
    daily_report=daily_report.drop('合计')


In [ ]:
daily_report

1,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,自然流量完成数,线下贷款,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,2000,0,209,1108,0.554,150,0,0,0,NaN,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,2000,0,NaN,500,0.25,150,21,61.7,8,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    kpi=daily_report[['指标']]

In [ ]:
kpi

1,指标,指标,指标,指标,指标
许闻多,2000,150,-,3,2
宋丽凤,2000,150,-,3,2
孙仰阳,2000,200,-,5,3
瞿逸程,2000,150,-,3,2
顾伟丽,2000,150,NaN,3,2
马越骋,2000,150,-,3,2
陈名,2000,150,-,3,2
万华,2000,150,-,3,2
茅敏艳,2000,150,-,3,2
钱潇伟,2000,150,-,3,2


In [ ]:
daily_report

1,指标,昨日,昨日,本月,完成率,指标,昨日,报表完成数,自然流量完成数,线下贷款,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,2000,0,42.5,1304.5,0.65225,150,10,63.7,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,2000,0,192.5,2902,1.451,150,0,61.1,6.4,NaN,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,2000,0,NaN,3699,1.8495,200,2,64.4222,13.8,NaN,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,2000,0,209,1108,0.554,150,0,0,0,NaN,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,2000,NaN,NaN,584,0.292,150,40,84.526,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,2000,0,32,2485,1.2425,150,0,0,0,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,2000,0,186,2956,1.478,150,0,34.2425,0,NaN,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,2000,0,NaN,1340,0.67,150,0,112.5334,0,NaN,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,2000,0,NaN,2478.5,1.23925,150,6,60.5465,17,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,2000,0,NaN,500,0.25,150,21,61.7,8,NaN,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    daily_report=daily_report.iloc[:,5:]


In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    daily_report=daily_report.iloc[:,5:]


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
kpi

1,指标,指标,指标,指标
许闻多,150,-,3,2
宋丽凤,150,-,3,2
孙仰阳,200,-,5,3
瞿逸程,150,-,3,2
顾伟丽,150,NaN,3,2
马越骋,150,-,3,2
陈名,150,-,3,2
万华,150,-,3,2
茅敏艳,150,-,3,2
钱潇伟,150,-,3,2


In [ ]:
    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']

In [ ]:
    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']


In [ ]:
kpi

,鑫e贷放款_指标,鑫e贷总授信（A/B款）_指标,鑫e贷授信-B款_指标,财富转介户数_指标
许闻多,150,-,3,2
宋丽凤,150,-,3,2
孙仰阳,200,-,5,3
瞿逸程,150,-,3,2
顾伟丽,150,NaN,3,2
马越骋,150,-,3,2
陈名,150,-,3,2
万华,150,-,3,2
茅敏艳,150,-,3,2
钱潇伟,150,-,3,2


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    yesterday_finished=daily_report[['本月']]


In [ ]:
yesterday_finished

1,本月,本月,本月,本月
许闻多,63.7,4,3,2
宋丽凤,67.5,0,0,1
孙仰阳,78.2222,1,1,0
瞿逸程,0,0,0,0
顾伟丽,84.526,2,2,0
马越骋,0,1,1,0
陈名,34.2425,2,2,0
万华,112.5334,3,3,2
茅敏艳,77.5465,2,1,0
钱潇伟,69.7,1,1,3


In [ ]:
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_本月','鑫e贷总授信（A/B款）_本月','鑫e贷授信-B款_本月','财富转介户数_本月']


In [ ]:
yesterday_finished

,鑫e贷放款_本月,鑫e贷总授信（A/B款）_本月,鑫e贷授信-B款_本月,财富转介户数_本月
许闻多,63.7,4,3,2
宋丽凤,67.5,0,0,1
孙仰阳,78.2222,1,1,0
瞿逸程,0,0,0,0
顾伟丽,84.526,2,2,0
马越骋,0,1,1,0
陈名,34.2425,2,2,0
万华,112.5334,3,3,2
茅敏艳,77.5465,2,1,0
钱潇伟,69.7,1,1,3


In [ ]:
    yesterday_finished_result=yesterday_finished[['	鑫e贷放款_本月']]


KeyError: "None of [Index(['\\t鑫e贷放款_本月'], dtype='object')] are in the [columns]"

In [ ]:
    yesterday_finished_result=yesterday_finished[['鑫e贷放款_本月']]


In [ ]:
yesterday_finished_result

,鑫e贷放款_本月
许闻多,63.7
宋丽凤,67.5
孙仰阳,78.2222
瞿逸程,0
顾伟丽,84.526
马越骋,0
陈名,34.2425
万华,112.5334
茅敏艳,77.5465
钱潇伟,69.7


In [ ]:
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_本月
许闻多,150,63.7
宋丽凤,150,67.5
孙仰阳,200,78.2222
瞿逸程,150,0
顾伟丽,150,84.526
马越骋,150,0
陈名,150,34.2425
万华,150,112.5334
茅敏艳,150,77.5465
钱潇伟,150,69.7


In [ ]:
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]

In [ ]:
yesterday_finished_result

,鑫e贷放款_昨日本月
许闻多,63.7
宋丽凤,67.5
孙仰阳,78.2222
瞿逸程,0
顾伟丽,84.526
马越骋,0
陈名,34.2425
万华,112.5334
茅敏艳,77.5465
钱潇伟,69.7


In [ ]:
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月
许闻多,150,63.7
宋丽凤,150,67.5
孙仰阳,200,78.2222
瞿逸程,150,0
顾伟丽,150,84.526
马越骋,150,0
陈名,150,34.2425
万华,150,112.5334
茅敏艳,150,77.5465
钱潇伟,150,69.7


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#数据路径设置

result_path=r'C:\Users\Wesle\Desktop\srcb_daily_report\result'

data_path=r'C:\Users\Wesle\Desktop\srcb_daily_report\原始数据'

#部门归属表文件名
department_list='员工部门归属表.xlsx'

#昨日日报表文件名
yesterday_daily_report='网点鑫e贷月度指标完成情况1226.xlsx'

yesterday_team_report='团队非按揭业绩日报20241218.xlsx'

#【浦东分行鑫e贷】客户经理营销数据文件名
client_manager_data='【浦东分行鑫e贷】客户经理营销数据_2024-12-18.xlsx'

retail_performance_data='零售市场部协同外拓及理财转介业绩报送-9.xlsx'

type_B_data='【浦东分行鑫e贷】鑫e贷b款明细_2024-12-18.xlsx'

T0_Date='2024-12-26'


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)


In [ ]:

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]

In [ ]:
monthly_fangkuan

,benyuefangkuanjine
kehujinglixingm,
顾佳怡,0.0
张毓琦,0.0
蔡利华,0.0
许闻多,100000.0
许闻多,0.0
...,...
李润杰,6648.0
严丹,30000.0
朱伟杰,0.0


In [ ]:
manager_total_fangkuan

NameError: name 'manager_total_fangkuan' is not defined

In [ ]:
    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()


In [ ]:
manager_total_fangkuan

,benyuefangkuanjine
kehujinglixingm,
丁丹萍,0.0
丁晓雄,0.0
万云峰,0.0
万佳来,0.0
万华,1141064.0
...,...
龚昊,0.0
龚欣怡,0.0
龚洁,107000.0


In [ ]:
manager_total_fangkuan

,benyuefangkuanjine
kehujinglixingm,
丁丹萍,0.0
丁晓雄,0.0
万云峰,0.0
万佳来,0.0
万华,1141064.0
...,...
龚昊,0.0
龚欣怡,0.0
龚洁,107000.0


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月
许闻多,150,63.7
宋丽凤,150,67.5
孙仰阳,200,78.2222
瞿逸程,150,0
顾伟丽,150,84.526
马越骋,150,0
陈名,150,34.2425
万华,150,112.5334
茅敏艳,150,77.5465
钱潇伟,150,69.7


In [ ]:
    result.loc[:,"鑫e贷放款_报表完成数"]=manager_total_fangkuan


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数
许闻多,150,63.7,637000.0
宋丽凤,150,67.5,811000.0
孙仰阳,200,78.2222,764222.0
瞿逸程,150,0,0.0
顾伟丽,150,84.526,845260.0
马越骋,150,0,0.0
陈名,150,34.2425,440425.0
万华,150,112.5334,1141064.0
茅敏艳,150,77.5465,614351.0
钱潇伟,150,69.7,617000.0


In [ ]:
    result.loc[:,"鑫e贷放款_报表完成数"]= result.loc[:,"鑫e贷放款_报表完成数"]/10000


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数
许闻多,150,63.7,63.7000
宋丽凤,150,67.5,81.1000
孙仰阳,200,78.2222,76.4222
瞿逸程,150,0,0.0000
顾伟丽,150,84.526,84.5260
马越骋,150,0,0.0000
陈名,150,34.2425,44.0425
万华,150,112.5334,114.1064
茅敏艳,150,77.5465,61.4351
钱潇伟,150,69.7,61.7000


In [ ]:
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")


In [ ]:
type_B_df

,fangkuanjine,fangkuanriq,1,1.1,1.2,1.3,1.4,1.5,1.6,yingxiaokehujingl,...,1.8,1.9,1.10,1.11,jingdiaokehujingl,1.12,1.13,1.14,1.15,1.16
0,220000,2023-06-19,4.896,4.89600,4.89600,623162*********6077,本行卡,0.0,-,杨小东,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
1,10000,2023-06-19,4.896,4.89600,4.89600,623162*********6077,本行卡,0.0,-,杨小东,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
2,250000,2023-06-28,4.896,4.89600,4.89600,623162*********6077,本行卡,0.0,-,杨小东,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
3,200000,2024-07-14,11.880,4.39560,11.88000,621688*********8202,本行卡,0.0,-,寿春连,...,31297.0,浦东分行本部,31328,北蔡支行,寿春连,77621,31297,浦东分行本部,31328,北蔡支行
4,50000,2024-09-13,4.806,4.80600,4.80600,623162*********5681,本行卡,0.0,-,吴亮宇,...,10002.0,业务处理中心,-,-,曹倩云,83960,31297,浦东分行本部,31298,浦东分行财务会计部
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5803,200000,2024-10-15,7.506,4.95396,4.95396,623162*********4570,本行卡,0.0,-,杨小东,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
5804,100000,2024-10-18,7.506,4.95396,4.95396,623162*********4570,本行卡,0.0,-,杨小东,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
5805,198000,2024-10-22,7.506,4.95396,4.95396,623162*********4570,本行卡,0.0,-,杨小东,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
5806,200000,2023-12-12,5.508,4.46148,4.46148,623162*********4723,本行卡,0.0,-,郭勤,...,31297.0,浦东分行本部,31311,川沙支行,郭勤,28233,31297,浦东分行本部,31311,川沙支行


In [ ]:
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)


In [ ]:
netural_result

,fangkuanjine
jingdiaokehujingl,
孙仰阳,138000
宋丽凤,100000
张子豪,50000
张馨怡,179698
徐君,1327916
曹倩云,3853271
李亚,5630260
杜星瑶,3282000
杨小东,3646000


In [ ]:
netural_result

,fangkuanjine
jingdiaokehujingl,
孙仰阳,138000
宋丽凤,100000
张子豪,50000
张馨怡,179698
徐君,1327916
曹倩云,3853271
李亚,5630260
杜星瑶,3282000
杨小东,3646000


In [ ]:
    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,100000.0
孙仰阳,200,78.2222,76.4222,138000.0
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,169000.0
钱潇伟,150,69.7,61.7000,80000.0


In [ ]:
    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result/10000


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,NaN
孙仰阳,200,78.2222,76.4222,NaN
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,NaN
钱潇伟,150,69.7,61.7000,NaN


In [ ]:
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,NaN
孙仰阳,200,78.2222,76.4222,NaN
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,NaN
钱潇伟,150,69.7,61.7000,NaN


In [ ]:
netural_result

,fangkuanjine
jingdiaokehujingl,
孙仰阳,138000
宋丽凤,100000
张子豪,50000
张馨怡,179698
徐君,1327916
曹倩云,3853271
李亚,5630260
杜星瑶,3282000
杨小东,3646000


In [ ]:
    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,NaN
孙仰阳,200,78.2222,76.4222,NaN
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,NaN
钱潇伟,150,69.7,61.7000,NaN


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果（报表完成数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()

    result.loc[:,"鑫e贷放款_报表完成数"]=manager_total_fangkuan

    result.loc[:,"鑫e贷放款_报表完成数"]= result.loc[:,"鑫e贷放款_报表完成数"]/10000

    #第四个结果（自然流量完成数）

    #读取b款数据

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,100000.0
孙仰阳,200,78.2222,76.4222,138000.0
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,169000.0
钱潇伟,150,69.7,61.7000,80000.0


In [ ]:
    result.loc[:,"鑫e贷放款_自然流量完成数"]=result.loc[:,"鑫e贷放款_自然流量完成数"]/10000


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,10.0000
孙仰阳,200,78.2222,76.4222,13.8000
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,16.9000
钱潇伟,150,69.7,61.7000,8.0000


In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    offline_loan=daily_report[['线下贷款']]


In [ ]:
offline_loan

1,线下贷款
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    offline_loan=daily_report[['线下贷款']]

    offline_loan.columns=['鑫e贷放款_线下贷款']

In [ ]:
offline_loan

,鑫e贷放款_线下贷款
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数
许闻多,150,63.7,63.7000,NaN
宋丽凤,150,67.5,81.1000,10.0000
孙仰阳,200,78.2222,76.4222,13.8000
瞿逸程,150,0,0.0000,NaN
顾伟丽,150,84.526,84.5260,NaN
马越骋,150,0,0.0000,NaN
陈名,150,34.2425,44.0425,NaN
万华,150,112.5334,114.1064,NaN
茅敏艳,150,77.5465,61.4351,16.9000
钱潇伟,150,69.7,61.7000,8.0000


In [ ]:
    offline_loan=daily_report[['线下贷款']]

    offline_loan.columns=['鑫e贷放款_线下贷款']

    result.loc[:,"鑫e贷放款_线下贷款"]=offline_loan

In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款
许闻多,150,63.7,63.7000,NaN,NaN
宋丽凤,150,67.5,81.1000,10.0000,NaN
孙仰阳,200,78.2222,76.4222,13.8000,NaN
瞿逸程,150,0,0.0000,NaN,NaN
顾伟丽,150,84.526,84.5260,NaN,NaN
马越骋,150,0,0.0000,NaN,NaN
陈名,150,34.2425,44.0425,NaN,NaN
万华,150,112.5334,114.1064,NaN,NaN
茅敏艳,150,77.5465,61.4351,16.9000,NaN
钱潇伟,150,69.7,61.7000,8.0000,NaN


In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    adjust_number=daily_report[['调整数']]


In [ ]:
adjust_number

1,调整数,调整数,调整数
许闻多,NaN,NaN,NaN
宋丽凤,NaN,NaN,NaN
孙仰阳,NaN,NaN,NaN
瞿逸程,NaN,NaN,NaN
顾伟丽,NaN,NaN,NaN
马越骋,NaN,NaN,NaN
陈名,NaN,NaN,NaN
万华,NaN,NaN,NaN
茅敏艳,NaN,NaN,NaN
钱潇伟,NaN,NaN,NaN


In [ ]:
    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']


In [ ]:
adjust_number

,鑫e贷放款_调整数,鑫e贷总授信（A/B款）_调整数,鑫e贷授信-B款_调整数
许闻多,NaN,NaN,NaN
宋丽凤,NaN,NaN,NaN
孙仰阳,NaN,NaN,NaN
瞿逸程,NaN,NaN,NaN
顾伟丽,NaN,NaN,NaN
马越骋,NaN,NaN,NaN
陈名,NaN,NaN,NaN
万华,NaN,NaN,NaN
茅敏艳,NaN,NaN,NaN
钱潇伟,NaN,NaN,NaN


In [ ]:
    adjust_number_result=adjust_number[['鑫e贷放款_调整数']]


In [ ]:
adjust_number_result

,鑫e贷放款_调整数
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    result.loc[:,"鑫e贷放款_调整数"]=adjust_number_result


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数
许闻多,150,63.7,63.7000,NaN,NaN,NaN
宋丽凤,150,67.5,81.1000,10.0000,NaN,NaN
孙仰阳,200,78.2222,76.4222,13.8000,NaN,NaN
瞿逸程,150,0,0.0000,NaN,NaN,NaN
顾伟丽,150,84.526,84.5260,NaN,NaN,NaN
马越骋,150,0,0.0000,NaN,NaN,NaN
陈名,150,34.2425,44.0425,NaN,NaN,NaN
万华,150,112.5334,114.1064,NaN,NaN,NaN
茅敏艳,150,77.5465,61.4351,16.9000,NaN,NaN
钱潇伟,150,69.7,61.7000,8.0000,NaN,NaN


In [ ]:
    result.loc[:,"本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum()


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,本月
许闻多,150,63.7,63.7000,NaN,NaN,NaN,NaN
宋丽凤,150,67.5,81.1000,10.0000,NaN,NaN,NaN
孙仰阳,200,78.2222,76.4222,13.8000,NaN,NaN,NaN
瞿逸程,150,0,0.0000,NaN,NaN,NaN,NaN
顾伟丽,150,84.526,84.5260,NaN,NaN,NaN,NaN
马越骋,150,0,0.0000,NaN,NaN,NaN,NaN
陈名,150,34.2425,44.0425,NaN,NaN,NaN,NaN
万华,150,112.5334,114.1064,NaN,NaN,NaN,NaN
茅敏艳,150,77.5465,61.4351,16.9000,NaN,NaN,NaN
钱潇伟,150,69.7,61.7000,8.0000,NaN,NaN,NaN


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果（报表完成数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()

    result.loc[:,"鑫e贷放款_报表完成数"]=manager_total_fangkuan

    result.loc[:,"鑫e贷放款_报表完成数"]= result.loc[:,"鑫e贷放款_报表完成数"]/10000

    #第四个结果（自然流量完成数）

    #读取b款数据

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

    result.loc[:,"鑫e贷放款_自然流量完成数"]=result.loc[:,"鑫e贷放款_自然流量完成数"]/10000



    #第四个指标（线下贷款）

    offline_loan=daily_report[['线下贷款']]

    offline_loan.columns=['鑫e贷放款_线下贷款']

    result.loc[:,"鑫e贷放款_线下贷款"]=offline_loan


    #第五个指标(调整数)

    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷放款_调整数']]

    result.loc[:,"鑫e贷放款_调整数"]=adjust_number_result

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数
许闻多,150,63.7,63.7000,NaN,NaN,NaN
宋丽凤,150,67.5,81.1000,10.0000,NaN,NaN
孙仰阳,200,78.2222,76.4222,13.8000,NaN,NaN
瞿逸程,150,0,0.0000,NaN,NaN,NaN
顾伟丽,150,84.526,84.5260,NaN,NaN,NaN
马越骋,150,0,0.0000,NaN,NaN,NaN
陈名,150,34.2425,44.0425,NaN,NaN,NaN
万华,150,112.5334,114.1064,NaN,NaN,NaN
茅敏艳,150,77.5465,61.4351,16.9000,NaN,NaN
钱潇伟,150,69.7,61.7000,8.0000,NaN,NaN


In [ ]:
    result=result.fillna(0)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数
许闻多,150,63.7000,63.7000,0.0000,0,0
宋丽凤,150,67.5000,81.1000,10.0000,0,0
孙仰阳,200,78.2222,76.4222,13.8000,0,0
瞿逸程,150,0.0000,0.0000,0.0000,0,0
顾伟丽,150,84.5260,84.5260,0.0000,0,0
马越骋,150,0.0000,0.0000,0.0000,0,0
陈名,150,34.2425,44.0425,0.0000,0,0
万华,150,112.5334,114.1064,0.0000,0,0
茅敏艳,150,77.5465,61.4351,16.9000,0,0
钱潇伟,150,69.7000,61.7000,8.0000,0,0


In [ ]:
    result.loc[:,"本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum()


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,本月
许闻多,150,63.7000,63.7000,0.0000,0,0,NaN
宋丽凤,150,67.5000,81.1000,10.0000,0,0,NaN
孙仰阳,200,78.2222,76.4222,13.8000,0,0,NaN
瞿逸程,150,0.0000,0.0000,0.0000,0,0,NaN
顾伟丽,150,84.5260,84.5260,0.0000,0,0,NaN
马越骋,150,0.0000,0.0000,0.0000,0,0,NaN
陈名,150,34.2425,44.0425,0.0000,0,0,NaN
万华,150,112.5334,114.1064,0.0000,0,0,NaN
茅敏艳,150,77.5465,61.4351,16.9000,0,0,NaN
钱潇伟,150,69.7000,61.7000,8.0000,0,0,NaN


In [ ]:
result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"]

,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数
许闻多,63.7000,0.0000,0,0
宋丽凤,81.1000,10.0000,0,0
孙仰阳,76.4222,13.8000,0,0
瞿逸程,0.0000,0.0000,0,0
顾伟丽,84.5260,0.0000,0,0
马越骋,0.0000,0.0000,0,0
陈名,44.0425,0.0000,0,0
万华,114.1064,0.0000,0,0
茅敏艳,61.4351,16.9000,0,0
钱潇伟,61.7000,8.0000,0,0


In [ ]:
    result.loc[:,"本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum(axis=0)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,本月
许闻多,150,63.7000,63.7000,0.0000,0,0,NaN
宋丽凤,150,67.5000,81.1000,10.0000,0,0,NaN
孙仰阳,200,78.2222,76.4222,13.8000,0,0,NaN
瞿逸程,150,0.0000,0.0000,0.0000,0,0,NaN
顾伟丽,150,84.5260,84.5260,0.0000,0,0,NaN
马越骋,150,0.0000,0.0000,0.0000,0,0,NaN
陈名,150,34.2425,44.0425,0.0000,0,0,NaN
万华,150,112.5334,114.1064,0.0000,0,0,NaN
茅敏艳,150,77.5465,61.4351,16.9000,0,0,NaN
钱潇伟,150,69.7000,61.7000,8.0000,0,0,NaN


In [ ]:
result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"]

,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数
许闻多,63.7000,0.0000,0,0
宋丽凤,81.1000,10.0000,0,0
孙仰阳,76.4222,13.8000,0,0
瞿逸程,0.0000,0.0000,0,0
顾伟丽,84.5260,0.0000,0,0
马越骋,0.0000,0.0000,0,0
陈名,44.0425,0.0000,0,0
万华,114.1064,0.0000,0,0
茅敏艳,61.4351,16.9000,0,0
钱潇伟,61.7000,8.0000,0,0


In [ ]:
    result.loc[:,"本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum(axis=1)


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,本月
许闻多,150,63.7000,63.7000,0.0000,0,0,63.7000
宋丽凤,150,67.5000,81.1000,10.0000,0,0,91.1000
孙仰阳,200,78.2222,76.4222,13.8000,0,0,90.2222
瞿逸程,150,0.0000,0.0000,0.0000,0,0,0.0000
顾伟丽,150,84.5260,84.5260,0.0000,0,0,84.5260
马越骋,150,0.0000,0.0000,0.0000,0,0,0.0000
陈名,150,34.2425,44.0425,0.0000,0,0,44.0425
万华,150,112.5334,114.1064,0.0000,0,0,114.1064
茅敏艳,150,77.5465,61.4351,16.9000,0,0,78.3351
钱潇伟,150,69.7000,61.7000,8.0000,0,0,69.7000


In [ ]:
    
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果（报表完成数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()

    result.loc[:,"鑫e贷放款_报表完成数"]=manager_total_fangkuan

    result.loc[:,"鑫e贷放款_报表完成数"]= result.loc[:,"鑫e贷放款_报表完成数"]/10000

    #第四个结果（自然流量完成数）

    #读取b款数据

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

    result.loc[:,"鑫e贷放款_自然流量完成数"]=result.loc[:,"鑫e贷放款_自然流量完成数"]/10000



    #第四个指标（线下贷款）

    offline_loan=daily_report[['线下贷款']]

    offline_loan.columns=['鑫e贷放款_线下贷款']

    result.loc[:,"鑫e贷放款_线下贷款"]=offline_loan


    #第五个指标(调整数)

    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷放款_调整数']]

    result.loc[:,"鑫e贷放款_调整数"]=adjust_number_result

    #计算本月和昨日轧差

    result=result.fillna(0)
    result.loc[:,"鑫e贷放款_本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum(axis=1)
    

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
    result.loc[:,"鑫e贷放款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷放款_本月"]-result.loc[:,"鑫e贷放款_昨日本月"]


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,鑫e贷放款_本月,鑫e贷放款_昨日完成数(轧差)
许闻多,150,63.7000,63.7000,0.0000,0,0,63.7000,0.0000
宋丽凤,150,67.5000,81.1000,10.0000,0,0,91.1000,23.6000
孙仰阳,200,78.2222,76.4222,13.8000,0,0,90.2222,12.0000
瞿逸程,150,0.0000,0.0000,0.0000,0,0,0.0000,0.0000
顾伟丽,150,84.5260,84.5260,0.0000,0,0,84.5260,0.0000
马越骋,150,0.0000,0.0000,0.0000,0,0,0.0000,0.0000
陈名,150,34.2425,44.0425,0.0000,0,0,44.0425,9.8000
万华,150,112.5334,114.1064,0.0000,0,0,114.1064,1.5730
茅敏艳,150,77.5465,61.4351,16.9000,0,0,78.3351,0.7886
钱潇伟,150,69.7000,61.7000,8.0000,0,0,69.7000,0.0000


In [ ]:
    result.loc[:,"鑫e贷放款_完成率"]=result.loc[:,"鑫e贷放款_本月"]/result.loc[:,"鑫e贷放款_指标"]


In [ ]:
result

,鑫e贷放款_指标,鑫e贷放款_昨日本月,鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,鑫e贷放款_本月,鑫e贷放款_昨日完成数(轧差),鑫e贷放款_完成率
许闻多,150,63.7000,63.7000,0.0000,0,0,63.7000,0.0000,0.424667
宋丽凤,150,67.5000,81.1000,10.0000,0,0,91.1000,23.6000,0.607333
孙仰阳,200,78.2222,76.4222,13.8000,0,0,90.2222,12.0000,0.451111
瞿逸程,150,0.0000,0.0000,0.0000,0,0,0.0000,0.0000,0.000000
顾伟丽,150,84.5260,84.5260,0.0000,0,0,84.5260,0.0000,0.563507
马越骋,150,0.0000,0.0000,0.0000,0,0,0.0000,0.0000,0.000000
陈名,150,34.2425,44.0425,0.0000,0,0,44.0425,9.8000,0.293617
万华,150,112.5334,114.1064,0.0000,0,0,114.1064,1.5730,0.760709
茅敏艳,150,77.5465,61.4351,16.9000,0,0,78.3351,0.7886,0.522234
钱潇伟,150,69.7000,61.7000,8.0000,0,0,69.7000,0.0000,0.464667


In [ ]:
    final_result=result[['鑫e贷放款_指标','鑫e贷放款_昨日完成数(轧差)',
                        '鑫e贷放款_报表数完成数','鑫e贷放款_自然流量完成数','线下贷款','鑫e贷放款_调整数',
                            '鑫e贷放款_本月','鑫e贷放款_完成率']]

KeyError: "['鑫e贷放款_报表数完成数', '线下贷款'] not in index"

In [ ]:
    final_result=result[['鑫e贷放款_指标','鑫e贷放款_昨日完成数(轧差)',
                        '鑫e贷放款_报表完成数','鑫e贷放款_自然流量完成数','鑫e贷放款_线下贷款','鑫e贷放款_调整数',
                            '鑫e贷放款_本月','鑫e贷放款_完成率']]

In [ ]:
final_result

,鑫e贷放款_指标,鑫e贷放款_昨日完成数(轧差),鑫e贷放款_报表完成数,鑫e贷放款_自然流量完成数,鑫e贷放款_线下贷款,鑫e贷放款_调整数,鑫e贷放款_本月,鑫e贷放款_完成率
许闻多,150,0.0000,63.7000,0.0000,0,0,63.7000,0.424667
宋丽凤,150,23.6000,81.1000,10.0000,0,0,91.1000,0.607333
孙仰阳,200,12.0000,76.4222,13.8000,0,0,90.2222,0.451111
瞿逸程,150,0.0000,0.0000,0.0000,0,0,0.0000,0.000000
顾伟丽,150,0.0000,84.5260,0.0000,0,0,84.5260,0.563507
马越骋,150,0.0000,0.0000,0.0000,0,0,0.0000,0.000000
陈名,150,9.8000,44.0425,0.0000,0,0,44.0425,0.293617
万华,150,1.5730,114.1064,0.0000,0,0,114.1064,0.760709
茅敏艳,150,0.7886,61.4351,16.9000,0,0,78.3351,0.522234
钱潇伟,150,0.0000,61.7000,8.0000,0,0,69.7000,0.464667


In [ ]:

def XY_Dai_Fang_Kuang_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):
    
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果（报表完成数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()

    result.loc[:,"鑫e贷放款_报表完成数"]=manager_total_fangkuan

    result.loc[:,"鑫e贷放款_报表完成数"]= result.loc[:,"鑫e贷放款_报表完成数"]/10000

    #第四个结果（自然流量完成数）

    #读取b款数据

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

    result.loc[:,"鑫e贷放款_自然流量完成数"]=result.loc[:,"鑫e贷放款_自然流量完成数"]/10000



    #第四个指标（线下贷款）

    offline_loan=daily_report[['线下贷款']]

    offline_loan.columns=['鑫e贷放款_线下贷款']

    result.loc[:,"鑫e贷放款_线下贷款"]=offline_loan


    #第五个指标(调整数)

    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷放款_调整数']]

    result.loc[:,"鑫e贷放款_调整数"]=adjust_number_result

    #计算本月和昨日轧差

    result=result.fillna(0)
    result.loc[:,"鑫e贷放款_本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum(axis=1)
    
    result.loc[:,"鑫e贷放款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷放款_本月"]-result.loc[:,"鑫e贷放款_昨日本月"]

    #完成率

    result.loc[:,"鑫e贷放款_完成率"]=result.loc[:,"鑫e贷放款_本月"]/result.loc[:,"鑫e贷放款_指标"]
    #
    final_result=result[['鑫e贷放款_指标','鑫e贷放款_昨日完成数(轧差)',
                        '鑫e贷放款_报表完成数','鑫e贷放款_自然流量完成数','鑫e贷放款_线下贷款','鑫e贷放款_调整数',
                            '鑫e贷放款_本月','鑫e贷放款_完成率']]

    final_result.to_excel(result_path+'\\'+'团队_鑫e贷放款完成情况.xlsx')
    
    print('恭喜米，团队鑫e贷放款计算完成')

    return final_result


XY_Dai_Fang_Kuang_Team_result=XY_Dai_Fang_Kuang_Team(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


ValueError: Length mismatch: Expected axis has 20 elements, new values have 44 elements

In [ ]:

def XY_Dai_Fang_Kuang_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):
    
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷放款_指标']]

    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷放款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    #第三个结果（报表完成数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_fangkuan=client_manager_df[['benyuefangkuanjine']]
    
    manager_total_fangkuan = monthly_fangkuan.groupby(monthly_fangkuan.index).sum()

    result.loc[:,"鑫e贷放款_报表完成数"]=manager_total_fangkuan

    result.loc[:,"鑫e贷放款_报表完成数"]= result.loc[:,"鑫e贷放款_报表完成数"]/10000

    #第四个结果（自然流量完成数）

    #读取b款数据

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户借据数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

    result.loc[:,"鑫e贷放款_自然流量完成数"]=result.loc[:,"鑫e贷放款_自然流量完成数"]/10000



    #第四个指标（线下贷款）

    offline_loan=daily_report[['线下贷款']]

    offline_loan.columns=['鑫e贷放款_线下贷款']

    result.loc[:,"鑫e贷放款_线下贷款"]=offline_loan


    #第五个指标(调整数)

    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷放款_调整数']]

    result.loc[:,"鑫e贷放款_调整数"]=adjust_number_result

    #计算本月和昨日轧差

    result=result.fillna(0)
    result.loc[:,"鑫e贷放款_本月"]=result.loc[:,"鑫e贷放款_报表完成数":"鑫e贷放款_调整数"].sum(axis=1)
    
    result.loc[:,"鑫e贷放款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷放款_本月"]-result.loc[:,"鑫e贷放款_昨日本月"]

    #完成率

    result.loc[:,"鑫e贷放款_完成率"]=result.loc[:,"鑫e贷放款_本月"]/result.loc[:,"鑫e贷放款_指标"]
    #
    final_result=result[['鑫e贷放款_指标','鑫e贷放款_昨日完成数(轧差)',
                        '鑫e贷放款_报表完成数','鑫e贷放款_自然流量完成数','鑫e贷放款_线下贷款','鑫e贷放款_调整数',
                            '鑫e贷放款_本月','鑫e贷放款_完成率']]

    final_result.to_excel(result_path+'\\'+'团队_鑫e贷放款完成情况.xlsx')
    
    print('恭喜米，团队鑫e贷放款计算完成')

    return final_result


XY_Dai_Fang_Kuang_Team_result=XY_Dai_Fang_Kuang_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)


恭喜米，团队鑫e贷放款计算完成


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

In [ ]:
kpi

1,指标,指标,指标,指标
许闻多,150,-,3,2
宋丽凤,150,-,3,2
孙仰阳,200,-,5,3
瞿逸程,150,-,3,2
顾伟丽,150,NaN,3,2
马越骋,150,-,3,2
陈名,150,-,3,2
万华,150,-,3,2
茅敏艳,150,-,3,2
钱潇伟,150,-,3,2


In [ ]:
    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']


In [ ]:
    kpi_result=kpi[['鑫e贷总授信（A/B款）_指标']]


In [ ]:
kpi_result


,鑫e贷总授信（A/B款）_指标
许闻多,-
宋丽凤,-
孙仰阳,-
瞿逸程,-
顾伟丽,NaN
马越骋,-
陈名,-
万华,-
茅敏艳,-
钱潇伟,-


In [ ]:
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷总授信（A/B款）_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)



In [ ]:
result

,鑫e贷总授信（A/B款）_指标,鑫e贷总授信（A/B款）_昨日本月
许闻多,-,4
宋丽凤,-,0
孙仰阳,-,1
瞿逸程,-,0
顾伟丽,NaN,2
马越骋,-,1
陈名,-,2
万华,-,3
茅敏艳,-,2
钱潇伟,-,1


In [ ]:

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

In [ ]:
    monthly_shouxin=client_manager_df[['benyueshouxinrenshu']]

    manager_total_shouxin = monthly_shouxin.groupby(monthly_shouxin.index).sum()


In [ ]:
manager_total_shouxin

,benyueshouxinrenshu
kehujinglixingm,
丁丹萍,0
丁晓雄,2
万云峰,0
万佳来,0
万华,5
...,...
龚昊,1
龚欣怡,0
龚洁,0


In [ ]:
    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_shouxin=client_manager_df[['benyueshouxinrenshu']]

    manager_total_shouxin = monthly_shouxin.groupby(monthly_shouxin.index).sum()

    result.loc[:,"鑫e贷总授信（A/B款）_报表数"]=manager_total_shouxin

In [ ]:
result

,鑫e贷总授信（A/B款）_指标,鑫e贷总授信（A/B款）_昨日本月,鑫e贷总授信（A/B款）_报表数
许闻多,-,4,4
宋丽凤,-,0,1
孙仰阳,-,1,1
瞿逸程,-,0,0
顾伟丽,NaN,2,2
马越骋,-,1,1
陈名,-,2,2
万华,-,3,5
茅敏艳,-,2,2
钱潇伟,-,1,1


In [ ]:
yesterday_retail_performance

NameError: name 'yesterday_retail_performance' is not defined

In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:
    yesterday_retail_performance=daily_report[['外拓双算数']]


In [ ]:
yesterday_retail_performance

1,外拓双算数
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    yesterday_retail_performance.columns=["鑫e贷总授信（A/B款）_昨日外拓双算数"]


In [ ]:
yesterday_retail_performance

,鑫e贷总授信（A/B款）_昨日外拓双算数
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)


In [ ]:
retail_performance_df

,外拓日期,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
0,45632.0,曹倩云,花木支行,2.0,6.0,1.0,4.0,1.0,2.0,3.0,9
1,45635.0,王美燕,曹路支行,5.0,1.0,4.0,1.0,1.0,0.0,1.0,6
2,45628.0,杨小东,陆家嘴支行,NaN,10.0,NaN,10.0,0.0,0.0,0.0,20
3,45629.0,杨小东,陆家嘴支行,NaN,18.0,NaN,18.0,0.0,0.0,0.0,36
4,45630.0,王美燕,陆家嘴支行,1.0,8.0,1.0,8.0,0.0,0.0,0.0,17
5,45630.0,徐君,营业部,6.0,1.0,6.0,1.0,0.0,0.0,0.0,8
6,45631.0,杨小东,陆家嘴支行,NaN,2.0,NaN,2.0,0.0,0.0,0.0,4
7,45632.0,杜星瑶,凌兆分理处,8.0,3.0,4.0,1.0,4.0,2.0,6.0,6
8,45635.0,徐君,高东支行,7.0,NaN,7.0,NaN,0.0,0.0,0.0,7
9,45636.0,杨小东,高桥支行,5.0,NaN,5.0,NaN,0.0,0.0,0.0,5


In [ ]:
    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)


In [ ]:
retail_performance_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
45632.0,曹倩云,花木支行,2.0,6.0,1.0,4.0,1.0,2.0,3.0,9
45635.0,王美燕,曹路支行,5.0,1.0,4.0,1.0,1.0,0.0,1.0,6
45628.0,杨小东,陆家嘴支行,NaN,10.0,NaN,10.0,0.0,0.0,0.0,20
45629.0,杨小东,陆家嘴支行,NaN,18.0,NaN,18.0,0.0,0.0,0.0,36
45630.0,王美燕,陆家嘴支行,1.0,8.0,1.0,8.0,0.0,0.0,0.0,17
45630.0,徐君,营业部,6.0,1.0,6.0,1.0,0.0,0.0,0.0,8
45631.0,杨小东,陆家嘴支行,NaN,2.0,NaN,2.0,0.0,0.0,0.0,4
45632.0,杜星瑶,凌兆分理处,8.0,3.0,4.0,1.0,4.0,2.0,6.0,6
45635.0,徐君,高东支行,7.0,NaN,7.0,NaN,0.0,0.0,0.0,7


In [ ]:
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')


In [ ]:
retail_performance_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-06,曹倩云,花木支行,2.0,6.0,1.0,4.0,1.0,2.0,3.0,9
2024-12-09,王美燕,曹路支行,5.0,1.0,4.0,1.0,1.0,0.0,1.0,6
2024-12-02,杨小东,陆家嘴支行,NaN,10.0,NaN,10.0,0.0,0.0,0.0,20
2024-12-03,杨小东,陆家嘴支行,NaN,18.0,NaN,18.0,0.0,0.0,0.0,36
2024-12-04,王美燕,陆家嘴支行,1.0,8.0,1.0,8.0,0.0,0.0,0.0,17
2024-12-04,徐君,营业部,6.0,1.0,6.0,1.0,0.0,0.0,0.0,8
2024-12-05,杨小东,陆家嘴支行,NaN,2.0,NaN,2.0,0.0,0.0,0.0,4
2024-12-06,杜星瑶,凌兆分理处,8.0,3.0,4.0,1.0,4.0,2.0,6.0,6
2024-12-09,徐君,高东支行,7.0,NaN,7.0,NaN,0.0,0.0,0.0,7


In [ ]:
    today_retail_df=retail_performance_df.loc[T0_Date,:]


In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-26,王美燕,三林支行,2.0,7.0,2.0,7.0,0.0,0.0,0.0,16
2024-12-26,徐君,营业部,2.0,4.0,2.0,4.0,0.0,0.0,0.0,10


In [ ]:
T0_Date='2024-12-18'


In [ ]:
    today_retail_df=retail_performance_df.loc[T0_Date,:]


In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-18,杜星瑶,六里支行,5.0,2.0,5.0,2.0,0.0,0.0,0.0,9
2024-12-18,徐君,杨思支行,10.0,NaN,5.0,NaN,5.0,0.0,5.0,5
2024-12-18,黄旭,王港支行,4.0,20.0,4.0,20.0,0.0,0.0,0.0,44
2024-12-18,王美燕,陆家嘴支行,NaN,6.0,NaN,6.0,0.0,0.0,0.0,12


In [ ]:
    today_retail_df=today_retail_df.fillna(0)


In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-18,杜星瑶,六里支行,5.0,2.0,5.0,2.0,0.0,0.0,0.0,9
2024-12-18,徐君,杨思支行,10.0,0.0,5.0,0.0,5.0,0.0,5.0,5
2024-12-18,黄旭,王港支行,4.0,20.0,4.0,20.0,0.0,0.0,0.0,44
2024-12-18,王美燕,陆家嘴支行,0.0,6.0,0.0,6.0,0.0,0.0,0.0,12


In [ ]:
T0_Date='2024-12-23'


In [ ]:
    yesterday_retail_performance=daily_report[['外拓双算数']]

    yesterday_retail_performance.columns=["鑫e贷总授信（A/B款）_昨日外拓双算数"]

    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-23,王美燕,陆家嘴支行,2.0,NaN,2.0,NaN,0.0,0.0,0.0,2


In [ ]:
T0_Date='2024-12-18'


In [ ]:
    today_retail_df=retail_performance_df.loc[T0_Date,:]


In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-18,杜星瑶,六里支行,5.0,2.0,5.0,2.0,0.0,0.0,0.0,9
2024-12-18,徐君,杨思支行,10.0,NaN,5.0,NaN,5.0,0.0,5.0,5
2024-12-18,黄旭,王港支行,4.0,20.0,4.0,20.0,0.0,0.0,0.0,44
2024-12-18,王美燕,陆家嘴支行,NaN,6.0,NaN,6.0,0.0,0.0,0.0,12


In [ ]:
    today_retail_df=today_retail_df.fillna(0)


In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
外拓日期,,,,,,,,,,
2024-12-18,杜星瑶,六里支行,5.0,2.0,5.0,2.0,0.0,0.0,0.0,9
2024-12-18,徐君,杨思支行,10.0,0.0,5.0,0.0,5.0,0.0,5.0,5
2024-12-18,黄旭,王港支行,4.0,20.0,4.0,20.0,0.0,0.0,0.0,44
2024-12-18,王美燕,陆家嘴支行,0.0,6.0,0.0,6.0,0.0,0.0,0.0,12


In [ ]:
    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nA款授信（户）','本场合计\nB款授信（户）','其中本人\nA款授信（户）','其中本人\nB款授信（户）']]


In [ ]:
today_retail_df

,客户经理姓名,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）
外拓日期,,,,,
2024-12-18,杜星瑶,5.0,2.0,5.0,2.0
2024-12-18,徐君,10.0,0.0,5.0,0.0
2024-12-18,黄旭,4.0,20.0,4.0,20.0
2024-12-18,王美燕,0.0,6.0,0.0,6.0


In [ ]:
    today_retail_df.loc[:,"鑫e贷总授信_今日协外拓双算数"]=today_retail_df.loc[:,"其本场合计\nA款授信（户）"]-today_retail_df.loc[:,"其中本人\nA款授信（户）"]+
                        today_retail_df.loc[:,"其本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]+
    

SyntaxError: invalid syntax (<ipython-input-209-a145342d8072>, line 1)

In [ ]:
    today_retail_df.loc[:,"鑫e贷总授信_今日协外拓双算数"]=today_retail_df.loc[:,"其本场合计\nA款授信（户）"]-today_retail_df.loc[:,"其中本人\nA款授信（户）"]+
                        today_retail_df.loc[:,"其本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]

SyntaxError: invalid syntax (<ipython-input-210-70e59aaad207>, line 1)

In [ ]:
    today_retail_df.loc[:,"鑫e贷总授信_今日协外拓双算数"]=today_retail_df.loc[:,"其本场合计\nA款授信（户）"]-today_retail_df.loc[:,"其中本人\nA款授信（户）"]+today_retail_df.loc[:,"其本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]


KeyError: '其本场合计\nA款授信（户）'

In [ ]:
    today_retail_df.loc[:,"鑫e贷总授信_今日外拓双算数"]=today_retail_df.loc[:,"本场合计\nA款授信（户）"]-today_retail_df.loc[:,"其中本人\nA款授信（户）"]+today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]


d:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [ ]:
today_retail_df

,客户经理姓名,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,鑫e贷总授信_今日外拓双算数
外拓日期,,,,,,
2024-12-18,杜星瑶,5.0,2.0,5.0,2.0,0.0
2024-12-18,徐君,10.0,0.0,5.0,0.0,5.0
2024-12-18,黄旭,4.0,20.0,4.0,20.0,0.0
2024-12-18,王美燕,0.0,6.0,0.0,6.0,0.0


In [ ]:
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷总授信_今日外拓双算数']]


In [ ]:
today_retail_result

,客户经理姓名,鑫e贷总授信_今日外拓双算数
外拓日期,,
2024-12-18,杜星瑶,0.0
2024-12-18,徐君,5.0
2024-12-18,黄旭,0.0
2024-12-18,王美燕,0.0


In [ ]:
    today_retail_result=today_retail_result.set_index('客户经理姓名	',drop=True)


KeyError: "None of ['客户经理姓名\\t'] are in the columns"

In [ ]:
    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)


In [ ]:
today_retail_result

,鑫e贷总授信_今日外拓双算数
客户经理姓名,
杜星瑶,0.0
徐君,5.0
黄旭,0.0
王美燕,0.0


In [ ]:
result

,鑫e贷总授信（A/B款）_指标,鑫e贷总授信（A/B款）_昨日本月,鑫e贷总授信（A/B款）_报表数
许闻多,-,4,4
宋丽凤,-,0,1
孙仰阳,-,1,1
瞿逸程,-,0,0
顾伟丽,NaN,2,2
马越骋,-,1,1
陈名,-,2,2
万华,-,3,5
茅敏艳,-,2,2
钱潇伟,-,1,1


In [ ]:
yesterday_retail_performance

,鑫e贷总授信（A/B款）_昨日外拓双算数
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)


In [ ]:
newest_result

,鑫e贷总授信（A/B款）_昨日外拓双算数,鑫e贷总授信_今日外拓双算数
许闻多,NaN,NaN
宋丽凤,NaN,NaN
孙仰阳,NaN,NaN
瞿逸程,NaN,NaN
顾伟丽,NaN,NaN
马越骋,NaN,NaN
陈名,NaN,NaN
万华,NaN,NaN
茅敏艳,NaN,NaN
钱潇伟,NaN,NaN


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷总授信（A/B款）_指标']]
    
    #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷总授信（A/B款）_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)

    
    #第三个结果(报表完成数）

    client_manager_df=pd.read_excel(data_path+'\\'+client_manager_data)

    client_manager_df=client_manager_df.set_index('kehujinglixingm',drop=True)

    monthly_shouxin=client_manager_df[['benyueshouxinrenshu']]

    manager_total_shouxin = monthly_shouxin.groupby(monthly_shouxin.index).sum()

    result.loc[:,"鑫e贷总授信（A/B款）_报表数"]=manager_total_shouxin


    #第四个结果(外拓双算数)

    #提取昨日协同外拓数

    yesterday_retail_performance=daily_report[['外拓双算数']]

    yesterday_retail_performance.columns=["鑫e贷总授信（A/B款）_昨日外拓双算数"]

    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nA款授信（户）','本场合计\nB款授信（户）','其中本人\nA款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷总授信（A/B款）__今日外拓双算数"]=today_retail_df.loc[:,"本场合计\nA款授信（户）"]-today_retail_df.loc[:,"其中本人\nA款授信（户）"]+today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷总授信_今日外拓双算数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


KeyError: "['鑫e贷总授信_今日外拓双算数'] not in index"

In [ ]:
    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nA款授信（户）','本场合计\nB款授信（户）','其中本人\nA款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷总授信（A/B款）__今日外拓双算数"]=today_retail_df.loc[:,"本场合计\nA款授信（户）"]-today_retail_df.loc[:,"其中本人\nA款授信（户）"]+today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    

In [ ]:
today_retail_df

,客户经理姓名,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,鑫e贷总授信（A/B款）__今日外拓双算数
外拓日期,,,,,,
2024-12-18,杜星瑶,5.0,2.0,5.0,2.0,0.0
2024-12-18,徐君,10.0,0.0,5.0,0.0,5.0
2024-12-18,黄旭,4.0,20.0,4.0,20.0,0.0
2024-12-18,王美燕,0.0,6.0,0.0,6.0,0.0


In [ ]:
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷总授信（A/B款）__今日外拓双算数']]


In [ ]:
today_retail_result

,客户经理姓名,鑫e贷总授信（A/B款）__今日外拓双算数
外拓日期,,
2024-12-18,杜星瑶,0.0
2024-12-18,徐君,5.0
2024-12-18,黄旭,0.0
2024-12-18,王美燕,0.0


In [ ]:
    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)


In [ ]:
    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)


In [ ]:
newest_result

,鑫e贷总授信（A/B款）_昨日外拓双算数,鑫e贷总授信（A/B款）__今日外拓双算数
许闻多,NaN,NaN
宋丽凤,NaN,NaN
孙仰阳,NaN,NaN
瞿逸程,NaN,NaN
顾伟丽,NaN,NaN
马越骋,NaN,NaN
陈名,NaN,NaN
万华,NaN,NaN
茅敏艳,NaN,NaN
钱潇伟,NaN,NaN


In [ ]:
    newest_result.loc[:, "鑫e贷总授信（A/B款）_外拓双算数"] = (
        newest_result.loc[:, "	鑫e贷总授信（A/B款）_昨日外拓双算数"].fillna(0) +
        newest_result.loc[:, "鑫e贷总授信（A/B款）__今日外拓双算数"].fillna(0)
    )


KeyError: '\t鑫e贷总授信（A/B款）_昨日外拓双算数'

In [ ]:
    newest_result.loc[:, "鑫e贷总授信（A/B款）_外拓双算数"] = (
        newest_result.loc[:, "鑫e贷总授信（A/B款）_昨日外拓双算数"].fillna(0) +
        newest_result.loc[:, "鑫e贷总授信（A/B款）__今日外拓双算数"].fillna(0)
    )

In [ ]:
newest_result

,鑫e贷总授信（A/B款）_昨日外拓双算数,鑫e贷总授信（A/B款）__今日外拓双算数,鑫e贷总授信（A/B款）_外拓双算数
许闻多,NaN,NaN,0.0
宋丽凤,NaN,NaN,0.0
孙仰阳,NaN,NaN,0.0
瞿逸程,NaN,NaN,0.0
顾伟丽,NaN,NaN,0.0
马越骋,NaN,NaN,0.0
陈名,NaN,NaN,0.0
万华,NaN,NaN,0.0
茅敏艳,NaN,NaN,0.0
钱潇伟,NaN,NaN,0.0


In [ ]:
    result.loc[:,"鑫e贷总授信（A/B款）_外拓双算数"]=newest_result.loc[:, "鑫e贷总授信（A/B款）_外拓双算数"]


In [ ]:
result

,鑫e贷总授信（A/B款）_指标,鑫e贷总授信（A/B款）_昨日本月,鑫e贷总授信（A/B款）_报表数,鑫e贷总授信（A/B款）_外拓双算数
许闻多,-,4,4,0.0
宋丽凤,-,0,1,0.0
孙仰阳,-,1,1,0.0
瞿逸程,-,0,0,0.0
顾伟丽,NaN,2,2,0.0
马越骋,-,1,1,0.0
陈名,-,2,2,0.0
万华,-,3,5,0.0
茅敏艳,-,2,2,0.0
钱潇伟,-,1,1,0.0


In [ ]:

    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('fangkuanriq')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

    result.loc[:,"鑫e贷放款_自然流量完成数"]=netural_result

    result.loc[:,"鑫e贷放款_自然流量完成数"]=result.loc[:,"鑫e贷放款_自然流量完成数"]/10000



KeyError: "None of ['fangkuanriq'] are in the columns"

In [ ]:
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]

    # 筛选出 yingxiaokehujingl 和 jingdiaokehujingl 不相等的行

    netural_df_filtered = netural_df[netural_df['yingxiaokehujingl'] != netural_df['jingdiaokehujingl']]

    netural_result= netural_df_filtered.groupby('jingdiaokehujingl', as_index=False)['fangkuanjine'].sum()
    
    netural_result=netural_result.set_index('jingdiaokehujingl',drop=True)

KeyError: "['jingdiaokehujingl', 'fangkuanjine'] not in index"

In [ ]:
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]


In [ ]:
filtered_df

,shengqingshijian,1,1.1,shouxintongguoshijian,jingdiaowanchengshij,1.2,1.3,1.4,1.5,1.6,...,1.17,1.18,1.19,1.20,jingdiaokehujingli,1.21,1.22,1.23,1.24,1.25
qianyueshijian,,,,,,,,,,,,,,,,,,,,,
2024-12-12 12:10:51,2024-12-12 11:59:57,49.8,4.806,2024-12-12 12:01:59,2024-12-12 12:07:39,否,NaN,NaN,NaN,0.0,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,曹倩云,83960,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-04 09:53:48,2024-12-04 09:35:02,49.8,4.806,2024-12-04 09:47:30,2024-12-04 09:42:27,否,NaN,NaN,NaN,0.0,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,王美燕,81606,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-05 09:06:23,2024-11-29 17:52:44,28.5,7.506,2024-11-29 17:54:46,2024-12-04 08:53:43,是,3.0,28.5,28.5,28.5,...,10002.0,业务处理中心,-,-,黄旭,53465,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-13 09:11:28,2024-12-12 10:24:39,49.8,7.506,2024-12-12 10:30:43,2024-12-13 09:03:39,是,1.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,黄旭,53465,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-13 15:18:46,2024-12-13 11:31:15,49.8,7.506,2024-12-13 11:31:51,2024-12-13 13:56:18,是,2.0,40.0,40.0,40.0,...,10002.0,业务处理中心,-,-,杜星瑶,50419,31297,浦东分行本部,31298,浦东分行财务会计部
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 16:56:45,2024-12-03 14:33:39,49.8,5.508,2024-12-03 15:06:56,2024-12-03 15:01:53,是,1.0,20.0,20.0,20.0,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-13 14:05:20,2024-12-09 18:20:52,33.1,5.508,2024-12-09 18:22:53,2024-12-13 14:02:55,是,1.0,10.0,10.0,10.0,...,10002.0,业务处理中心,-,-,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-07 20:52:22,2024-09-06 16:53:50,49.8,4.806,2024-09-06 16:55:46,2024-12-07 09:14:21,否,NaN,NaN,NaN,0.0,...,10002.0,业务处理中心,-,-,曹倩云,83960,31297,浦东分行本部,31298,浦东分行财务会计部


In [ ]:
    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl','fangkuanjine']]


KeyError: "['jingdiaokehujingl', 'fangkuanjine'] not in index"

In [ ]:
    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]


In [ ]:
filtered_df

,shengqingshijian,1,1.1,shouxintongguoshijian,jingdiaowanchengshij,1.2,1.3,1.4,1.5,1.6,...,1.17,1.18,1.19,1.20,jingdiaokehujingli,1.21,1.22,1.23,1.24,1.25
qianyueshijian,,,,,,,,,,,,,,,,,,,,,
2024-12-12 12:10:51,2024-12-12 11:59:57,49.8,4.806,2024-12-12 12:01:59,2024-12-12 12:07:39,否,NaN,NaN,NaN,0.0,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,曹倩云,83960,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-04 09:53:48,2024-12-04 09:35:02,49.8,4.806,2024-12-04 09:47:30,2024-12-04 09:42:27,否,NaN,NaN,NaN,0.0,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,王美燕,81606,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-05 09:06:23,2024-11-29 17:52:44,28.5,7.506,2024-11-29 17:54:46,2024-12-04 08:53:43,是,3.0,28.5,28.5,28.5,...,10002.0,业务处理中心,-,-,黄旭,53465,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-13 09:11:28,2024-12-12 10:24:39,49.8,7.506,2024-12-12 10:30:43,2024-12-13 09:03:39,是,1.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,黄旭,53465,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-13 15:18:46,2024-12-13 11:31:15,49.8,7.506,2024-12-13 11:31:51,2024-12-13 13:56:18,是,2.0,40.0,40.0,40.0,...,10002.0,业务处理中心,-,-,杜星瑶,50419,31297,浦东分行本部,31298,浦东分行财务会计部
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-03 16:56:45,2024-12-03 14:33:39,49.8,5.508,2024-12-03 15:06:56,2024-12-03 15:01:53,是,1.0,20.0,20.0,20.0,...,31297.0,浦东分行本部,31298,浦东分行财务会计部,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-13 14:05:20,2024-12-09 18:20:52,33.1,5.508,2024-12-09 18:22:53,2024-12-13 14:02:55,是,1.0,10.0,10.0,10.0,...,10002.0,业务处理中心,-,-,杨小东,81607,31297,浦东分行本部,31298,浦东分行财务会计部
2024-12-07 20:52:22,2024-09-06 16:53:50,49.8,4.806,2024-09-06 16:55:46,2024-12-07 09:14:21,否,NaN,NaN,NaN,0.0,...,10002.0,业务处理中心,-,-,曹倩云,83960,31297,浦东分行本部,31298,浦东分行财务会计部


In [ ]:
    netural_df=filtered_df[['jingdiaokehujingl','yingxiaokehujingl']]


KeyError: "['jingdiaokehujingl'] not in index"

In [ ]:
    netural_df=filtered_df[['jingdiaokehujingli','yingxiaokehujingl']]


In [ ]:
netural_df

,jingdiaokehujingli,yingxiaokehujingl
qianyueshijian,,
2024-12-12 12:10:51,曹倩云,曹倩云
2024-12-04 09:53:48,王美燕,王美燕
2024-12-05 09:06:23,黄旭,刘丽
2024-12-13 09:11:28,黄旭,NaN
2024-12-13 15:18:46,杜星瑶,田建全
...,...,...
2024-12-03 16:56:45,杨小东,杨小东
2024-12-13 14:05:20,杨小东,陈绪灿
2024-12-07 20:52:22,曹倩云,田建全


In [ ]:
    netural_df=filtered_df[['jingdiaokehujingli','yingxiaokehujingl','fangkuanjine']]


KeyError: "['fangkuanjine'] not in index"

In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
kpi_result

,鑫e贷授信-B款_指标
许闻多,3
宋丽凤,3
孙仰阳,5
瞿逸程,3
顾伟丽,3
马越骋,3
陈名,3
万华,3
茅敏艳,3
钱潇伟,3


In [ ]:

    #第二个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]


In [ ]:

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月
许闻多,3,3
宋丽凤,3,0
孙仰阳,5,1
瞿逸程,3,0
顾伟丽,3,2
马越骋,3,1
陈名,3,2
万华,3,3
茅敏艳,3,1
钱潇伟,3,1


In [ ]:
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

In [ ]:
DD_time

,jingdiaokehujingli
qianyueshijian,
2024-12-12 12:10:51,曹倩云
2024-12-04 09:53:48,王美燕
2024-12-05 09:06:23,黄旭
2024-12-13 09:11:28,黄旭
2024-12-13 15:18:46,杜星瑶
...,...
2024-12-03 16:56:45,杨小东
2024-12-13 14:05:20,杨小东
2024-12-07 20:52:22,曹倩云


In [ ]:
    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']

In [ ]:
DD_result

,index,count
0,杨小东,83
1,黄旭,50
2,王美燕,47
3,李亚,46
4,杜星瑶,33
5,曹倩云,30
6,徐君,18
7,许嘉陆,6
8,费文婷,6
9,万华,5


In [ ]:
    DD_result=DD_result.set_index('index',drop=True)


In [ ]:
DD_result

,count
index,
杨小东,83
黄旭,50
王美燕,47
李亚,46
杜星瑶,33
曹倩云,30
徐君,18
许嘉陆,6
费文婷,6


In [ ]:
    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数
许闻多,3,3,3.0
宋丽凤,3,0,1.0
孙仰阳,5,1,1.0
瞿逸程,3,0,NaN
顾伟丽,3,2,2.0
马越骋,3,1,1.0
陈名,3,2,2.0
万华,3,3,5.0
茅敏艳,3,1,1.0
钱潇伟,3,1,1.0


In [ ]:

    yesterday_retail_performance=daily_report[['协同外拓']]

    yesterday_retail_performance.columns=['鑫e贷总授信_昨日协同外拓','鑫e贷放款_昨日协同外拓','鑫e贷授信-B款_昨日协同外拓']

    yesterday_retail_performance=yesterday_retail_performance[['鑫e贷授信-B款_昨日协同外拓']]
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

KeyError: "None of [Index(['协同外拓'], dtype='object', name=1)] are in the [columns]"

In [ ]:
daily_report

1,指标,昨日,报表完成数,自然流量完成数,线下贷款,调整数,本月,完成率,指标,昨日,...,指标,昨日,报表数,外拓数,调整数,本月,完成率,指标,本月,完成率
许闻多,150,10,63.7,0,NaN,NaN,63.7,0.424667,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
宋丽凤,150,0,61.1,6.4,NaN,NaN,67.5,0.45,-,0,...,3,0,0,NaN,NaN,0,0,2,1,0.5
孙仰阳,200,2,64.4222,13.8,NaN,NaN,78.2222,0.391111,-,0,...,5,0,1,NaN,NaN,1,0.2,3,0,0
瞿逸程,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,0,NaN,NaN,0,0,2,0,0
顾伟丽,150,40,84.526,0,NaN,NaN,84.526,0.563507,NaN,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
马越骋,150,0,0,0,NaN,NaN,0,0,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
陈名,150,0,34.2425,0,NaN,NaN,34.2425,0.228283,-,0,...,3,0,2,NaN,NaN,2,0.666667,2,0,0
万华,150,0,112.5334,0,NaN,NaN,112.5334,0.750223,-,0,...,3,0,3,NaN,NaN,3,1,2,2,1
茅敏艳,150,6,60.5465,17,NaN,NaN,77.5465,0.516977,-,1,...,3,0,1,NaN,NaN,1,0.333333,2,0,0
钱潇伟,150,21,61.7,8,NaN,NaN,69.7,0.464667,-,0,...,3,0,1,NaN,NaN,1,0.333333,2,3,1.5


In [ ]:

    yesterday_retail_performance=daily_report[['外拓数']]

In [ ]:
yesterday_retail_performance

1,外拓数
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']


In [ ]:
yesterday_retail_performance

,鑫e贷授信-B款_昨日外拓数
许闻多,NaN
宋丽凤,NaN
孙仰阳,NaN
瞿逸程,NaN
顾伟丽,NaN
马越骋,NaN
陈名,NaN
万华,NaN
茅敏艳,NaN
钱潇伟,NaN


In [ ]:
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)


In [ ]:
retail_performance_df

,外拓日期,客户经理姓名,协同外拓网点,本场合计\nA款授信（户）,本场合计\nB款授信（户）,其中本人\nA款授信（户）,其中本人\nB款授信（户）,网点A,网点B,本人加,网点加
0,45632.0,曹倩云,花木支行,2.0,6.0,1.0,4.0,1.0,2.0,3.0,9
1,45635.0,王美燕,曹路支行,5.0,1.0,4.0,1.0,1.0,0.0,1.0,6
2,45628.0,杨小东,陆家嘴支行,NaN,10.0,NaN,10.0,0.0,0.0,0.0,20
3,45629.0,杨小东,陆家嘴支行,NaN,18.0,NaN,18.0,0.0,0.0,0.0,36
4,45630.0,王美燕,陆家嘴支行,1.0,8.0,1.0,8.0,0.0,0.0,0.0,17
5,45630.0,徐君,营业部,6.0,1.0,6.0,1.0,0.0,0.0,0.0,8
6,45631.0,杨小东,陆家嘴支行,NaN,2.0,NaN,2.0,0.0,0.0,0.0,4
7,45632.0,杜星瑶,凌兆分理处,8.0,3.0,4.0,1.0,4.0,2.0,6.0,6
8,45635.0,徐君,高东支行,7.0,NaN,7.0,NaN,0.0,0.0,0.0,7
9,45636.0,杨小东,高桥支行,5.0,NaN,5.0,NaN,0.0,0.0,0.0,5


In [ ]:
    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]
    
    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df[['客户经理姓名','协同外拓网点','其中本人\nA款授信（户）','其中本人\nB款授信（户）']]
    
    today_retail_df=today_retail_df.fillna(0)

In [ ]:
today_retail_df

,客户经理姓名,协同外拓网点,其中本人\nA款授信（户）,其中本人\nB款授信（户）
外拓日期,,,,
2024-12-18,杜星瑶,六里支行,5.0,2.0
2024-12-18,徐君,杨思支行,5.0,0.0
2024-12-18,黄旭,王港支行,4.0,20.0
2024-12-18,王美燕,陆家嘴支行,0.0,6.0


In [ ]:
    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]


In [ ]:
today_retail_df

,客户经理姓名,本场合计\nB款授信（户）,其中本人\nB款授信（户）
外拓日期,,,
2024-12-18,杜星瑶,2.0,2.0
2024-12-18,徐君,0.0,0.0
2024-12-18,黄旭,20.0,20.0
2024-12-18,王美燕,6.0,6.0


In [ ]:
    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]


In [ ]:
today_retail_df

,客户经理姓名,本场合计\nB款授信（户）,其中本人\nB款授信（户）,鑫e贷授信-B款__今日外拓数
外拓日期,,,,
2024-12-18,杜星瑶,2.0,2.0,0.0
2024-12-18,徐君,0.0,0.0,0.0
2024-12-18,黄旭,20.0,20.0,0.0
2024-12-18,王美燕,6.0,6.0,0.0


In [ ]:
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]


In [ ]:
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

In [ ]:
newest_result

,鑫e贷授信-B款_昨日外拓数,鑫e贷授信-B款__今日外拓数
许闻多,NaN,NaN
宋丽凤,NaN,NaN
孙仰阳,NaN,NaN
瞿逸程,NaN,NaN
顾伟丽,NaN,NaN
马越骋,NaN,NaN
陈名,NaN,NaN
万华,NaN,NaN
茅敏艳,NaN,NaN
钱潇伟,NaN,NaN


In [ ]:
    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

In [ ]:
newest_result

,鑫e贷授信-B款_昨日外拓数,鑫e贷授信-B款__今日外拓数,鑫e贷授信-B款__外拓数
许闻多,NaN,NaN,0.0
宋丽凤,NaN,NaN,0.0
孙仰阳,NaN,NaN,0.0
瞿逸程,NaN,NaN,0.0
顾伟丽,NaN,NaN,0.0
马越骋,NaN,NaN,0.0
陈名,NaN,NaN,0.0
万华,NaN,NaN,0.0
茅敏艳,NaN,NaN,0.0
钱潇伟,NaN,NaN,0.0


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数
许闻多,3,3,3.0
宋丽凤,3,0,1.0
孙仰阳,5,1,1.0
瞿逸程,3,0,NaN
顾伟丽,3,2,2.0
马越骋,3,1,1.0
陈名,3,2,2.0
万华,3,3,5.0
茅敏艳,3,1,1.0
钱潇伟,3,1,1.0


In [ ]:
    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数
许闻多,3,3,3.0,0.0
宋丽凤,3,0,1.0,0.0
孙仰阳,5,1,1.0,0.0
瞿逸程,3,0,NaN,0.0
顾伟丽,3,2,2.0,0.0
马越骋,3,1,1.0,0.0
陈名,3,2,2.0,0.0
万华,3,3,5.0,0.0
茅敏艳,3,1,1.0,0.0
钱潇伟,3,1,1.0,0.0


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数
许闻多,3,3,3.0,0.0
宋丽凤,3,0,1.0,0.0
孙仰阳,5,1,1.0,0.0
瞿逸程,3,0,NaN,0.0
顾伟丽,3,2,2.0,0.0
马越骋,3,1,1.0,0.0
陈名,3,2,2.0,0.0
万华,3,3,5.0,0.0
茅敏艳,3,1,1.0,0.0
钱潇伟,3,1,1.0,0.0


In [ ]:
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"鑫e贷放款_调整数"]=adjust_number_result

In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷放款_调整数
许闻多,3,3,3.0,0.0,NaN
宋丽凤,3,0,1.0,0.0,NaN
孙仰阳,5,1,1.0,0.0,NaN
瞿逸程,3,0,NaN,0.0,NaN
顾伟丽,3,2,2.0,0.0,NaN
马越骋,3,1,1.0,0.0,NaN
陈名,3,2,2.0,0.0,NaN
万华,3,3,5.0,0.0,NaN
茅敏艳,3,1,1.0,0.0,NaN
钱潇伟,3,1,1.0,0.0,NaN


In [ ]:
adjust_number

,鑫e贷放款_调整数,鑫e贷总授信（A/B款）_调整数,鑫e贷授信-B款_调整数
许闻多,NaN,NaN,NaN
宋丽凤,NaN,NaN,NaN
孙仰阳,NaN,NaN,NaN
瞿逸程,NaN,NaN,NaN
顾伟丽,NaN,NaN,NaN
马越骋,NaN,NaN,NaN
陈名,NaN,NaN,NaN
万华,NaN,NaN,NaN
茅敏艳,NaN,NaN,NaN
钱潇伟,NaN,NaN,NaN


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷放款_调整数
许闻多,3,3,3.0,0.0,NaN
宋丽凤,3,0,1.0,0.0,NaN
孙仰阳,5,1,1.0,0.0,NaN
瞿逸程,3,0,NaN,0.0,NaN
顾伟丽,3,2,2.0,0.0,NaN
马越骋,3,1,1.0,0.0,NaN
陈名,3,2,2.0,0.0,NaN
万华,3,3,5.0,0.0,NaN
茅敏艳,3,1,1.0,0.0,NaN
钱潇伟,3,1,1.0,0.0,NaN


In [ ]:
    result.loc[:,"鑫e贷授信-B款_完成数"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

KeyError: "['鑫e贷授信-B款_调整数'] not in index"

In [ ]:
    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"'鑫e贷授信-B款_调整数"]=adjust_number_result


In [ ]:

    #第六个结果（完成数）

    result.loc[:,"鑫e贷授信-B款_完成数"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

KeyError: "['鑫e贷授信-B款_调整数'] not in index"

In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


    #第三个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result

    #第四个结果（外拓数）

    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]

    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"'鑫e贷授信-B款_调整数"]=adjust_number_result

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,'鑫e贷授信-B款_调整数
许闻多,3,3,3.0,0.0,NaN
宋丽凤,3,0,1.0,0.0,NaN
孙仰阳,5,1,1.0,0.0,NaN
瞿逸程,3,0,NaN,0.0,NaN
顾伟丽,3,2,2.0,0.0,NaN
马越骋,3,1,1.0,0.0,NaN
陈名,3,2,2.0,0.0,NaN
万华,3,3,5.0,0.0,NaN
茅敏艳,3,1,1.0,0.0,NaN
钱潇伟,3,1,1.0,0.0,NaN


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


    #第三个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result

    #第四个结果（外拓数）

    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]

    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"鑫e贷授信-B款_调整数"]=adjust_number_result



d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷授信-B款_调整数
许闻多,3,3,3.0,0.0,NaN
宋丽凤,3,0,1.0,0.0,NaN
孙仰阳,5,1,1.0,0.0,NaN
瞿逸程,3,0,NaN,0.0,NaN
顾伟丽,3,2,2.0,0.0,NaN
马越骋,3,1,1.0,0.0,NaN
陈名,3,2,2.0,0.0,NaN
万华,3,3,5.0,0.0,NaN
茅敏艳,3,1,1.0,0.0,NaN
钱潇伟,3,1,1.0,0.0,NaN


In [ ]:
    result.loc[:,"鑫e贷授信-B款_完成数"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷授信-B款_调整数,鑫e贷授信-B款_完成数
许闻多,3,3,3.0,0.0,NaN,3.0
宋丽凤,3,0,1.0,0.0,NaN,1.0
孙仰阳,5,1,1.0,0.0,NaN,1.0
瞿逸程,3,0,NaN,0.0,NaN,0.0
顾伟丽,3,2,2.0,0.0,NaN,2.0
马越骋,3,1,1.0,0.0,NaN,1.0
陈名,3,2,2.0,0.0,NaN,2.0
万华,3,3,5.0,0.0,NaN,5.0
茅敏艳,3,1,1.0,0.0,NaN,1.0
钱潇伟,3,1,1.0,0.0,NaN,1.0


In [ ]:

    result.loc[:,"鑫e贷授信-B款_本月"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

    result.loc[:, "鑫e贷授信-B款_完成率"] = (result.loc[:, "鑫e贷授信-B款_本月"] /
                             result.loc[:, "鑫e贷授信-B款_指标"]).fillna(0).round(2)
                                   

In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷授信-B款_调整数,鑫e贷授信-B款_完成数,鑫e贷授信-B款_本月,鑫e贷授信-B款_完成率
许闻多,3,3,3.0,0.0,NaN,3.0,3.0,1.00
宋丽凤,3,0,1.0,0.0,NaN,1.0,1.0,0.33
孙仰阳,5,1,1.0,0.0,NaN,1.0,1.0,0.20
瞿逸程,3,0,NaN,0.0,NaN,0.0,0.0,0.00
顾伟丽,3,2,2.0,0.0,NaN,2.0,2.0,0.67
马越骋,3,1,1.0,0.0,NaN,1.0,1.0,0.33
陈名,3,2,2.0,0.0,NaN,2.0,2.0,0.67
万华,3,3,5.0,0.0,NaN,5.0,5.0,1.67
茅敏艳,3,1,1.0,0.0,NaN,1.0,1.0,0.33
钱潇伟,3,1,1.0,0.0,NaN,1.0,1.0,0.33


In [ ]:
    result.loc[:,"鑫e贷授信-B款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷授信-B款_本月"]-result.loc[:,"鑫e贷授信-B款_昨日本月"]


In [ ]:
result

,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日本月,鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷授信-B款_调整数,鑫e贷授信-B款_完成数,鑫e贷授信-B款_本月,鑫e贷授信-B款_完成率,鑫e贷授信-B款_昨日完成数(轧差)
许闻多,3,3,3.0,0.0,NaN,3.0,3.0,1.00,0.0
宋丽凤,3,0,1.0,0.0,NaN,1.0,1.0,0.33,1.0
孙仰阳,5,1,1.0,0.0,NaN,1.0,1.0,0.20,0.0
瞿逸程,3,0,NaN,0.0,NaN,0.0,0.0,0.00,0.0
顾伟丽,3,2,2.0,0.0,NaN,2.0,2.0,0.67,0.0
马越骋,3,1,1.0,0.0,NaN,1.0,1.0,0.33,0.0
陈名,3,2,2.0,0.0,NaN,2.0,2.0,0.67,0.0
万华,3,3,5.0,0.0,NaN,5.0,5.0,1.67,2.0
茅敏艳,3,1,1.0,0.0,NaN,1.0,1.0,0.33,0.0
钱潇伟,3,1,1.0,0.0,NaN,1.0,1.0,0.33,0.0


In [ ]:

    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_调整数','鑫e贷授信-B款','鑫e贷授信-B款_完成率']]
    
    result.to_excel(result_path+'\\'+'鑫e贷授信-B款团队_完成情况.xlsx')


KeyError: "['鑫e贷授信-B款'] not in index"

In [ ]:
    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_调整数','鑫e贷授信-B款本月','鑫e贷授信-B款_完成率']]
    

KeyError: "['鑫e贷授信-B款本月'] not in index"

In [ ]:

    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_调整数','鑫e贷授信-B款_本月','鑫e贷授信-B款_完成率']]
    

In [ ]:
def B_Kuang_Shou_Xin_Team(data_path,yesterday_daily_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


    #第三个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result

    #第四个结果（外拓数）

    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]

    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"鑫e贷授信-B款_调整数"]=adjust_number_result


    #第六个结果（完成数）

    result.loc[:,"鑫e贷授信-B款_本月"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

    result.loc[:, "鑫e贷授信-B款_完成率"] = (result.loc[:, "鑫e贷授信-B款_本月"] /
                             result.loc[:, "鑫e贷授信-B款_指标"]).fillna(0).round(2)
    

    #计算本月和昨日轧差

    result=result.fillna(0)
    
    result.loc[:,"鑫e贷授信-B款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷授信-B款_本月"]-result.loc[:,"鑫e贷授信-B款_昨日本月"]


    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_调整数','鑫e贷授信-B款_本月','鑫e贷授信-B款_完成率']]
    
    final_result.to_excel(result_path+'\\'+'鑫e贷授信-B款团队_完成情况.xlsx')

    print('恭喜米，鑫e贷授信-B款计算完成')

    return final_result


In [ ]:

def B_Kuang_Shou_Xin_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


    #第三个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result

    #第四个结果（外拓数）

    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]

    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"鑫e贷授信-B款_调整数"]=adjust_number_result


    #第六个结果（完成数）

    result.loc[:,"鑫e贷授信-B款_本月"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

    result.loc[:, "鑫e贷授信-B款_完成率"] = (result.loc[:, "鑫e贷授信-B款_本月"] /
                             result.loc[:, "鑫e贷授信-B款_指标"]).fillna(0).round(2)
    

    #计算本月和昨日轧差

    result=result.fillna(0)
    
    result.loc[:,"鑫e贷授信-B款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷授信-B款_本月"]-result.loc[:,"鑫e贷授信-B款_昨日本月"]


    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_调整数','鑫e贷授信-B款_本月','鑫e贷授信-B款_完成率']]
    
    final_result.to_excel(result_path+'\\'+'鑫e贷授信-B款团队_完成情况.xlsx')

    print('恭喜米，鑫e贷授信-B款计算完成')

    return final_result


B_Kuang_Shou_Xin_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


恭喜米，鑫e贷授信-B款计算完成


,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日完成数(轧差),鑫e贷授信-B款_报表数,鑫e贷授信-B款_调整数,鑫e贷授信-B款_本月,鑫e贷授信-B款_完成率
许闻多,3,0.0,3.0,0,3.0,1.00
宋丽凤,3,1.0,1.0,0,1.0,0.33
孙仰阳,5,0.0,1.0,0,1.0,0.20
瞿逸程,3,0.0,0.0,0,0.0,0.00
顾伟丽,3,0.0,2.0,0,2.0,0.67
马越骋,3,0.0,1.0,0,1.0,0.33
陈名,3,0.0,2.0,0,2.0,0.67
万华,3,2.0,5.0,0,5.0,1.67
茅敏艳,3,0.0,1.0,0,1.0,0.33
钱潇伟,3,0.0,1.0,0,1.0,0.33


In [ ]:
def B_Kuang_Shou_Xin_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


    #第三个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result

    #第四个结果（外拓数）

    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]

    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"鑫e贷授信-B款_调整数"]=adjust_number_result


    #第六个结果（完成数）

    result.loc[:,"鑫e贷授信-B款_本月"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

    result.loc[:, "鑫e贷授信-B款_完成率"] = (result.loc[:, "鑫e贷授信-B款_本月"] /
                             result.loc[:, "鑫e贷授信-B款_指标"]).fillna(0).round(2)
    

    #计算本月和昨日轧差

    result=result.fillna(0)
    
    result.loc[:,"鑫e贷授信-B款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷授信-B款_本月"]-result.loc[:,"鑫e贷授信-B款_昨日本月"]


    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数'
                        '鑫e贷授信-B款_调整数','鑫e贷授信-B款_本月','鑫e贷授信-B款_完成率']]
    
    final_result.to_excel(result_path+'\\'+'鑫e贷授信-B款团队_完成情况.xlsx')

    print('恭喜米，鑫e贷授信-B款计算完成')

    return final_result


B_Kuang_Shou_Xin_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


KeyError: "['鑫e贷授信-B款_外拓数鑫e贷授信-B款_调整数'] not in index"

In [ ]:
def B_Kuang_Shou_Xin_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['鑫e贷授信-B款_指标']]
    

    #第二个结果昨日完成数
        #第二个结果（昨日完成数），提取完成数的列表
    yesterday_finished=daily_report[['本月']]

    yesterday_finished.columns=['鑫e贷放款_昨日本月','鑫e贷总授信（A/B款）_昨日本月','鑫e贷授信-B款_昨日本月','财富转介户数_昨日本月']

    yesterday_finished_result=yesterday_finished[['鑫e贷授信-B款_昨日本月']]
    
    result=pd.merge(kpi_result,yesterday_finished_result,right_index=True,left_index=True)


    #第三个结果（报表数）
    
    type_B_df=pd.read_excel(data_path+'\\'+type_B_data,sheet_name="鑫e贷大额客户授信数据")

    type_B_df=type_B_df.set_index('qianyueshijian')

    type_B_df.index=pd.to_datetime(type_B_df.index)

    year = pd.to_datetime(T0_Date).year  # 提取年份

    month = pd.to_datetime(T0_Date).month  # 提取月份

    filtered_df = type_B_df[(type_B_df.index.year == year) & (type_B_df.index.month == month)]

    DD_time=filtered_df[['jingdiaokehujingli']]

    DD_result = DD_time['jingdiaokehujingli'].value_counts().reset_index()

    DD_result.columns=['index','count']
    
    DD_result=DD_result.set_index('index',drop=True)

    result.loc[:,"鑫e贷授信-B款_报表数"]=DD_result

    #第四个结果（外拓数）

    yesterday_retail_performance=daily_report[['外拓数']]

    yesterday_retail_performance.columns=['鑫e贷授信-B款_昨日外拓数']
    
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data)

    retail_performance_df=retail_performance_df.set_index('外拓日期',drop=True)
    
    retail_performance_df.index=pd.to_datetime(retail_performance_df.index, unit='d', origin='1899-12-30')

    today_retail_df=retail_performance_df.loc[T0_Date,:]

    # today_retail_df = today_retail_df.to_frame().T

    today_retail_df=today_retail_df.fillna(0)

    today_retail_df=today_retail_df[['客户经理姓名','本场合计\nB款授信（户）','其中本人\nB款授信（户）']]

    today_retail_df.loc[:,"鑫e贷授信-B款__今日外拓数"]=today_retail_df.loc[:,"本场合计\nB款授信（户）"]-today_retail_df.loc[:,"其中本人\nB款授信（户）"]
    
    today_retail_result=today_retail_df[['客户经理姓名','鑫e贷授信-B款__今日外拓数']]

    today_retail_result=today_retail_result.set_index('客户经理姓名',drop=True)

    newest_result=pd.concat([yesterday_retail_performance,today_retail_result],axis=1)

    newest_result.loc[:, "鑫e贷授信-B款__外拓数"] = (
        newest_result.loc[:, "鑫e贷授信-B款_昨日外拓数"].fillna(0) +
        newest_result.loc[:, "鑫e贷授信-B款__今日外拓数"].fillna(0)
    )

    result.loc[:,"鑫e贷授信-B款_外拓数"]=newest_result.loc[:, "鑫e贷授信-B款__外拓数"]

    #第五个结果（调整数）
    adjust_number=daily_report[['调整数']]

    adjust_number.columns=['鑫e贷放款_调整数','鑫e贷总授信（A/B款）_调整数','鑫e贷授信-B款_调整数']

    adjust_number_result=adjust_number[['鑫e贷授信-B款_调整数']]

    result.loc[:,"鑫e贷授信-B款_调整数"]=adjust_number_result


    #第六个结果（完成数）

    result.loc[:,"鑫e贷授信-B款_本月"]=result[['鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                                '鑫e贷授信-B款_调整数']].sum(axis=1)

    result.loc[:, "鑫e贷授信-B款_完成率"] = (result.loc[:, "鑫e贷授信-B款_本月"] /
                             result.loc[:, "鑫e贷授信-B款_指标"]).fillna(0).round(2)
    

    #计算本月和昨日轧差

    result=result.fillna(0)
    
    result.loc[:,"鑫e贷授信-B款_昨日完成数(轧差)"]=result.loc[:,"鑫e贷授信-B款_本月"]-result.loc[:,"鑫e贷授信-B款_昨日本月"]


    final_result=result[['鑫e贷授信-B款_指标','鑫e贷授信-B款_昨日完成数(轧差)',
                        '鑫e贷授信-B款_报表数','鑫e贷授信-B款_外拓数',
                        '鑫e贷授信-B款_调整数','鑫e贷授信-B款_本月','鑫e贷授信-B款_完成率']]
    
    final_result.to_excel(result_path+'\\'+'鑫e贷授信-B款团队_完成情况.xlsx')

    print('恭喜米，鑫e贷授信-B款计算完成')

    return final_result


B_Kuang_Shou_Xin_Team(data_path,yesterday_team_report,
department_list,client_manager_data,T0_Date,type_B_data,result_path)

恭喜米，鑫e贷授信-B款计算完成


,鑫e贷授信-B款_指标,鑫e贷授信-B款_昨日完成数(轧差),鑫e贷授信-B款_报表数,鑫e贷授信-B款_外拓数,鑫e贷授信-B款_调整数,鑫e贷授信-B款_本月,鑫e贷授信-B款_完成率
许闻多,3,0.0,3.0,0.0,0,3.0,1.00
宋丽凤,3,1.0,1.0,0.0,0,1.0,0.33
孙仰阳,5,0.0,1.0,0.0,0,1.0,0.20
瞿逸程,3,0.0,0.0,0.0,0,0.0,0.00
顾伟丽,3,0.0,2.0,0.0,0,2.0,0.67
马越骋,3,0.0,1.0,0.0,0,1.0,0.33
陈名,3,0.0,2.0,0.0,0,2.0,0.67
万华,3,2.0,5.0,0.0,0,5.0,1.67
茅敏艳,3,0.0,1.0,0.0,0,1.0,0.33
钱潇伟,3,0.0,1.0,0.0,0,1.0,0.33


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['财富转介户数_指标']]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
kpi_result

,财富转介户数_指标
许闻多,2
宋丽凤,2
孙仰阳,3
瞿逸程,2
顾伟丽,2
马越骋,2
陈名,2
万华,2
茅敏艳,2
钱潇伟,2


In [ ]:
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data,sheet_name="财富业务转介报送")


In [ ]:
retail_performance_df

,客户经理姓名,转介客户姓名,落地理财经理,转介产品名称,金额,Unnamed: 5
0,许闻多,滕俊杰,刘一谨,天天鑫A,10万,NaN
1,许闻多,孙湖宁,殷凤,苏盈1号,42万,NaN
2,万华,黄鑫,王喆,苏盈1号,1w,NaN
3,万华,孙洁,裴乐园,天天盈A,50w,NaN
4,杨小东,潘波,28690,90天代销理财,1,NaN
5,杨小东,钟扬怡,28690,90天代销理财,1,NaN
6,杨小东,杨小东,28690,90天理财代销,1,NaN
7,杜星瑶,周金初,李书吟,苏银理财恒源1年定开32期A,1,NaN
8,曹倩云,袁雪红,01178,苏银天添鑫,1.5,补登11月
9,杨小东,周洪,28690,90天理财代销,1,NaN


In [ ]:
    retail_performance_df=retail_performance_df.set_index('客户经理姓名',drop=True)

In [ ]:
retail_performance_df

,转介客户姓名,落地理财经理,转介产品名称,金额,Unnamed: 5
客户经理姓名,,,,,
许闻多,滕俊杰,刘一谨,天天鑫A,10万,NaN
许闻多,孙湖宁,殷凤,苏盈1号,42万,NaN
万华,黄鑫,王喆,苏盈1号,1w,NaN
万华,孙洁,裴乐园,天天盈A,50w,NaN
杨小东,潘波,28690,90天代销理财,1,NaN
杨小东,钟扬怡,28690,90天代销理财,1,NaN
杨小东,杨小东,28690,90天理财代销,1,NaN
杜星瑶,周金初,李书吟,苏银理财恒源1年定开32期A,1,NaN
曹倩云,袁雪红,01178,苏银天添鑫,1.5,补登11月


In [ ]:
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data,sheet_name="财富业务转介报送")


In [ ]:
retail_performance_df

,客户经理姓名,转介客户姓名,落地理财经理,转介产品名称,金额,Unnamed: 5
0,许闻多,滕俊杰,刘一谨,天天鑫A,10万,NaN
1,许闻多,孙湖宁,殷凤,苏盈1号,42万,NaN
2,万华,黄鑫,王喆,苏盈1号,1w,NaN
3,万华,孙洁,裴乐园,天天盈A,50w,NaN
4,杨小东,潘波,28690,90天代销理财,1,NaN
5,杨小东,钟扬怡,28690,90天代销理财,1,NaN
6,杨小东,杨小东,28690,90天理财代销,1,NaN
7,杜星瑶,周金初,李书吟,苏银理财恒源1年定开32期A,1,NaN
8,曹倩云,袁雪红,01178,苏银天添鑫,1.5,补登11月
9,杨小东,周洪,28690,90天理财代销,1,NaN


In [ ]:
    DD_result = DD_time['客户经理姓名'].value_counts().reset_index()


KeyError: '客户经理姓名'

In [ ]:
    DD_result = retail_performance_df['客户经理姓名'].value_counts().reset_index()


In [ ]:
DD_result

,index,客户经理姓名
0,孙仰阳,10
1,杨小东,6
2,王美燕,6
3,宋丽凤,4
4,许闻多,3
5,万华,3
6,黄旭,3
7,钱潇伟,3
8,李亚,3
9,徐君,3


In [ ]:
    DD_result.columns=['index','count']

In [ ]:
DD_result

,index,count
0,孙仰阳,10
1,杨小东,6
2,王美燕,6
3,宋丽凤,4
4,许闻多,3
5,万华,3
6,黄旭,3
7,钱潇伟,3
8,李亚,3
9,徐君,3


In [ ]:
    DD_result=DD_result.set_index('index',drop=True)

In [ ]:
DD_result

,count
index,
孙仰阳,10
杨小东,6
王美燕,6
宋丽凤,4
许闻多,3
万华,3
黄旭,3
钱潇伟,3
李亚,3


In [ ]:
    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data,sheet_name="财富业务转介报送")

    DD_result = retail_performance_df['客户经理姓名'].value_counts().reset_index()

    DD_result.columns=['index','财富转介户数_本月']

    DD_result=DD_result.set_index('index',drop=True)

In [ ]:
DD_result

,财富转介户数_本月
index,
孙仰阳,10
杨小东,6
王美燕,6
宋丽凤,4
许闻多,3
万华,3
黄旭,3
钱潇伟,3
李亚,3


In [ ]:
kpi_result

,财富转介户数_指标
许闻多,2
宋丽凤,2
孙仰阳,3
瞿逸程,2
顾伟丽,2
马越骋,2
陈名,2
万华,2
茅敏艳,2
钱潇伟,2


In [ ]:
    result=pd.merge(kpi_result,DD_result,right_index=True,left_index=True)

In [ ]:
result

,财富转介户数_指标,财富转介户数_本月
许闻多,2,3
宋丽凤,2,4
孙仰阳,3,10
瞿逸程,2,2
陈名,2,2
万华,2,3
茅敏艳,2,2
钱潇伟,2,3
杨小东,5,6
王美燕,3,6


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#数据路径设置

result_path=r'C:\Users\Wesle\Desktop\srcb_daily_report\result'

data_path=r'C:\Users\Wesle\Desktop\srcb_daily_report\原始数据'

#部门归属表文件名
department_list='员工部门归属表.xlsx'

#昨日日报表文件名
yesterday_daily_report='网点鑫e贷月度指标完成情况1226.xlsx'

yesterday_team_report='团队非按揭业绩日报20241218.xlsx'

#【浦东分行鑫e贷】客户经理营销数据文件名
client_manager_data='【浦东分行鑫e贷】客户经理营销数据_2024-12-18.xlsx'

retail_performance_data='零售市场部协同外拓及理财转介业绩报送-5.xlsx'

type_B_data='【浦东分行鑫e贷】鑫e贷b款明细_2024-12-18.xlsx'

T0_Date='2024-12-18'


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['财富转介户数_指标']]

    #第二个指标（本月）

    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data,sheet_name="财富业务转介报送")

    DD_result = retail_performance_df['客户经理姓名'].value_counts().reset_index()

    DD_result.columns=['index','财富转介户数_本月']

    DD_result=DD_result.set_index('index',drop=True)

    result=pd.merge(kpi_result,DD_result,right_index=True,left_index=True)

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
result

,财富转介户数_指标,财富转介户数_本月
许闻多,2,2
宋丽凤,2,3
孙仰阳,3,4
万华,2,2
钱潇伟,2,3
杨小东,5,6
王美燕,3,6
杜星瑶,2,1
曹倩云,2,2
黄旭,2,1


In [ ]:
    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['财富转介户数_指标']]

d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
kpi_result

,财富转介户数_指标
许闻多,2
宋丽凤,2
孙仰阳,3
瞿逸程,2
顾伟丽,2
马越骋,2
陈名,2
万华,2
茅敏艳,2
钱潇伟,2


In [ ]:
    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['财富转介户数_指标']]

    #第二个指标（本月）

    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data,sheet_name="财富业务转介报送")

    DD_result = retail_performance_df['客户经理姓名'].value_counts().reset_index()

    DD_result.columns=['index','财富转介户数_本月']

    DD_result=DD_result.set_index('index',drop=True)

    result=pd.concat([kpi_result,DD_result],axis=1)

In [ ]:
result

,财富转介户数_指标,财富转介户数_本月
许闻多,2,2.0
宋丽凤,2,3.0
孙仰阳,3,4.0
瞿逸程,2,NaN
顾伟丽,2,NaN
马越骋,2,NaN
陈名,2,NaN
万华,2,2.0
茅敏艳,2,NaN
钱潇伟,2,3.0


In [ ]:
    result.loc[:,"财富转介户数_完成率"]=result.loc[:,"财富转介户数_本月"]/esult.loc[:,"财富转介户数_指标"]

NameError: name 'esult' is not defined

In [ ]:
    result.loc[:,"财富转介户数_完成率"]=result.loc[:,"财富转介户数_本月"]/result.loc[:,"财富转介户数_指标"]

In [ ]:
result

,财富转介户数_指标,财富转介户数_本月,财富转介户数_完成率
许闻多,2,2.0,1.0
宋丽凤,2,3.0,1.5
孙仰阳,3,4.0,1.333333
瞿逸程,2,NaN,NaN
顾伟丽,2,NaN,NaN
马越骋,2,NaN,NaN
陈名,2,NaN,NaN
万华,2,2.0,1.0
茅敏艳,2,NaN,NaN
钱潇伟,2,3.0,1.5


In [ ]:

def WM_Bussiness_Number_Team(ata_path,yesterday_team_report,retail_performance_data):

    yesterday_daily_report_df=pd.read_excel(data_path+'\\'+yesterday_team_report)

    yesterday_daily_report_df=yesterday_daily_report_df.iloc[1:,:]

    new_columns = yesterday_daily_report_df.iloc[0,2:].dropna()  # 选择第一行作为新的列名

    index = yesterday_daily_report_df.iloc[1:, 1:2].dropna()  # 假设这是你提取出来的列数据
    index_list = index.squeeze().tolist()  # 将提取出的列转换为列表

    daily_report=yesterday_daily_report_df.iloc[1:21, 2:33]

    daily_report.index=index_list

    daily_report.columns=new_columns

    daily_report=daily_report.drop('合计')

    daily_report=daily_report.iloc[:,5:]

    #第一个结果（指标）提取昨日报表指标列
    kpi=daily_report[['指标']]

    kpi.columns=['鑫e贷放款_指标','鑫e贷总授信（A/B款）_指标','鑫e贷授信-B款_指标','财富转介户数_指标']
    
    kpi_result=kpi[['财富转介户数_指标']]

    #第二个指标（本月）

    retail_performance_df=pd.read_excel(data_path+'\\'+retail_performance_data,sheet_name="财富业务转介报送")

    DD_result = retail_performance_df['客户经理姓名'].value_counts().reset_index()

    DD_result.columns=['index','财富转介户数_本月']

    DD_result=DD_result.set_index('index',drop=True)

    result=pd.concat([kpi_result,DD_result],axis=1)
    
    result.loc[:,"财富转介户数_完成率"]=result.loc[:,"财富转介户数_本月"]/result.loc[:,"财富转介户数_指标"]

    result.to_excel(result_path+'\\'+'财富转介户数团队_完成情况.xlsx')

    print('恭喜米，财富转介户数团队计算完成')

    return result





In [ ]:
WM_Bussiness_Number_Team(ata_path,yesterday_team_report,retail_performance_data)

NameError: name 'ata_path' is not defined

In [ ]:
WM_Bussiness_Number_Team(data_path,yesterday_team_report,retail_performance_data)

恭喜米，财富转介户数团队计算完成


d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,财富转介户数_指标,财富转介户数_本月,财富转介户数_完成率
许闻多,2,2.0,1.0
宋丽凤,2,3.0,1.5
孙仰阳,3,4.0,1.333333
瞿逸程,2,NaN,NaN
顾伟丽,2,NaN,NaN
马越骋,2,NaN,NaN
陈名,2,NaN,NaN
万华,2,2.0,1.0
茅敏艳,2,NaN,NaN
钱潇伟,2,3.0,1.5
